
<img src="logos/Icos_cp_Logo_RGB.svg" width="400" align="left"/>
<!-- <img src="logos/magnifying_glass_over_globe.png" width="80" align="right"/> -->
<!-- <img src="logos/magnifying-glass-color.png" width="85" align="right"/> -->
<!-- <img src="logos/magnifying-galss-33363_960_720.png" width="50" align="right"/> -->
<img src="logos/magnifying_glass_black.png" width="85" align="right"/>




<a id='introduction'></a>

# Plotting ICOS Level-1 and Level-2 Atmospheric Data

## 1.1. Introduction
In this notebook we present atmospheric observations from the [ICOS Atmospheric Station Network](https://www.icos-cp.eu/stations). All observations are stored and can be downloaded from the [ICOS Carbon Portal](https://www.icos-cp.eu/) (ICOS CP). More in particular, this notebook contains visualizations of all available data from all current ICOS atmospheric stations. As data is retrieved in real-time from ICOS CP, the content of the visualizations is always up to date including all latest data submissions.
<br>
<br>

The notebook is divided in the following six main parts:
<br>
- [Importing Python modules and functions](#icos_level2_atc) to retrieve, process and plot data. Note that a module has been created containing functions to read in the content of ICOS Atmospheric data files to pandas dataframes. All plots and maps are interactive and have been developed using the [Bokeh](https://bokeh.pydata.org/en/latest/) interactive visualization library. 
<br>
<br>
- [Exploring plots of ICOS Level-2 Atmospheric Data](#exploring_atc_l2)
    -  [Exploring (Single Station - Single Tracer)](#exploring_single_station_single_tracer_atc_l2)
    -  [Exploring (Single Station - Multiple Tracers)](#exploring_single_station_multiple_tracers_atc_l2)
    -  [Exploring (Single Tracer - Multiple Stations)](#exploring_single_tracer_multiple_stations_atc_l2)
<br>
<br>
- [Focusing on Data](#focusing_atc_l2)
<br>
<br>
- [Statistics](#statistics_atc_l2)
    -  [Basic Statistics](#basic_statistics_atc_l2)
    -  [Correlation Statistics](#statistics_correlation_atc_l2)
    -  [Smoothing](#statistics_smoothing_atc_l2)
<br>
<br>
- [Comparing Plots from Multiple Stations](#comparing_atc_l2)
<br>
<br>
- [ICOS Level-1 (Near Real Time - NRT) Atmospheric Data](#icos_level1_atc)
<br>
<br>
- [Get Access to ICOS Jupyter Notebook Developing Environment](#create_nb_account_info)
<br>
<br>

All aforementioned parts, _with the exception of parts containing subdivisions_, include two sections; the first section presents information regarding the logic behind the computed result together with its corresponding python code and the second section outputs the result. In cases where a part is subdivided to more parts, every subdivision contains two sections with the same content as above.

<br>
<br>
<br>

<a id='icos_level2_atc'></a>

## 2. ICOS Level 2 Atmospheric Data - Importing Python Modules and Functions
This part contains Python code to import all necessary Python modules and functions to retrieve, store, process and plot ICOS Level-1 and Level-2 Atmospheric Data. 

In [ ]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines){
    return false;}

In [ ]:
#Import modules:
import numpy as np
from numpy import nan
import pandas as pd
from datetime import datetime
import requests
import fnmatch
from ipywidgets import interact, interact_manual, ColorPicker, Dropdown, SelectMultiple, Checkbox, DatePicker
from bokeh.io import show, reset_output, output_notebook

#Import ICOS tools:
from icoscp.sparql import sparqls
from icoscp.sparql.runsparql import RunSparql
from icoscp.cpb.dobj import Dobj

#Set the notebook as the selected output location:
reset_output()
output_notebook()

In [ ]:
def get_coords_icos_stations_atc():
    
    """
    Project:         'ICOS Carbon Portal'
    Created:          Thur Mar 28 17:27:00 2019
    Last Changed:     Thur Mar 28 17:27:00 2019
    Version:          1.0.0
    Author(s):        Karolina, Oleg, Claudio
    
    Description:      Download ICOS station metadata from ICOS CP with a SPARQL-query.
    Input parameters: No input parameter/s
    Output:           pandas dataframe
                      columns: 
                            1. URL to station landing page (var_name: 'station', var_type: String)
                            2. Name of station PI (var_name: 'PI_names', var_type: String)
                            3. Station name (var_name: 'stationName', var_type: String)
                            4. 3-character Station ID (var_name: 'stationId', var_type: String)
                            5. 2-character Country (var_name: 'Country', var_type: String)
                            6. Station Latitude (var_name: 'lat', var_type: String)
                            7. Station Longitude (var_name: 'lon', var_type: String)

    """
    
    #Import modules:
    import requests
    import pandas as pd
    
    #Define URL:
    url = 'https://meta.icos-cp.eu/sparql'
    
    #Define sparQL-query:
    query = """
        prefix cpmeta: <http://meta.icos-cp.eu/ontologies/cpmeta/>
        select *
        from <http://meta.icos-cp.eu/resources/icos/>
        where{
        {
        select ?station (GROUP_CONCAT(?piLname; separator=";") AS ?PI_names)
        where{
          ?station a cpmeta:AS .
          ?pi cpmeta:hasMembership ?piMemb .
          ?piMemb cpmeta:atOrganization ?station  .
           ?piMemb cpmeta:hasRole <http://meta.icos-cp.eu/resources/roles/PI> .
           filter not exists {?piMemb cpmeta:hasEndTime []}
           ?pi cpmeta:hasLastName ?piLname .
        }
        group by ?station
        }
        ?station a cpmeta:AS .
        ?station cpmeta:hasName ?stationName ;
           cpmeta:hasStationId ?stationId ;
           cpmeta:countryCode ?Country ;
           cpmeta:hasLatitude ?lat ;
           cpmeta:hasLongitude ?lon .
        }
        order by ?Short_name
    """
    
    #Send request:
    r = requests.get(url, params = {'format': 'json', 'query': query})
    
    #Get response as json:
    data = r.json()
    
    #Convert the the response into a table.
    #Output is an array, where each row contains information about the station:
    data_ls = [[data['results']['bindings'][row].get(col, {}).get('value')
                for col in data['head']['vars']]
               for row in range(len(data['results']['bindings']))]

    #Create a pandas dataframe from the list:
    df = pd.DataFrame(data_ls, columns=data['head']['vars'])
    
    #Return dataframe:
    return df

In [ ]:
def get_icos_stations_atc_L1():

    """
    Project:         'ICOS Carbon Portal'
    Created:          Thu Mar 28 17:27:00 2019
    Last Changed:     Tue Apr 01 09:20:00 2019
    Version:          1.1.0
    Author(s):        Karolina, Oleg, Claudio
    
    Description:      Download ICOS station names for all L1 gases from ICOS CP with a SPARQL-query.
    Input parameters: No input parameter/s
    Output:           Pandas Dataframe
                      columns: 
                            1. URL to ICOS RI Data Object Landing Page (var_name: 'dobj', var_type: String)
                            2. Filename for Data Object (var_name: 'filename', var_type: String)
                            3. Name of gas (var_name: 'variable', var_type: String)
                            4. Station name (var_name: 'stationName', var_type: String)
                            5. Sampling height a.g.l. (var_name: 'height', var_type: String)
                            6. Sampling Start Time (var_name: 'timeStart', var_type: String)
                            7. Sampling End Time (var_naem: 'timeEnd', var_type: String)

    """
    
    #Import modules:
    import requests
    import pandas as pd
    
    #Define URL:
    url = 'https://meta.icos-cp.eu/sparql'
    
    #Define SPARQL-query:
    query = """
        prefix cpres: <http://meta.icos-cp.eu/resources/cpmeta/>
        prefix cpmeta: <http://meta.icos-cp.eu/ontologies/cpmeta/>
        prefix prov: <http://www.w3.org/ns/prov#>
        select ?dobj ?fileName ?variable ?stationName ?height ?timeStart ?timeEnd #?stationId
        where{
           values ?vtype { cpres:co2MixingRatio cpres:coMixingRatioPpb cpres:ch4MixingRatioPpb}
           #values ?spec {cpres:atcCo2NrtGrowingDataObject cpres:atcCoNrtGrowingDataObject cpres:atcCh4NrtGrowingDataObject}
           ?vtype rdfs:label ?variable .
           ?col cpmeta:hasValueType ?vtype .
           ?dset cpmeta:hasColumn ?col .
           ?spec cpmeta:containsDataset ?dset .
           ?spec cpmeta:hasAssociatedProject <http://meta.icos-cp.eu/resources/projects/icos> .
           ?spec cpmeta:hasDataLevel "1"^^xsd:integer .
           ?dobj cpmeta:hasObjectSpec ?spec .
           ?dobj cpmeta:hasName ?fileName .
           ?dobj cpmeta:hasSizeInBytes ?fileSize .
           filter not exists {[] cpmeta:isNextVersionOf ?dobj}
           ?dobj cpmeta:wasAcquiredBy [
              #prov:wasAssociatedWith/cpmeta:hasStationId ?stationId ;
              prov:startedAtTime ?timeStart ;
              prov:endedAtTime ?timeEnd ;
              prov:wasAssociatedWith/cpmeta:hasName ?stationName ;
              cpmeta:hasSamplingHeight ?height
            ]
        }
        order by ?variable ?stationName ?height
    """
    
    #Send request:
    r = requests.get(url, params = {'format': 'json', 'query': query})
    
    #Get response in JSON:
    data = r.json()

    #Convert the the result into a table
    # output is an array, where each row contains 
    # information about the station:
    data_ls = [[data['results']['bindings'][row].get(col, {}).get('value')
                for col in data['head']['vars']]
               for row in range(len(data['results']['bindings']))]

    #Create a pandas dataframe from the list:
    df = pd.DataFrame(data_ls, columns=data['head']['vars'])

    #Return dataframe:
    return df

In [ ]:
def create_lookup_df_atc_L1():
    
    """
    Project:     'ICOS Carbon Portal'
    Created:      Wed Mar 28 17:40:00 2019
    Last Changed: Tue Apr 01 10:00:00 2019
    Version:      1.1.0
    Author(s):    Karolina
    
    Description:  Return a pandas dataframe with information for all available ICOS Level-1 Atmospheric Data Files.
    Input:        No input parameter/s  
    Output:       pandas dataframe
                  columns:
                      1. URL to ICOS RI Data Object Landing Page (var_name: 'dobj', var_type: String)
                      2. Filename for Data Object (var_name: 'filename', var_type: String)
                      3. Name of gas/tracer (var_name: 'variable', var_type: String)
                      4. Station name (var_name: 'stationName', var_type: String)
                      5. Sampling height a.g.l. (var_name: 'height', var_type: String)
                      6. Sampling start time (var_name:'timeStart', var_type: String)
                      7. Sampling end time (var_name: 'timeEnd', var_type: String)
                      8. 3-character Station ID (var_name: 'stationId', var_type: String)
    """
    #Import modules:
    import pandas as pd
    
    #Get ICOS-stations with level-1 gas-data:
    icos_stations_L1_gas_df = get_icos_stations_atc_L1()
    
    #Get ICOS-station info:
    station_info_df = get_coords_icos_stations_atc()
    
    #Create lookup dataframe:
    lookup_df = icos_stations_L1_gas_df.join(station_info_df.filter(['stationName',
                                                                     'stationId']).set_index('stationName'),
                                             on='stationName')
    
    #Return dataframe:
    return lookup_df

In [ ]:
def get_icos_stations_atc_L2():

    """
    Project:         'ICOS Carbon Portal'
    Created:          Thu Mar 28 17:27:00 2019
    Last Changed:     Tue Apr 30 17:27:00 2019
    Version:          1.1.0
    Author(s):        Karolina, Oleg, Claudio
    
    Description:      Download ICOS station names for all L2 gases from ICOS CP with a SPARQL-query.
    Input parameters: No input parameter/s
    Output:           pandas dataframe
                      columns: 
                            1. URL to ICOS RI Data Object Landing Page (var_name: 'dobj', var_type: String)
                            2. Filename for Data Object (var_name: 'filename', var_type: String)
                            3. Name of gas (var_name: 'variable', var_type: String)
                            4. Station name (var_name: 'stationName', var_type: String)
                            5. Sampling height a.g.l. (var_name: 'height', var_type: String)
                            6. Sampling Start Time (var_name: "timeStart", var_type: Datetime Object)
                            7. Sampling End Time (var_name: "timeEnd", var_type: Datetime Object)

    """
    
    #Import modules:
    import requests
    import pandas as pd
    
    #Define URL:
    url = 'https://meta.icos-cp.eu/sparql'
    
    #Define SPARQL-query:
    query = """
        prefix cpres: <http://meta.icos-cp.eu/resources/cpmeta/>
        prefix cpmeta: <http://meta.icos-cp.eu/ontologies/cpmeta/>
        prefix prov: <http://www.w3.org/ns/prov#>
        select ?dobj ?fileName ?variable ?stationName ?height ?timeStart ?timeEnd #?stationId
        where{
           values ?vtype {cpres:coMixingRatioPpb cpres:ch4MixingRatioPpb cpres:co2MixingRatio }
           #values ?spec {cpres:atcCo2L2DataObject cpres:atcCoL2DataObject cpres:atcCh4L2DataObject}
           ?vtype rdfs:label ?variable .
           ?col cpmeta:hasValueType ?vtype .
           ?dset cpmeta:hasColumn ?col .
           ?spec cpmeta:containsDataset ?dset .
           ?spec cpmeta:hasAssociatedProject <http://meta.icos-cp.eu/resources/projects/icos> .
           ?spec cpmeta:hasDataLevel "2"^^xsd:integer .
           ?dobj cpmeta:hasObjectSpec ?spec .
           ?dobj cpmeta:hasName ?fileName .
           filter not exists {[] cpmeta:isNextVersionOf ?dobj}
           ?dobj cpmeta:wasAcquiredBy [
              #prov:wasAssociatedWith/cpmeta:hasStationId ?stationId ;
              prov:startedAtTime ?timeStart ;
              prov:endedAtTime ?timeEnd ;
              prov:wasAssociatedWith/cpmeta:hasName ?stationName ;
              cpmeta:hasSamplingHeight ?height
            ]
        }
        order by ?variable ?stationName ?height
    """
    
    #Send request:
    r = requests.get(url, params = {'format': 'json', 'query': query})
    
    #Get response as JSON:
    data = r.json()

    #Convert the the result into a list
    # output is an array, where each row contains 
    # information about the station
    data_ls = [[data['results']['bindings'][row].get(col, {}).get('value')
                for col in data['head']['vars']]
               for row in range(len(data['results']['bindings']))]

    #Create a pandas dataframe from the list:
    df = pd.DataFrame(data_ls, columns=data['head']['vars'])

    #Return dataframe:
    return df

In [ ]:
def create_lookup_df_atc_L2():

    """
    Project:     'ICOS Carbon Portal'
    Created:      Wed Apr 01 10:10:00 2019
    Last Changed: Wed Apr 01 10:10:00 2019
    Version:      1.0.0
    Author(s):    Karolina
    
    Description:  Return a pandas dataframe with information for all available ICOS Level-2 Atmospheric Data Files.
    Input:        No input parameter/s  
    Output:       pandas dataframe
                  columns:
                      1. URL to ICOS RI Data Object Landing Page (var_name: 'dobj', var_type: String)
                      2. Filename for Data Object (var_name: 'filename', var_type: String)
                      3. Name of gas/tracer (var_name: 'variable', var_type: String)
                      4. Station name (var_name: 'stationName', var_type: String)
                      5. Sampling height a.g.l. (var_name: 'height', var_type: String)
                      6. Sampling start time (var_name:'timeStart', var_type: String)
                      7. Sampling end time (var_name: 'timeEnd', var_type: String)
                      8. 3-character Station ID (var_name: 'stationId', var_type: String)
    """
    #Import modules:
    import pandas as pd
    
    #Get ICOS-stations with level-2 gas-data:
    icos_stations_L2_gas_df = get_icos_stations_atc_L2()
    
    #Get ICOS-station info:
    station_info_df = get_coords_icos_stations_atc()
    
    #Create lookup dataframe:
    lookup_df = icos_stations_L2_gas_df.join(station_info_df.filter(['stationName',
                                                                     'stationId']).set_index('stationName'),
                                             on='stationName')
    
    #Return dataframe:
    return lookup_df

In [ ]:
def get_icos_citation(dataObject):
    
    """
    Project:         'ICOS Carbon Portal'
    Created:          Fri May 10 12:35:00 2019
    Last Changed:     Fri May 10 12:35:00 2019
    Version:          1.0.0
    Author(s):        Oleg, Karolina
    
    Description:      Function that takes a string variable representing the URL with data object ID as input and
                      returns a string with the citation for the corresponding data object. The data object ID is
                      a unique identifier of every separate ICOS dataset.
    
    Input parameters: Data Object ID (var_name: "dataObject", var_type: String)
    
    Output:           Citation (var_type: String)

    """
    
    #Define URL:
    url = 'https://meta.icos-cp.eu/sparql'
    
    #Define SPARQL-query to get the citation for the given data object id:
    query = """
        prefix cpmeta: <http://meta.icos-cp.eu/ontologies/cpmeta/>
        select * where{
        optional{<"""+dataObject+"""> cpmeta:hasCitationString ?cit}}
    """
    
    #Send request with the defined SPARQL-query & set the response format to JSON:
    r = requests.get(url, params = {'format': 'json', 'query': query})
    
    #Get the response as JSON:
    data = r.json()

    #Convert the JSON repsonse into a list
    # output is an array, where each row contains 
    # information about the station
    data_ls = [[data['results']['bindings'][row].get(col, {}).get('value')
                for col in data['head']['vars']]
               for row in range(len(data['results']['bindings']))]

    #Create a pandas dataframe from the list:
    df = pd.DataFrame(data_ls, columns=data['head']['vars'])

    #Return dataframe:
    return df
    

In [ ]:
def printmd(string):
    
    """
    Project:         'ICOS Carbon Portal'
    Created:          Fri May 10 12:00:00 2019
    Last Changed:     Fri May 10 12:00:00 2019
    Version:          1.0.0
    Author(s):        Karolina
    
    Description:      Function that allows you to print the string input parameter using
                      markdown formatting code.
                      
    Input parameters: String of characters
                      (var_name: 'string', var_type: String)

    Output:           String     
    
    """
    
    #import module:
    from IPython.display import Markdown, display
    
    #Enable displaying string with markdown code:
    display(Markdown(string))

In [ ]:
def changeprojection(inprojection, outprojection, dataset_df, longitude_col_name, latitude_col_name):
    
    """
    Project:         'ICOS Carbon Portal'
    Created:          Fri Apr 04 09:30:00 2019
    Last Changed:     Fri Apr 04 09:30:00 2019
    Version:          1.0.0
    Author(s):        Karolina
    
    Description:      Function that changes the projection of x- and y-coordinates stored in a pandas dataframe. 
                      The function takes 5 input parameters (input projection, output projection, dataframe,
                      column name for longitude data, column name for latitude data), and adds to the existing
                      dataframe two new columns containing the reprojected x- and y-coordinates correspondigly. 
                      
    Input parameters: 1. Projection of Input Coordinates (var_name: 'inprojection', var_type: String)
                      2. Projection of Output Coordinates (var_name: 'outprojection', var_type: String)
                      3. Dataframe Containing Coordinates (var_name: 'dataset_df', var_type: Pandas Dataframe)
                      4. Column Name of Input Longitude Coordinates (var_name: 'longitude_col_name', var_type: String)
                      5. Column Name of Input Latitude Coordinates (var_name: 'latitude_col_name', var_type: String)

    Output:           pandas dataframe (with 2 new additonal columns; proj_x, proj_y)
    
    """
    
    #Import Python projection modules:
    from pyproj import Proj, transform
    
    #Reproject coordinates of point data and store the result in a list:
    #OBS! The order is important, so longitude has to be set before latitude
    proj_coords = [transform(Proj(init=inprojection),
                             Proj(init=outprojection),
                             dataset_df[longitude_col_name].iloc[i],
                             dataset_df[latitude_col_name].iloc[i]) for i in range(len(dataset_df))]
    
    #Set the labels for the projected coordinate columns:
    labels = ['proj_x','proj_y']
    
    #Create a pandas dataframe from the list of reprojected coordinates:
    coords_df = pd.DataFrame.from_records(proj_coords,columns =labels)
    
    #Add the columns with the projected coordinates to the initial pandas dataframe:
    dataset_df[coords_df.columns] = coords_df
    
    #Return dataframe containing reprojected coordinates:
    return dataset_df

In [ ]:
def rounddown_100(x):
    
    """
    Project:         'ICOS Carbon Portal'
    Created:          Tue May 07 09:00:00 2018
    Last Changed:     Tue May 07 09:00:00 2019
    Version:          1.0.0
    Author(s):        Karolina
    
    Description:      Function that takes a number as input and floors it to the nearest "100".
                      
    Input parameters: Number (var_name: 'x', var_type: Integer or Float)

    Output:           Float
    
    """
    
    #Import module:
    import numbers
    
    #Check if input parameter is numeric:
    if(isinstance(x, numbers.Number)==True):

        #If the number is an integral multiple of 100:
        if(((x/100.0)%2==0) or (x<=0) or (x==100)):

            return(int(x / 100.0) * 100) - 100
        
        #If the input number is NOT an integral multiple of 100:
        else:

            return(int(x / 100.0) * 100)

    #If input parameter is not numeric, prompt an error message:
    else:
        print("Input parameter is not numeric!")
    
    

In [ ]:
def roundup_100(x):
    
    """
    Project:         'ICOS Carbon Portal'
    Created:          Tue May 07 09:00:00 2018
    Last Changed:     Tue May 07 09:00:00 2019
    Version:          1.0.0
    Author(s):        Karolina
    
    Description:      Function that takes a number as input and rounds it up to the nearest "100".
                      
    Input parameters: Number (var_name: 'x', var_type: Integer or Float)

    Output:           Float
    
    """
    
    #Import modules:
    import math
    import numbers
    
    #Check if input parameter is numeric:
    if(isinstance(x, numbers.Number)==True):
        
        #for integral mulitples of 100 and for the special cases of 100, 0 and -100:
        if(((x/100.0)%2==0) or (x==100) or (x==-100)):  
            return int(math.ceil(x / 100.0)) * 100 + 100

        else:
            return int(math.ceil(x / 100.0)) * 100
    
    #If input parameter is not numeric, prompt an error message:
    else:
        print("Input parameter is not numeric!")
    

In [ ]:
def rounddown_20(x):
    
    """
    Project:         'ICOS Carbon Portal'
    Created:          Tue May 07 09:00:00 2018
    Last Changed:     Tue May 07 09:00:00 2019
    Version:          1.0.0
    Author(s):        Karolina
    
    Description:      Function that takes a number as input and floors it to the nearest "20".
                      
    Input parameters: Number (var_name: 'x', var_type: Integer or Float)

    Output:           Float
    
    """
    
    #Import module:
    import math
    import numbers
    
    #Check if input parameter is numeric:
    if(isinstance(x, numbers.Number)==True):
    
        #If the 2nd digit from the decimal point is an even number:
        if(int(x/10.0)%2==0):

            return(int(x / 10.0) * 10) - 20

        #If the 2nd digit from the decimal point is an odd number:
        else:

            return(int(x / 10.0) * 10) - 10
        
    #If input parameter is not numeric, prompt an error message:
    else:
        print("Input parameter is not numeric!")

In [ ]:
def roundup_20(x):
    
    """
    Project:         'ICOS Carbon Portal'
    Created:          Tue May 07 09:00:00 2018
    Last Changed:     Tue May 07 09:00:00 2019
    Version:          1.0.0
    Author(s):        Karolina
    
    Description:      Function that takes a number as input and rounds it up to the closest "20".
                      
    Input parameters: Number (var_name: 'x', var_type: Integer or Float)

    Output:           Float
    
    """
    
    #Import module:
    import math
    import numbers
    
    #Check if input parameter is numeric:
    if(isinstance(x, numbers.Number)==True):
    
        #for positive numbers, multiples of 20.0:
        if((x>=0)&(((x/10.0)%20)%2 == 0)):  
            return int(math.ceil(x / 10.0)) * 10 +20

        #for positive numbers with an even number as 2nd digit:
        elif((x>0)&(int(x/10.0)%2==0)):
            return int(math.ceil(x / 10.0)) * 10 +10

        #for positive and negative numbers, whose 2nd digit is an odd number (except for i in [-1,-9]):
        elif(int(x/10.0)%2!=0):
            return int((x / 10.0)) * 10 +10

        #for negative numbers, whose 1st or 2nd digit is an even number:
        elif((x<-10) & (int(x)%2==0)):   
            return int((x / 10.0)) * 10 +20

        else:
            return 0
    
    #If input parameter is NOT numeric, prompt an error message:
    else:
        print("Input parameter is not numeric!")
    

In [ ]:
def set_yranges_2y(y1_min, y1_max, y2_min, y2_max, y1_step, y2_step ,new_yrange_name):
    
    """
    Project:         'ICOS Carbon Portal'
    Created:          Tue May 07 10:30:00 2018
    Last Changed:     Tue May 07 10:30:00 2019
    Version:          1.0.0
    Author(s):        Karolina
    
    Description:      Function that takes the primary and secondary y-axis min/max values as well as
                      the step values for every y-axis and the secondary y-axis new range name as input
                      parameters, performs computations so that the two axes are alligned and returns
                      their corresponding RangeId objects. Works only for Bokeh plots.
                      
    Input parameters: 1. Min value of primary y-axis (var_name: 'y1_min', var_type: Integer or Float)
                      2. Max value of primary y-axis (var_name: 'y1_max', var_type: Integer or Float)
                      3. Min value of secondary y-axis (var_name: 'y2_min', var_type: Integer or Float)
                      4. Max value of secondary y-axis (var_name: 'y2_max', var_type: Integer or Float)
                      5. Step of primary y-axis (var_name: 'y1_step', var_type: Integer or Float)
                      6. Step of secondary y-axis (var_name: 'y2_step', var_type: Integer or Float)
                      7. Name of new yrange object for secondary y-axis
                         (var_name: "new_yrange_name", var_type: Bokeh Plot yrange object)

    Output:           Bokeh Plot yrange objects for primary and secondary y-axes.
    
    """
    
    #import modules:
    import numpy as np
    from bokeh.models import Range1d
          
    #yrange and tick function for plot with primary and secondary y-axis:
    yticks1 = np.arange(y1_min, y1_max + y1_step, y1_step)
    yticks2 = np.arange(y2_min, y2_max + y2_step, y2_step)

    #Get difference in total number of ticks between primary and secondary y-axis:  
    diff = abs(len(yticks2)-len(yticks1))

    #Get how many times the step needs to be added to start and end:
    num_of_steps = int(diff/2)
    
    #If the primary and the secondary y-axis have the same number of ticks:
    if(diff==0):

        #Set the range of the 1st y-axis:
        y_range = Range1d(start=y1_min, end=y1_max)

        #Set the 2nd y-axis, range-name, range:
        extra_y_ranges = {new_yrange_name: Range1d(start=y2_min, end=y2_max)}

        
    #If the primary y-axis has fewer ticks than the secondary y-axis:    
    elif(len(yticks2)>len(yticks1)):
    
        #If the difference in ticks between the two axes is an odd number:
        if(diff%2==1):

            #Set the range of the 1st y-axis:
            y_range = Range1d(start=y1_min-(y1_step*(num_of_steps+1)), end=y1_max+(y1_step*num_of_steps))

            #Set the 2nd y-axis, range-name, range:
            extra_y_ranges = {new_yrange_name: Range1d(start=y2_min, end=y2_max)}

        #If the difference in ticks between the two axes is an even number:
        else:
            #Set the range of the 1st y-axis:
            y_range = Range1d(start=y1_min-(y1_step*num_of_steps), end=y1_max+(y1_step*num_of_steps))

            #Set the 2nd y-axis, range-name, range:
            extra_y_ranges = {new_yrange_name: Range1d(start=y2_min, end=y2_max)}
            
            
    #If the primary y-axis has more ticks than the secondary y-axis, e.g. len(yticks1)>len(yticks2_test):
    else:
        
        #If the difference in ticks between the two axes is an odd number:
        if(diff%2==1):
            
            #Set the range of the 1st y-axis:
            y_range = Range1d(start=y1_min, end=y1_max)

            #Set the 2nd y-axis, range-name, range:
            extra_y_ranges = {new_yrange_name: Range1d(start=y2_min - (y2_step*(num_of_steps)), end=y2_max + (y2_step*(num_of_steps+1)))}

        #If the difference in ticks between the two axes is an even number:
        else:
            #Set the range of the 1st y-axis:
            y_range = Range1d(start=y1_min, end=y1_max)

            #Set the 2nd y-axis, range-name, range:
            extra_y_ranges = {new_yrange_name: Range1d(start=y2_min - (y2_step*num_of_steps), end=y2_max + (y2_step*num_of_steps))}
              
        
    #Return y-range for primary and secondary y-axes:
    return y_range, extra_y_ranges


In [ ]:
def set_yranges_3y(y1_min, y1_max, y2_min, y2_max, y3_min, y3_max, y1_step, y2_step, y3_step):
    
    """
    Project:         'ICOS Carbon Portal'
    Created:          Tue May 07 10:30:00 2018
    Last Changed:     Tue May 07 10:30:00 2019
    Version:          1.0.0
    Author(s):        Karolina
    
    Description:      Function that takes the primary, secondary and third y-axis min/max values as well as
                      the step values for every y-axis as input parameters, performs computations so that the
                      three axes are alligned and returns their corresponding RangeId objects.
                      Works only for Bokeh plots.
                      
    Input parameters: 1. Min value of primary y-axis (var_name: 'y1_min', var_type: Integer or Float)
                      2. Max value of primary y-axis (var_name: 'y1_max', var_type: Integer or Float)
                      3. Min value of secondary y-axis (var_name: 'y2_min', var_type: Integer or Float)
                      4. Max value of secondary y-axis (var_name: 'y2_max', var_type: Integer or Float)
                      5. Min value of third y-axis (var_name: 'y3_min', var_type: Integer or Float)
                      6. Max value of third y-axis (var_name: 'y3_max', var_type: Integer or Float)
                      7. Step of primary y-axis (var_name: 'y1_step', var_type: Integer or Float)
                      8. Step of secondary y-axis (var_name: 'y2_step', var_type: Integer or Float)
                      9. Step of third y-axis (var_name: 'y3_step', var_type: Integer or Float)

    Output:           Bokeh Plot yrange objects for primary and secondary y-axes.
    
    """
    
    #import modules:
    import numpy as np
    from bokeh.models import Range1d
          
    #yrange and tick function for plot with primary and secondary y-axis:
    yticks1 = np.arange(y1_min, y1_max + y1_step, y1_step)
    yticks2 = np.arange(y2_min, y2_max + y2_step, y2_step)
    yticks3 = np.arange(y3_min, y3_max + y3_step, y3_step)
    
    #Get the number of ticks per y-axis:
    y1_num_of_ticks = len(yticks1)
    y2_num_of_ticks = len(yticks2)
    y3_num_of_ticks = len(yticks3)

    #Get difference in total number of ticks between primary and secondary y-axis:  
    diff_12 = abs(len(yticks2)-len(yticks1))
    diff_13 = abs(len(yticks3)-len(yticks1))
    diff_23 = abs(len(yticks3)-len(yticks2))

    #Get how many times the step needs to be added to start and end:
    num_of_steps_12 = int(diff_12/2)
    num_of_steps_13 = int(diff_13/2)
    num_of_steps_23 = int(diff_23/2)
    
    
    #If the primary, secondary and 3rd y-axis have the same number of ticks:
    if((diff_12==0) and (diff_13==0) and (diff_23==0)):

        #Set the range of the 1st y-axis:
        y_range = Range1d(start=y1_min, end=y1_max)

        #Set the 2nd y-axis, range-name, range:
        extra_y_ranges_1 = Range1d(start=y2_min, end=y2_max)
        
        #Set the 3rd y-axis, range-name, range:
        extra_y_ranges_2 = Range1d(start=y3_min, end=y3_max)

        
        
    #if y-axis 1 is the axis with the highest number of ticks:
    elif(max(y1_num_of_ticks, y2_num_of_ticks, y3_num_of_ticks)==y1_num_of_ticks):
        
        #Check if the difference between y-axis 1 and the other axes is an even number:
        if((diff_12%2==0) and (diff_13%2==0)):
            
            #Set the range of the 1st y-axis:
            y_range = Range1d(start=y1_min, end=y1_max)

            #Set the 2nd y-axis, range-name, range:
            extra_y_ranges_1 = Range1d(start=y2_min - (y2_step*num_of_steps_12),
                                       end=y2_max + (y2_step*num_of_steps_12))
            
            #Set the 3rd y-axis, range-name, range:
            extra_y_ranges_2 = Range1d(start=y3_min - (y3_step*num_of_steps_13), 
                                       end=y3_max + (y3_step*num_of_steps_13))

        
        #Check if the difference between y-axis 1 and the other axes is an odd number:
        elif((diff_12%2==1) and (diff_13%2==1)):
            
            #Set the range of the 1st y-axis:
            y_range = Range1d(start=y1_min, end=y1_max)

            #Set the 2nd y-axis, range-name, range:
            extra_y_ranges_1 = Range1d(start=y2_min - (y2_step*(num_of_steps_12)),
                                       end=y2_max + (y2_step*(num_of_steps_12+1)))
            
            #Set the 3rd y-axis, range-name, range:
            extra_y_ranges_2 = Range1d(start=y3_min - (y3_step*(num_of_steps_13)),
                                       end=y3_max + (y3_step*(num_of_steps_13+1)))

            
        #Check if the difference between y-axis 1 and the other axes is an even/odd number:
        elif((diff_12%2==0) and (diff_13%2==1)):
            
            #Set the range of the 1st y-axis:
            y_range = Range1d(start=y1_min, end=y1_max)

            #Set the 2nd y-axis, range-name, range: --- > even diff
            extra_y_ranges_1 = Range1d(start=y2_min - (y2_step*num_of_steps_12),
                                       end=y2_max + (y2_step*num_of_steps_12))
            
            #Set the 3rd y-axis, range-name, range: --- > odd diff
            extra_y_ranges_2 = Range1d(start=y3_min - (y3_step*(num_of_steps_13)),
                                       end=y3_max + (y3_step*(num_of_steps_13+1)))
         
        
        #Check if the difference between y-axis 1 and the other axes is an odd/even number:
        #I.e. (diff_12%2==1) and (diff_13%2==0)
        else:
            
            #Set the range of the 1st y-axis:
            y_range = Range1d(start=y1_min, end=y1_max)

            #Set the 2nd y-axis, range-name, range: --- > odd diff
            extra_y_ranges_1 = Range1d(start=y2_min - (y2_step*(num_of_steps_12)),
                                       end=y2_max + (y2_step*(num_of_steps_12+1)))
            
            #Set the 3rd y-axis, range-name, range: --- > even diff
            extra_y_ranges_2 = Range1d(start=y3_min - (y3_step*num_of_steps_13), 
                                       end=y3_max + (y3_step*num_of_steps_13))
        
      
    
    #if y-axis 2 is the axis with the highest number of ticks:
    elif(max(y1_num_of_ticks, y2_num_of_ticks, y3_num_of_ticks)==y2_num_of_ticks):
        
        #Check if the difference between y-axis 2 and the other axes is an even number:
        if((diff_12%2==0) and (diff_23%2==0)):
            
            #Set the range of the 1st y-axis:
            y_range = Range1d(start=y1_min-(y1_step*num_of_steps_12),
                              end=y1_max+(y1_step*num_of_steps_12))

            #Set the 2nd y-axis, range-name, range:
            extra_y_ranges_1 = Range1d(start=y2_min, end=y2_max)
            
            #Set the 3rd y-axis, range-name, range:
            extra_y_ranges_2 = Range1d(start=y3_min-(y3_step*num_of_steps_23),
                                       end=y3_max+(y3_step*num_of_steps_23))
            
            
        #Check if the difference between y-axis 2 and the other axes is an odd number:
        elif((diff_12%2==1) and (diff_23%2==1)):
            
            #Set the range of the 1st y-axis:
            y_range = Range1d(start=y1_min-(y1_step*(num_of_steps_12+1)),
                              end=y1_max+(y1_step*num_of_steps_12))

            #Set the 2nd y-axis, range-name, range:
            extra_y_ranges_1 = Range1d(start=y2_min, end=y2_max)
            
            #Set the 3rd y-axis, range-name, range:
            extra_y_ranges_2 = Range1d(start=y3_min-(y3_step*(num_of_steps_23+1)),
                                       end=y3_max+(y3_step*num_of_steps_23))
      
            
        #Check if the difference between y-axis 2 and the other axes is an even/odd number:
        elif((diff_12%2==0) and (diff_23%2==1)):
            
            #Set the range of the 1st y-axis: --- > even diff
            y_range = Range1d(start=y1_min-(y1_step*num_of_steps_12),
                              end=y1_max+(y1_step*num_of_steps_12))

            #Set the 2nd y-axis, range-name, range: 
            extra_y_ranges_1 = Range1d(start=y2_min, end=y2_max)
            
            #Set the 3rd y-axis, range-name, range: --- > odd diff
            extra_y_ranges_2 = Range1d(start=y3_min-(y3_step*(num_of_steps_23+1)),
                                       end=y3_max+(y3_step*num_of_steps_23))
            
         
        #Check if the difference between y-axis 2 and the other axes is an odd/even number:
        #I.e. (diff_12%2==1) and (diff_23%2==0)
        else:
            
            #Set the range of the 1st y-axis: --- > odd diff
            y_range = Range1d(start=y1_min-(y1_step*(num_of_steps_12+1)),
                              end=y1_max+(y1_step*num_of_steps_12))

            #Set the 2nd y-axis, range-name, range:
            extra_y_ranges_1 = Range1d(start=y2_min, end=y2_max)
            
            #Set the 3rd y-axis, range-name, range: --- > even diff
            extra_y_ranges_2 = Range1d(start=y3_min-(y3_step*num_of_steps_23),
                                       end=y3_max+(y3_step*num_of_steps_23))
            
    
    
    #if y-axis 3 is the axis with the highest number of ticks:
    elif(max(y1_num_of_ticks, y2_num_of_ticks, y3_num_of_ticks)==y3_num_of_ticks):
        
        #Check if the difference between y-axis 3 and the other axes is an even number:
        if((diff_13%2==0) and (diff_23%2==0)):
            
            #Set the range of the 1st y-axis:
            y_range = Range1d(start=y1_min-(y1_step*num_of_steps_13),
                              end=y1_max+(y1_step*num_of_steps_13))

            #Set the 2nd y-axis, range-name, range:
            extra_y_ranges_1 = Range1d(start=y2_min-(y2_step*num_of_steps_23),
                                       end=y2_max+(y2_step*num_of_steps_23))
            
            #Set the 3rd y-axis, range-name, range:
            extra_y_ranges_2 = Range1d(start=y3_min, end=y3_max)
            
        
        #Check if the difference between y-axis 3 and the other axes is an odd number:
        elif((diff_13%2==1) and (diff_23%2==1)):
            
            #Set the range of the 1st y-axis:
            y_range = Range1d(start=y1_min-(y1_step*(num_of_steps_13+1)),
                              end=y1_max+(y1_step*num_of_steps_13))
            
            #Set the 2nd y-axis, range-name, range:
            extra_y_ranges_1 = Range1d(start=y2_min-(y2_step*(num_of_steps_23+1)),
                                       end=y2_max+(y2_step*num_of_steps_23))
            
            #Set the 3rd y-axis, range-name, range:
            extra_y_ranges_2 = Range1d(start=y3_min, end=y3_max)
            
            
            
        #Check if the difference between y-axis 3 and the other axes is an even/odd number:
        elif((diff_13%2==0) and (diff_23%2==1)):
            
            #Set the range of the 1st y-axis:
            y_range = Range1d(start=y1_min-(y1_step*num_of_steps_13),
                              end=y1_max+(y1_step*num_of_steps_13))
            
            #Set the 2nd y-axis, range-name, range:
            extra_y_ranges_1 = Range1d(start=y2_min-(y2_step*(num_of_steps_23+1)),
                                       end=y2_max+(y2_step*num_of_steps_23))
            
            #Set the 3rd y-axis, range-name, range:
            extra_y_ranges_2 = Range1d(start=y3_min, end=y3_max)
            
            
        #Check if the difference between y-axis 3 and the other axes is an odd/even number:
        #I.e. (diff_13%2==1) and (diff_23%2==0)
        else:
            
            #Set the range of the 1st y-axis:
            y_range = Range1d(start=y1_min-(y1_step*(num_of_steps_13+1)),
                              end=y1_max+(y1_step*num_of_steps_13))
            
            #Set the 2nd y-axis, range-name, range:
            extra_y_ranges_1 = Range1d(start=y2_min-(y2_step*num_of_steps_23),
                                       end=y2_max+(y2_step*num_of_steps_23))
            
            #Set the 3rd y-axis, range-name, range:
            extra_y_ranges_2 = Range1d(start=y3_min, end=y3_max)
        
    else:
        y_range = None
        extra_y_ranges_1 = None
        extra_y_ranges_2 = None
      
        
    #Return y-range for primary and secondary y-axes:
    return y_range, extra_y_ranges_1, extra_y_ranges_2


In [ ]:
def get_colormap(num_of_items):
      
    """
    Project:         'ICOS Carbon Portal'
    Created:          Fri Apr 04 17:00:00 2019
    Last Changed:     Fri Apr 04 17:00:00 2019
    Version:          1.0.0
    Author(s):        Karolina
    
    Description:      Function that takes an integer representing the total number of items that should receive
                      a sepparate color and  returns a colormap (i.e. a list of strings, where every string
                      represents a different color in hexadecimal code) with the same amount of colors.
                      The function can return colormaps for 1 - 256 items.
                      
    Input parameters: Number of items to be colored in a sepparate color
                      (var_name: 'num_of_items', var_type: Integer)

    Output:           List of strings (colormap)
    
    """
    
    #Check input:
    if(isinstance(num_of_items, int)):
    
        #import module:
        from bokeh.palettes import all_palettes, Colorblind, magma

        #Check the number of items to be colored (1-2 items):
        if((num_of_items>0) and (num_of_items<3)):
            return ['#2b83ba','#fdae61'] #return colormap selection
        
        #Check the number of items to be colored (3-8 items):
        elif((num_of_items>2) and (num_of_items<9)):
            return all_palettes['Colorblind'][num_of_items] #return colormap selection
        
        #Check the number of items to be colored (9-256 items):
        elif((num_of_items>8) and (num_of_items<257)):
            return magma(num_of_items) #return colormap selection         
        
        #If the number of items exceeds the number 256:
        else:
            print('Error! Number of items to be colored is zero or higher than 256.')
    
    #If the input parameter is not an integer:
    else:
        print('Input is not an integer.')
        

In [ ]:
def get_country_fullname_from_iso3166_2char(countryCode):
    
    #Get iso 3166 translation pandas dataframe:
    country_names_codes_iso3166 = pd.read_csv('data/country_names_codes_iso_3166.csv',
                                              header=0,
                                              delimiter=';')
    
    #Check if the input is a valid 2-character long ISO 3166 country code:
    if ('SE' in country_names_codes_iso3166.Alpha_2_code.values):
        
        #Return the fullname of a country based on the given iso 3166 2-character country code:
        return country_names_codes_iso3166.Country.loc[country_names_codes_iso3166.Alpha_2_code==countryCode].values[0]

    
    #If the input is not a valid 2-character long ISO 3166 country code:
    else:
        print('Error! Invalid ISO 3166 2-char country code')
    

<br>
<br>
<br>

<a id='exploring_atc_l2'></a>

## 2.1. Exploring - ICOS Level 2 Atmospheric Data
This part is dedicated on displaying observations plots for different tracers at different stations. It is subdivided in three parts. In the first part, [Exploring (Single Station - Single Tracer)](#exploring_single_station_single_tracer_atc_l2), it is possible to view plots for one tracer and one station at a time. An interactive map with depicting the location of the chosen station is also included. The second part [Exploring (Single Station - Multiple Tracers)](#exploring_single_station_multiple_tracers_atc_l2) presents plots for one or more tracer observations from a specific station. Finally, the last part, [Exploring (Single Tracer - Multiple Stations)](#exploring_single_tracer_multiple_stations_atc_l2), presents tracer observations from one or more stations in the same plot.

<br>

Each part is subdivided into two parts; the first part contains explanatory text regarding how the result was computed and how to use the interactive elements (i.e. widgets) of the plot, whilst the second part presents the Python code used.

<br>
<br>
<div style="text-align: right"> 
    <a href="#introduction">Back to top</a>
</div>

<a id='exploring_single_station_single_tracer_atc_l2'></a>

### 2.1.1. Exploring (Single Station - Single Tracer) - ICOS Level 2 Atmospheric Data
This part is dedicated on exploring the tracer-observations for one station at a time. The user can select to view results for different tracers and different stations in different colors. The selection is handled by a set of dropdown-list widgets. The result of the selection becomes visible once the "Update Plot"-button is clicked. Note that the color of the "Update Plot"-button is light blue when the system is processing.

An interactive toolbar is available, on the right side of the plot, with tools such as _pan_, _BoxZoom_ (zoom-in by rectangle), _WheelZoom_ (zoom-in by scrolling), _undo_(undo the last change in the plot's state), _redo_(redo the last change in the plot's state), _reset_ (reset plot to initial state), _save_ (save plot as png) and _hover_ (display tooltips on hover-events). The tools in the toolbar can be activated or deactivated by _on-click_-events. An active tool is highligted with a blue line on the left side of its corresponding icon.

The selection of a station in the dropdown list also produces an interactive map with the locations of all stations. The location of the selected station is highlighted. It is possible to get additional information about a station by hovering over its location on the map. To get more detailed information click on the station's location and be redirected to the station's landing page hosted on ICOS Carbon Portal. The location of the "clicked" station will be highlighted in light gray. Click on _reset_ to set the map to its initial state. 

An interactive toolbar is included on the right side of the map. Note that the _TapTool_ has to be active in order to be redirected to the selected station's landing page on the ICOS Carbon Portal. The same goes for the _HoverTool_.

<br>
<div style="text-align: right"> 
    <a href="#exploring_atc_l2">[Back to exploring]</a>
</div>
<br>
<br>

#### 2.1.1.1. Exploring (Single Station - Single Tracer) - ICOS Level 2 Atmospheric Data ---> Python Code
This section contains the Python code used to present a plot with tracer-values for a given station and a map over the stations of the ICOS network. Note that the following code may include function calls to functions in the code-cells above.

<br>
<div style="text-align: right"> 
    <a href="#exploring_plot_atc_l2">[Go to plot]</a>
</div>

In [ ]:
def create_station_labels(lookup_df):
    
    """
    Project:         'ICOS Carbon Portal'
    Created:          Mon Apr 01 10:26:00 2019
    Last Changed:     Mon Apr 01 10:26:00 2019
    Version:          1.0.0
    Author(s):        Karolina
    
    Description:      Return a list of tuples. Everey tuple contains the label for a specific station and a
                      list with the station's 3-character code and sampling height. The first item of every tuple
                      is used as a label in the station dropdown list (i.e. Select or Multi-select widget).
    Input parameters: lookup table (pandas dataframe)
    Output:           List of tuples (e.g. [('Gartow (alt. 30.0)', ['GAT', '30.0']),...]
                      tuple: 
                            1. Constructed Station Label (var_type: String)
                            2. List of two items
                                i. Station 3-character Code (var_type: String)
                                ii. Station Sampling Height (var_type: String)

    """
    
    #Filter the dataframe and get a dataframe of unique combinations of station-names and 
    #corresponding sampling heights:
    df = lookup_df.filter(['stationName', 'height','stationId']).drop_duplicates(subset=['stationName',
                                                                                         'height',
                                                                                         'stationId'])

    #Get a list of tuples for every station that has provided tracer-data:
    #Every tuple is constructed like: ('Gartow (alt. 30.0)', ['GAT', '30.0'])
    station_labels = [(df.stationName.iloc[i]+ " (alt. " + df.height.iloc[i] + ")",
                       [df.stationId.iloc[i], df.height.iloc[i]]) for i in range(len(df))]
    
    #Return list:
    return station_labels
    

In [ ]:
def plotmap(stations_df, selected_station):
    
    """
    Project:         'ICOS Carbon Portal'
    Created:          Tue Feb 04 10:40:00 2020
    Last Changed:     Tue Feb 04 10:40:00 2020
    Version:          1.0.0
    Author(s):        Karolina
    
    Description:      Function that takes a dataframe containing info about ICOS Stations and the 3-character
                      station code of a selected station as input and returns an interactive Folium Map, with
                      the location of the selected station highlighted in red. 
                      Folium (URL): https://python-visualization.github.io/folium/quickstart.html
                      
    Input parameters: 1. Dataframe with Information regarding ICOS Stations
                         (var_name: 'stations_df', var_type: Pandas Dataframe)
                      2. Station 3-character Code
                         (var_name: 'selected_station', var_type: String)

    Output:           Folium Map (Folium Map Object)
    
    """
    
    #Import modules:
    import folium

    #Create folium map-object:
    m = folium.Map(
        location=[60.07921, 10.0000],
        zoom_start=3)

    #Add marker-tooltip:
    tooltip = 'Click to view station info'

    def add_marker(map_obj, station_code, marker_color):

        #Add popup text:
        popup=folium.Popup("""<meta content="text/html; charset=UTF-8"><style>td{padding: 3px;}</style><table>"""+
                           '<tr><td>Name:</td><td><b>'+stations_df.stationName.loc[stations_df.stationId==station_code].values[0]+'</b></td></tr>'+
                           '<tr><td>Code:</td><td><b>'+station_code+'</b></td></tr>'+
                           '<tr><td>Country:</td><td><b>'+stations_df.Country.loc[stations_df.stationId==station_code].values[0]+'</b></td></tr>'+
                           '<tr><td>Latitude:</td><td><b>'+str(stations_df.lat.loc[stations_df.stationId==station_code].values[0])+'</b></td></tr>'+
                           '<tr><td>Longitude:</td><td><b>'+str(stations_df.lon.loc[stations_df.stationId==station_code].values[0])+'</b></td></tr>'+
                           '<tr><td>URL:</td><td><b><a href="'+stations_df.station.loc[stations_df.stationId==station_code].values[0]+
                           '"target="_blank">link</a></b></td></tr></table>',
                           max_width=450)

        #Create marker and add it to the map:
        folium.Marker(location=[float(stations_df.lat.loc[stations_df.stationId==station_code].values[0]),
                                float(stations_df.lon.loc[stations_df.stationId==station_code].values[0])],
                      popup=popup,
                      icon=folium.Icon(color=marker_color, icon='cloud'),
                      tooltip=tooltip).add_to(map_obj)


    #Get list of stations (not incl. selected station):
    station_ls = [i for i in stations_df.stationId.values if i!=selected_station]  

    #Create markers for all stations except selected station:
    for st in station_ls:
        add_marker(m, st, 'orange') 

    #Add marker for selected station:
    add_marker(m, selected_station, 'darkred')

    #Show map:
    display(m)



In [ ]:
def plot_icos_single_station_binary(df_data, station_info_dict, tracer_info_dict, level=2, color='#0F0C08'):
    
    """
    Project:         'ICOS Carbon Portal'
    Created:          Mon Apr 07 09:30:00 2019
    Last Changed:     Mon Apr 07 09:30:00 2019
    Version:          1.0.0
    Author(s):        Karolina
    
    Description:      Function that plots the content of an ICOS Level-1 or Level-2 Atmospheric Data file to an 
                      interactive plot using the Bokeh interactive visualization library.
                      Bokeh (URL): https://bokeh.pydata.org/en/latest/
                      
    Input parameters: 1. ICOS Level-1 or Level-2 Atmospheric Observation Data
                         (var_name: 'df_data', var_type: pandas dataframe) 
                      2. Dictionary with station info
                         (var_name: 'station_info_dict', var_type: dictionary) 
                      3. Dictionary with tracer info
                         (var_name: 'tracer_info_dict', var_type: dictionary) 
                      4. Data level [optional]
                         (var_name: 'level', var_type: Integer)
                      5. Color for Line- or Circle Glyph [optional]
                         (var_name: 'color', var_type: String)
                     
    
    Default value for color: The default value for line-glyph or circle-glyph color is "lightblue".  
    
    Default value for level: The default value for data level is "2". Function calls for Level-2 data do not have
                             to include a value for the level input parameter. 
    
    Output:           Bokeh Figure Object (plot) 
    
    """
    
    #Import modules to create figure:
    import pandas as pd
    from bokeh.plotting import figure
    from bokeh.models import ColumnDataSource, HoverTool, Label, Legend
    from datetime import datetime

    #Dictionary for subscript transformations of numbers:
    SUB = str.maketrans("0123456789", "₀₁₂₃₄₅₆₇₈₉")
    SUP = str.maketrans("0123456789", "⁰¹²³⁴⁵⁶⁷⁸⁹")

    #Define Datasets:
    x = pd.to_datetime(df_data['TIMESTAMP'], unit='ms')
    y = df_data[tracer_info_dict['tracer_info'].replace(' mixing ratio (dry mole fraction)','').lower()].values #give tracer as parameter, where tracer can be ['co', 'co2']
    z = df_data["NbPoints"].values
    w = df_data["Stdev"].values
    o = df_data["Flag"].values
    #u = df_data["InstrumentId"].values

    #Create a ColumnDataSource object:
    source = ColumnDataSource( data = {'x':x, 'y':y, 'z':z, 'w':w, 'o':o,} )


    #Create a figure object:
    p = figure(plot_width=900,
               plot_height=400,
               x_axis_label='Time (UTC)', 
               y_axis_label=tracer_info_dict['tracer_info'].replace(' mixing ratio (dry mole fraction)','').translate(SUB)+ ' (' +
               tracer_info_dict['tracer_unit'].translate(SUP) + ')',
               x_axis_type='datetime',
               title = tracer_info_dict['tracer_info'].translate(SUB)+'    '+
               station_info_dict['station_name'] +', '+
               station_info_dict['station_country'] +', '+
               station_info_dict['station_sampling_height'] +' m.a.g.l.',
               tools='pan,box_zoom,wheel_zoom,undo,redo,reset,save')

    #Create glyphs:
    g0 = p.circle('x','y', source=source, radius=.02, color=color)
    
    #If data is level-2 data:
    if(level==2):
        g1 = p.line('x','y', source=source, line_width=1, color=color, name=tracer_info_dict['tracer_info'].replace(' mixing ratio (dry mole fraction)','').translate(SUB))
    
    #If data is level-1 data:
    else:
        g1 = p.line('x','y', source=source, line_width=2, color=color, name=tracer_info_dict['tracer_info'].replace(' mixing ratio (dry mole fraction)','').translate(SUB),
                    line_dash='dotted')
      
    
    #Add tooltip on hover:
    p.add_tools(HoverTool(tooltips=[
        ('Station Code',station_info_dict['station_code']),
        ('Latitude',station_info_dict['station_lat']),
        ('Longitude',station_info_dict['station_lon']),
        ('Time (UTC)','@x{%Y-%m-%d %H:%M:%S}'),
        (tracer_info_dict['tracer_info'].replace(' mixing ratio (dry mole fraction)','').translate(SUB),'@y{0.f}'),
        ('St dev', '@w{0.f}'),
        ('NbPoints', '@z'),
        ('Flag', '@o')
        ],
        formatters={
            '@x'      : 'datetime', # use 'datetime' formatter for 'date' field
            },
        # display a tooltip whenever the cursor is vertically in line with a glyph
        mode='vline'
        ))    


    #Set title attributes:
    p.title.align = 'center'
    p.title.text_font_size = '13pt'
    p.title.offset = 15

    #Set label font style:
    p.xaxis.axis_label_text_font_style = 'normal'
    p.yaxis.axis_label_text_font_style = 'normal'
    p.xaxis.axis_label_standoff = 15 #Sets the distance of the label from the x-axis in screen units
    p.yaxis.axis_label_standoff = 15 #Sets the distance of the label from the y-axis in screen units

    #Set the copyright label position:
    label_opts = dict(x=0, y=10,
                      x_units='screen', y_units='screen')

    #Create a label object and format it:
    caption1 = Label(text="© ICOS ERIC", **label_opts)
    caption1.text_font_size = '8pt'
    
    #Deactivate hover-tool, which is by default active:
    p.toolbar.active_inspect = None

    #Add label to plot:
    p.add_layout(caption1, 'below')

    #return plot:
    return p

In [ ]:
def update_icos_single_station_plot_binary(data_obj_id_ls, station, tracer, color):
    
    """
    Project:         'ICOS Carbon Portal'
    Created:          Fri Apr 07 10:05:00 2019
    Last Changed:     Fri Mar 27 10:05:00 2020
    Version:          1.1.0
    Author(s):        Karolina
    
    Description:      Function that gets the user's selection of station, tracer and color as input parameters, 
                      accesses and reads in the corresponding datafiles and outputs a Bokeh plot with observations
                      for the selected tracer.
                      
    Input parameters: 1. list of data object IDs
                         (var_name: 'data_obj_id_ls', var_type: List of Strings)
                      2. list of station info, containing 3-character station code and sampling height,
                         e.g. ['GAT', '30.0'] 
                         (var_name: 'station', var_type: List of Strings)
                      3. gas/traser, e.g. 'co', 'co2' or 'ch4'
                         (var_name: 'tracer', var_type: String)
                      4. color in hexadecimal code (var_name: 'color', var_type: String)

    Output:           Bokeh Plot
    
    """
    
    #import modules:
    from bokeh.layouts import column
    from icoscp.cpb.dobj import Dobj
    
    #Create dictionary to store tracer info:
    tracer_info_dict = {}
    
    #Create dict to store the station info:
    station_info_dict = {}
    
    #Create a file object from the 1st object in the data object id list:  
    file = Dobj(data_obj_id_ls[0])
    
    #Get the tracer description:
    tracer_info_dict['tracer_info'] = file.info[1].valueType.loc[file.info[1].colName==tracer].values[0]
    
    #Get tracer unit:
    tracer_info_dict['tracer_unit'] = file.info[1].unit.loc[file.info[1].colName==tracer].values[0]
    
    
    #Get pandas dataframe with all ICOS stations:
    icos_stations_df = get_coords_icos_stations_atc()
    
    #Get station info:
    station_info_dict['station_name'] = icos_stations_df.stationName.loc[icos_stations_df.stationId==station[0]].values[0]
    station_info_dict['station_code'] = station[0]
    station_info_dict['station_sampling_height'] = station[1]
    station_info_dict['station_country_code'] = icos_stations_df.Country.loc[icos_stations_df.stationId==station[0]].values[0]
    station_info_dict['station_country'] = get_country_fullname_from_iso3166_2char(station_info_dict['station_country_code'])
    station_info_dict['station_lat'] = icos_stations_df.lat.loc[icos_stations_df.stationId==station[0]].values[0]
    station_info_dict['station_lon'] = icos_stations_df.lon.loc[icos_stations_df.stationId==station[0]].values[0]
    
    
    #Create list to store the data dataframes of all data object IDs:
    data_df_ls = []
    
    #Loop through every data object ID in the list:
    for dobjid in data_obj_id_ls: 
        
        #Get a pandas dataframe with all the columns for the selected data-object id:
        obs_data_df = Dobj(dobjid).get()
        
        #Add data dataframe of the current data object ID to the list:
        data_df_ls.append(obs_data_df)
        
    #Concatenate data dataframes to one dataframe:
    data_df = pd.concat(data_df_ls)
    
    #Sort the dataframe index in ascending order:
    data_df.sort_index(inplace=True)
    
    
    ### Plot ###
    #Plot station:
    p = plot_icos_single_station_binary(data_df,
                                        station_info_dict,
                                        tracer_info_dict,
                                        color=color)

    #Return plot:
    return p

In [ ]:
def create_widgets_exploring():
    
    """
    Project:         'ICOS Carbon Portal'
    Created:          Fri Apr 07 10:00:00 2019
    Last Changed:     Fri Apr 07 10:00:00 2019
    Version:          1.0.0
    Author(s):        Karolina
    
    Description:      Function that creates a set of widgets; a tracer dropdown list, a station dropdown list,
                      a colorpicker and a button, populates the dropdown lists with values, captures the user's
                      input and calls a function to update the contents of the "exploring"-plot.
                      
    Input parameters: No input parameter/s

    Output:           Plot with Map or Warning Message
    
    """
    
    #Dictionary for subscript transformations of numbers:
    SUB = str.maketrans("0123456789", "₀₁₂₃₄₅₆₇₈₉")

    #Create lookup dataframe:
    df_lookup = create_lookup_df_atc_L2()
    
    #Create a list including all tracers (e.g. CO2, CO, CH4)
    tracers = df_lookup.variable.unique().tolist()
    
    #reverse list order:
    tracers.reverse()

    #Create widgets:
    tracer = Dropdown(options = tracers)
    station =  Dropdown(options = create_station_labels(df_lookup))

    #Function that calls functions to update the plot/s and/or map,
    #based on the selected tracer, station and color:
    def update_plot_func(Tracer, Station, color, Citation):
        

        #Get tracer short:
        tracer = Tracer.replace(' mixing ratio (dry mole fraction)', '').lower()

        #Get a list of data obect URLs that refer to the selected station and tracer:
        data_obj_url_ls = df_lookup.dobj.loc[(df_lookup.stationId==Station[0]) &
                                             (df_lookup.height==Station[1]) &
                                             (df_lookup.variable==Tracer)].values
        
        
        #If L2-data is available for the selected tracer and station:
        if(data_obj_url_ls.size>0):
            
            #Get a list of data object IDs (L2-data):
            data_obj_id_ls = [data_obj_url_ls[i].replace('https://meta.icos-cp.eu/objects/', '') 
                                 for i in range(data_obj_url_ls.size)]
            
            #Call function to return plot for the selected station:
            p = update_icos_single_station_plot_binary(data_obj_id_ls, Station, tracer, color)
        
            #Output should be in the notebook
            output_notebook()

            #Show plot:
            show(p, notebook_handle=True)
            
            #If the "citation" checkbox is checked:
            if(Citation):
                
                #Get a list with citation info for every ICOS Level-2 data object:
                cit_ls_L2 = [get_icos_citation(dobj).cit.iloc[0] for dobj in data_obj_url_ls]
                    
                #Print citation title:
                print('\n\n\033[1m' + 'Data Citation:' +  '\033[0m')
                    
                #Loop through all citations:
                for cit in cit_ls_L2:
                    
                    #Print data object citation:
                    printmd("<sub>"+cit+"</sub>")
                    
            #Get pandas dataframe with all ICOS stations:
            icos_stations_df = get_coords_icos_stations_atc()
            
            #Show map:
            plotmap(icos_stations_df, Station[0])

        #If no L2-data are available for the selected tracer and station:
        else:
            print("\033[0;31;1m "+'No '+tracer.upper().translate(SUB)+' Level-2 data available for the selected station...'+"\033[0;31;0m\n\n")
                

           


    #Create function that contains a box of widgets:
    interact_c = interact_manual(update_plot_func,
                                 Tracer=tracer,
                                 Station=station,
                                 color=ColorPicker(concise=False,
                                                   description='Pick a color',
                                                   value='#3973ac',
                                                   disabled=False),
                                 Citation=Checkbox(value=True, description='Citation', disabled=False))

    #Set the font of the widgets included in interact_manual:
    interact_c.widget.children[0].layout.width = '430px'
    interact_c.widget.children[0].layout.margin = '40px 2px 2px 2px'
    interact_c.widget.children[1].layout.width = '430px'
    interact_c.widget.children[2].layout.width = '430px'
    interact_c.widget.children[3].layout.width = '430px'
    interact_c.widget.children[4].description = 'Update Plot'
    interact_c.widget.children[4].button_style = 'danger'
    interact_c.widget.children[4].style.button_color = '#3973ac'
    interact_c.widget.children[4].layout.margin = '10px 10px 40px 180px' # top/right/bottom/left

<a id='exploring_plot_atc_l2'></a>
#### 2.1.1.2. Exploring (Single Station - Single Tracer) - ICOS Level 2 Atmospheric Data ---> Plot

In [ ]:
#Call function to display widgets for the corresponding plot type:
create_widgets_exploring()

<div style="text-align: right"> 
    <a href="#exploring_single_station_single_tracer_atc_l2">[Back to exploring - single station single tracer]</a>
    &ensp;&ensp;
    <a href="#introduction">Back to top</a>
</div>
<br>

<a id='exploring_single_station_multiple_tracers_atc_l2'></a>

### 2.2. Exploring (Single Station - Multiple Tracers) - ICOS Level 2 Atmospheric Data
This part is dedicated on exploring the tracer-observations for one station at a time. The user can select to view results for different tracers and different stations. The selection is handled by a set of dropdown-list widgets. It is possible to select and view results for more than one tracers. Multiple values can be selected with _shift_ and/or _ctrl_ (or _command_) pressed and _mouse clicks_. The result of the selection becomes visible once the "Update plot"-button is clicked. 

An interactive toolbar is available, on the right side of theplot, with tools such as _pan_, _ZoomBox_ (zoom-in by rectangle), _WheelZoom_ (zoom-in by scrolling), _undo_ (undo the last change in the plot's state), _redo_ (redo the last change in the plot's state), _reset_ (reset plot to initial state), _save_ (save plot as png) and _hover_ (display tooltips on hover-events). The tools in the toolbar can be activated or deactivated by _on-click_-events. An active tool is highligted with a blue line on the left side of its corresponding icon. 


<br>
<div style="text-align: right"> 
    <a href="#exploring_atc_l2">[Back to exploring]</a>
</div>

#### 2.2.1. Exploring (Single Station - Multiple Tracers) -  ICOS Level 2 Atmospheric Data ---> Python Code

<div style="text-align: right"> 
    <a href="#exploring_single_station_multiple_tracers_plot_atc_l2">[Go to plot]</a>
</div>

In [ ]:
def plot_icos_single_station_multiple_tracers_binary(df_list):
    
    """
    Project:         'ICOS Carbon Portal'
    Created:          Fri Apr 07 10:00:00 2019
    Last Changed:     Fri Apr 07 10:00:00 2019
    Version:          1.0.0
    Author(s):        Karolina
    
    Description:      Function that creates an interactive Bokeh plot with ICOS Level-2 Atmospheric Data 
                      ('CO2', 'CO', 'CH4'). The plot has a sepparate y-axis for every tracer.
                      
    Input parameters: List of lists of station-info dictionaries, tracer-info dictionaries and
                      data dataframes with ICOS Level-2 Atmospheric Data
                      (var_name: "df_list", var_type: List of Pandas DataFrames)

    Output:           Bokeh Plot
    
    """
    
    #Import modules to create figure:
    import pandas as pd
    from bokeh.plotting import figure
    from bokeh.models import ColumnDataSource, HoverTool, Label, Legend, LinearAxis, Range1d, SingleIntervalTicker
    from datetime import datetime
    
 
    #Dictionaries for subscript/superscript transformations of numbers:
    SUB = str.maketrans("0123456789", "₀₁₂₃₄₅₆₇₈₉")
    SUP = str.maketrans("0123456789", "⁰¹²³⁴⁵⁶⁷⁸⁹")

    #Create a figure object:
    p = figure(plot_width=900,
               plot_height=500,
               x_axis_label='Time (UTC)', 
               y_axis_label=  df_list[0][1]['tracer_info'].replace(' mixing ratio (dry mole fraction)','').translate(SUB)+' (' +
               df_list[0][1]['tracer_unit'].translate(SUP) + ')',
               x_axis_type='datetime',
               title = 'Tracer Observations ('+
               df_list[0][0]['station_name']+', '+
               df_list[0][0]['station_country']+', '+
               df_list[0][0]['station_sampling_height']+' m. a. g. l.)',
               tools='pan,box_zoom,wheel_zoom,reset, save')
    

    #Dictionary containing tracer colors:
    colors = {'co':'#ff7502',#'#bf812d','#d6604d','#CD6839','#f46d43','#d2691e', '#fc8d59', 
              'co2':'#543005',#'#48240A','#993404', 
              'ch4':'#e4c981'}#'#fee090'}'#dfc27d'}'#FFB90F'}'#fdcc8a'}
    
    #Create an empty list that will store the legend info:
    legend_it = []
    
    #Extract time and tracer values for every tracer category:
    x = pd.to_datetime(df_list[0][2]['TIMESTAMP'], unit='ms') 
    y = df_list[0][2][df_list[0][1]['tracer_info'].replace(' mixing ratio (dry mole fraction)','').lower()]

    #Create a circle and line glyph for the values of every emission category:
    r0 = p.circle(x, y, radius=.12, color=colors[df_list[0][1]['tracer_info'].replace(' mixing ratio (dry mole fraction)','').lower()])
    r1 = p.line(x, y, line_width=1, color=colors[df_list[0][1]['tracer_info'].replace(' mixing ratio (dry mole fraction)','').lower()],
                name=df_list[0][1]['tracer_info'].replace(' mixing ratio (dry mole fraction)','').translate(SUB))

    #Add the name and glyph info (i.e. colour and marker type) to the legend:
    legend_it.append((df_list[0][1]['tracer_info'].replace(' mixing ratio (dry mole fraction)','').translate(SUB), [r0,r1]))
    
    
    #If 2 tracers have been selected:
    if(len(df_list)==2):
        
        #Get the total min/max values for every tracer:
        tracer1_min = df_list[0][2][df_list[0][1]['tracer_info'].replace(' mixing ratio (dry mole fraction)','').lower()].min()
        tracer1_max = df_list[0][2][df_list[0][1]['tracer_info'].replace(' mixing ratio (dry mole fraction)','').lower()].max()
        tracer2_min = df_list[1][2][df_list[1][1]['tracer_info'].replace(' mixing ratio (dry mole fraction)','').lower()].min()
        tracer2_max = df_list[1][2][df_list[1][1]['tracer_info'].replace(' mixing ratio (dry mole fraction)','').lower()].max()
    
        #If two tracers are selected anf if one of the tracers is co2:
        if((len(df_list)==2) and (df_list[0][1]['tracer_info'].replace(' mixing ratio (dry mole fraction)','')=='CO2')):
            p.y_range, p.extra_y_ranges = set_yranges_2y(rounddown_20(tracer1_min),
                                                         roundup_20(tracer1_max), 
                                                         rounddown_100(tracer2_min), 
                                                         roundup_100(tracer2_max), 20.0, 100.0, 'Yaxis2')

            #Set primary y-axis ticker:
            ticker_1 = SingleIntervalTicker(interval= 20)

            #Add primary y-axis ticker to plot:
            p.yaxis.ticker = ticker_1

            #Set secondary y-axis ticker:
            ticker_2 = SingleIntervalTicker(interval=100)

            #Create 2nd y-axis: 
            bg_yaxis = LinearAxis(y_range_name="Yaxis2",
                                  axis_label=df_list[1][1]['tracer_info'].replace(' mixing ratio (dry mole fraction)','').translate(SUB)+' (' +
                                  df_list[1][1]['tracer_unit'].translate(SUP) + ')',
                                  ticker=ticker_2, axis_label_standoff = 15) 

            #Add secondary y-axis to plot:
            p.add_layout(bg_yaxis, 'right')

        else: 
            p.y_range, p.extra_y_ranges = set_yranges_2y(rounddown_100(tracer1_min),
                                                         roundup_100(tracer1_max),
                                                         rounddown_100(tracer2_min),
                                                         roundup_100(tracer2_max),
                                                         100.0, 100.0, 'Yaxis2')

            #Set primary y-axis ticker:
            ticker_1 = SingleIntervalTicker(interval= 100)

            #Add primary y-axis ticker to plot:
            p.yaxis.ticker = ticker_1

            #Set secondary y-axis ticker:
            ticker_2 = SingleIntervalTicker(interval=100)

            #Create 2nd y-axis: 
            yaxis_2 = LinearAxis(y_range_name="Yaxis2",
                                 axis_label=df_list[1][1]['tracer_info'].replace(' mixing ratio (dry mole fraction)','').translate(SUB)+' (' +
                                 df_list[1][1]['tracer_unit'].translate(SUP) + ')', 
                                 ticker=ticker_2, 
                                 axis_label_standoff = 15) 

            #Add secondary y-axis to plot:
            p.add_layout(yaxis_2, 'right')
            
            
            
            
            
        #Set the text color of the yaxis for both y-axes:
        p.yaxis[0].axis_label_text_color = colors[df_list[0][1]['tracer_info'].replace(' mixing ratio (dry mole fraction)','').lower()]
        p.yaxis[1].axis_label_text_color = colors[df_list[1][1]['tracer_info'].replace(' mixing ratio (dry mole fraction)','').lower()]
    
    
    
    
    
        #Extract time and tracer values for every tracer category:
        x2i = pd.to_datetime(df_list[1][2]['TIMESTAMP'], unit='ms') 
        y2i = df_list[1][2][df_list[1][1]['tracer_info'].replace(' mixing ratio (dry mole fraction)','').lower()]

        #Create a circle and line glyph for the values of every emission category:
        r2 = p.circle(x2i, y2i, radius=.12, color=colors[df_list[1][1]['tracer_info'].replace(' mixing ratio (dry mole fraction)','').lower()],
                      y_range_name="Yaxis2")
        r3 = p.line(x2i, y2i, line_width=1, color=colors[df_list[1][1]['tracer_info'].replace(' mixing ratio (dry mole fraction)','').lower()],
                    name=df_list[1][1]['tracer_info'].replace(' mixing ratio (dry mole fraction)','').translate(SUB), y_range_name="Yaxis2")

        #Add the name and glyph info (i.e. colour and marker type) to the legend:
        legend_it.append((df_list[1][1]['tracer_info'].replace(' mixing ratio (dry mole fraction)','').translate(SUB), [r2,r3]))

        
 
        
        
    #If three tracers have been selected:    
    if(len(df_list)==3):
        
        #Get the total min/max values for every tracer:
        tracer1_min = df_list[0][2][df_list[0][1]['tracer_info'].replace(' mixing ratio (dry mole fraction)','').lower()].min()
        tracer1_max = df_list[0][2][df_list[0][1]['tracer_info'].replace(' mixing ratio (dry mole fraction)','').lower()].max()
        tracer2_min = df_list[1][2][df_list[1][1]['tracer_info'].replace(' mixing ratio (dry mole fraction)','').lower()].min()
        tracer2_max = df_list[1][2][df_list[1][1]['tracer_info'].replace(' mixing ratio (dry mole fraction)','').lower()].max()
        tracer3_min = df_list[2][2][df_list[2][1]['tracer_info'].replace(' mixing ratio (dry mole fraction)','').lower()].min()
        tracer3_max = df_list[2][2][df_list[2][1]['tracer_info'].replace(' mixing ratio (dry mole fraction)','').lower()].max()
    
        #Get the ranges for every y-axis:
        p.y_range, p.extra_y_ranges['Yaxis2'], p.extra_y_ranges['Yaxis3']= set_yranges_3y(rounddown_20(tracer1_min),
                                                                                          roundup_20(tracer1_max), 
                                                                                          rounddown_100(tracer2_min), 
                                                                                          roundup_100(tracer2_max),
                                                                                          rounddown_100(tracer3_min), 
                                                                                          roundup_100(tracer3_max),
                                                                                          20.0, 100.0, 100.0)

        #Set primary y-axis ticker:
        ticker_1 = SingleIntervalTicker(interval= 20)

        #Add primary y-axis ticker to plot:
        p.yaxis.ticker = ticker_1

        #Set secondary y-axis ticker:
        ticker_2 = SingleIntervalTicker(interval=100)

        #Create 2nd y-axis: 
        yaxis_2 = LinearAxis(y_range_name="Yaxis2",
                             axis_label=df_list[1][1]['tracer_info'].replace(' mixing ratio (dry mole fraction)','').translate(SUB)+' (' +
                             df_list[1][1]['tracer_unit'].translate(SUP) + ')',
                             ticker=ticker_2, 
                             axis_label_standoff = 15) 
        
        #Create 3rd y-axis:
        yaxis_3 = LinearAxis(y_range_name="Yaxis3", 
                             axis_label=df_list[2][1]['tracer_info'].replace(' mixing ratio (dry mole fraction)','').translate(SUB)+' (' +
                             df_list[2][1]['tracer_unit'].translate(SUP) + ')',
                             ticker=ticker_2,
                             axis_label_standoff = 15) 
        
        #Add secondary y-axis to plot:
        p.add_layout(yaxis_2, 'right') 
        
        #Add third y-axis to plot:
        p.add_layout(yaxis_3, 'right') 
        
        df_list[1][1]['tracer_info'].replace(' mixing ratio (dry mole fraction)','').lower
        
        #Set yaxis tick-label color for all 3 y-axes:
        p.yaxis[0].major_label_text_color = colors[df_list[0][1]['tracer_info'].replace(' mixing ratio (dry mole fraction)','').lower()]
        p.yaxis[1].major_label_text_color = colors[df_list[1][1]['tracer_info'].replace(' mixing ratio (dry mole fraction)','').lower()]
        p.yaxis[2].major_label_text_color = colors[df_list[2][1]['tracer_info'].replace(' mixing ratio (dry mole fraction)','').lower()]
        
        #Set the text color of the yaxis for all 3 y-axes:
        p.yaxis[0].axis_label_text_color = colors[df_list[0][1]['tracer_info'].replace(' mixing ratio (dry mole fraction)','').lower()]
        p.yaxis[1].axis_label_text_color = colors[df_list[1][1]['tracer_info'].replace(' mixing ratio (dry mole fraction)','').lower()]
        p.yaxis[2].axis_label_text_color = colors[df_list[2][1]['tracer_info'].replace(' mixing ratio (dry mole fraction)','').lower()]
        
        
        
        
        
        
        
        

        #Extract time and tracer values for every tracer category:
        x2 = pd.to_datetime(df_list[1][2]['TIMESTAMP'], unit='ms') 
        y2 = df_list[1][2][df_list[1][1]['tracer_info'].replace(' mixing ratio (dry mole fraction)','').lower()]

        #Create a circle and line glyph for the values of every emission category:
        r2 = p.circle(x2, y2, radius=.12,
                      color=colors[df_list[1][1]['tracer_info'].replace(' mixing ratio (dry mole fraction)','').lower()],
                      y_range_name="Yaxis2")
        r3 = p.line(x2, y2, line_width=1,
                    color=colors[df_list[1][1]['tracer_info'].replace(' mixing ratio (dry mole fraction)','').lower()],
                    name=df_list[1][1]['tracer_info'].replace(' mixing ratio (dry mole fraction)','').translate(SUB), y_range_name="Yaxis2")

        #Add the name and glyph info (i.e. colour and marker type) to the legend:
        legend_it.append((df_list[1][1]['tracer_info'].replace(' mixing ratio (dry mole fraction)','').translate(SUB), [r2,r3]))
        
        #Extract time and tracer values for every tracer category:
        x3 = pd.to_datetime(df_list[2][2]['TIMESTAMP'], unit='ms') 
        y3 = df_list[2][2][df_list[2][1]['tracer_info'].replace(' mixing ratio (dry mole fraction)','').lower()]

        #Create a circle and line glyph for the values of every emission category:
        r4 = p.circle(x3, y3, radius=.12,
                      color=colors[df_list[2][1]['tracer_info'].replace(' mixing ratio (dry mole fraction)','').lower()],
                      y_range_name="Yaxis3")
        r5 = p.line(x3, y3, line_width=1,
                    color=colors[df_list[2][1]['tracer_info'].replace(' mixing ratio (dry mole fraction)','').lower()],
                    name=df_list[2][1]['tracer_info'].replace(' mixing ratio (dry mole fraction)','').translate(SUB), y_range_name="Yaxis3")

        #Add the name and glyph info (i.e. colour and marker type) to the legend:
        legend_it.append((df_list[2][1]['tracer_info'].replace(' mixing ratio (dry mole fraction)','').translate(SUB), [r4,r5]))
        
        
        
        
        
    #Add tooltip on hover:
    p.add_tools(HoverTool(tooltips=[
        ('Tracer type','$name'),
        ('Time (UTC)','@x{%Y-%m-%d %H:%M:%S}'),
        ('Tracer value','@y{0.f}'),
        ],
        formatters={
            '@x'      : 'datetime', # use 'datetime' formatter for 'date' field
            },
        # display a tooltip whenever the cursor is vertically in line with a glyph
        mode='vline'
        ))    

    #Create legend:
    legend = Legend(items=legend_it, location= 'bottom_center')
    legend.orientation = 'horizontal'
    legend.click_policy='hide'
    legend.spacing = 10 #sets the distance between legend entries

    #Set title attributes:
    p.title.align = 'center'
    p.title.text_font_size = '13pt'
    p.title.offset = 15

    #Set axis label font style:
    p.xaxis.axis_label_text_font_style = 'normal'
    p.yaxis.axis_label_text_font_style = 'normal'
    p.xaxis.axis_label_standoff = 15 #Sets the distance of the label from the x-axis in screen units
    p.yaxis.axis_label_standoff = 15 #Sets the distance of the label from the y-axis in screen units

    #Set the copyright label position:
    label_opts = dict(x=0, y=10,
                      x_units='screen', y_units='screen')

    #Create a label object and format it:
    caption1 = Label(text="© ICOS ERIC", **label_opts)
    caption1.text_font_size = '8pt'
    
    #Deactivate hover-tool, which is by default active:
    p.toolbar.active_inspect = None

    #Add label to plot:
    p.add_layout(caption1, 'below')
    
    #Add legend to figure:
    p.add_layout(legend, 'below')

    #return plot:
    return p


In [ ]:
def update_exploring_multiple_tracers_binary(selection_list):
    
    """
    Project:         'ICOS Carbon Portal'
    Created:          Tue May 07 10:30:00 2018
    Last Changed:     Tue May 07 10:30:00 2019
    Version:          1.0.0
    Author(s):        Karolina
    
    Description:      Function that takes as input a list of sublists, where every sublist contains
                      a data object ID, a tracer string and a station ID string.
                      The function reads the corresponding ICOS Level-2 Atmospheric Tracer data files
                      for every sublist into sepparate pandas dataframes. Every data file produces two 
                      sepparate pandas dataframes; metadata dataframe and data dataframe.
                      These dataframes are then set as input parameters to a plot function, 
                      that returns a Bokeh Figure (plot).
                      The Bokeh Figure is then returned as output.
                      
                      
    Input parameters: List with sublists of data object IDs & tracers
                      (var_name: 'selection_list', var_type: List)

    Output:           Bokeh Plot
    
    """
    
    #import modules:
    from bokeh.layouts import column
    from icoscp.cpb.dobj import Dobj
    
    #Create lists to store lists of metadata & data dataframes for every station-tracer combination:
    co2_df_list = []
    co_df_list = []
    ch4_df_list = []
    
    #Create dict to store the station info:
    station_info_dict = {}
    
    #Get pandas dataframe with all ICOS stations:
    icos_stations_df = get_coords_icos_stations_atc()
    
    #Get station info:
    station_info_dict['station_name'] = icos_stations_df.stationName.loc[icos_stations_df.stationId==selection_list[0][2]].values[0]
    station_info_dict['station_country_code'] = icos_stations_df.Country.loc[icos_stations_df.stationId==selection_list[0][2]].values[0]
    station_info_dict['station_country'] = get_country_fullname_from_iso3166_2char(station_info_dict['station_country_code'])
    station_info_dict['station_lat'] = icos_stations_df.lat.loc[icos_stations_df.stationId==selection_list[0][2]].values[0]
    station_info_dict['station_lon'] = icos_stations_df.lon.loc[icos_stations_df.stationId==selection_list[0][2]].values[0]
    station_info_dict['station_sampling_height'] =  selection_list[0][3]   
        
    #Loop throught list of data-object-IDs and tracers and download and read file/s with ICOS data:
    for selection in selection_list:
        
        #Check tracer type:
        if(selection[1]=='co2'):
            
            #Create dictionary to store tracer info:
            tracer_info_dict_co2 = {}
            
            #Create a file object from the current data object id:
            file_co2 = Dobj(selection[0])  

            #Get the tracer description:
            tracer_info_dict_co2['tracer_info'] = file_co2.info[1].valueType.loc[file_co2.info[1].colName==selection[1]].values[0]

            #Get tracer unit:
            tracer_info_dict_co2['tracer_unit'] = file_co2.info[1].unit.loc[file_co2.info[1].colName==selection[1]].values[0]
            
            
            #Add data dataframe for the current data object ID to the list:
            co2_df_list.append(file_co2.get())
            
            
        elif(selection[1]=='co'):
            
            #Create dictionary to store tracer info:
            tracer_info_dict_co = {}
            
            #Create a file object from the current data object id:
            file_co = Dobj(selection[0])  

            #Get the tracer description:
            tracer_info_dict_co['tracer_info'] = file_co.info[1].valueType.loc[file_co.info[1].colName==selection[1]].values[0]

            #Get tracer unit:
            tracer_info_dict_co['tracer_unit'] = file_co.info[1].unit.loc[file_co.info[1].colName==selection[1]].values[0]
            
            #Add data dataframe for the current data object ID to the list:
            co_df_list.append(file_co.get())
            
            
        elif(selection[1]=='ch4'):
            
            #Create dict to store the tracer info:
            tracer_info_dict_ch4 = {}
            
            #Create a file object from the current data object id:
            file_ch4 = Dobj(selection[0])  

            #Get the tracer description:
            tracer_info_dict_ch4['tracer_info'] = file_ch4.info[1].valueType.loc[file_ch4.info[1].colName==selection[1]].values[0]

            #Get tracer unit:
            tracer_info_dict_ch4['tracer_unit'] = file_ch4.info[1].unit.loc[file_ch4.info[1].colName==selection[1]].values[0]
            
            #Add data dataframe for the current data object ID to the list:
            ch4_df_list.append(file_ch4.get())
        
        
        #If tracer is not one of the following: "CO2", "CO" or "CH4":
        else:
            print("\033[0;31;1m "+'Error! No support for this tracer!'+"\033[0;31;0m\n\n")
        
        
    #Create list to store lists of metadata & data dataframes for every station-tracer combination:
    info_list = []
        
    #Add all non-empty co2 lists to df_list:
    if(len(co2_df_list)>0):
        
        #Concatenate data dataframes to one dataframe:
        co2_data_df = pd.concat(co2_df_list)

        #Sort the dataframe index in ascending order:
        co2_data_df.sort_index(inplace=True)
            
        #Add the station_info dict, tracer_info dict and data dataframe to a list:
        info_list.append([station_info_dict, tracer_info_dict_co2, co2_data_df])
         
    #Add all non-empty tracer lists to df_list:
    if(len(co_df_list)>0):
        
        #Concatenate data dataframes to one dataframe:
        co_data_df = pd.concat(co_df_list)

        #Sort the dataframe index in ascending order:
        co_data_df.sort_index(inplace=True)
            
        #Add the pair of dataframes as a list to the list of dataframes:
        info_list.append([station_info_dict, tracer_info_dict_co, co_data_df])

    #Add all non-empty tracer lists to df_list:
    if(len(ch4_df_list)>0):
        
        #Concatenate data dataframes to one dataframe:
        ch4_data_df = pd.concat(ch4_df_list)

        #Sort the dataframe index in ascending order:
        ch4_data_df.sort_index(inplace=True)
            
        #Add the pair of dataframes as a list to the list of dataframes:
        info_list.append([station_info_dict, tracer_info_dict_ch4, ch4_data_df])

        
    #Call a function to plot data from all dataframes in the list of dataframes:
    p = plot_icos_single_station_multiple_tracers_binary(info_list)

    
    #Set output channel:
    output_notebook()
    
    #Show plot:
    show(p)

In [ ]:
def create_widgets_exploring_multiple_tracers():
    
    """
    Project:         'ICOS Carbon Portal'
    Created:          Tue May 07 10:30:00 2018
    Last Changed:     Tue May 07 10:30:00 2019
    Version:          1.0.0
    Author(s):        Karolina
    
    Description:      Function that creates a set of widgets; a tracer multiselect dropdown list, a
                      station dropdown list and a button. The function populates the dropdown lists
                      with values and outputs the result.
                      
                      
    Input parameters: No Input Parameter(s)

    Output:           Python Widgets
    
    """

    #Create lookup dataframe:
    df_lookup = create_lookup_df_atc_L2()
    
    #Create a list including all tracers (e.g. CO2, CO, CH4)
    tracers = df_lookup.variable.unique().tolist()
    
    #reverse list order:
    tracers.reverse()

    #Create widgets:
    station =  Dropdown(options = create_station_labels(df_lookup))
    tracer = SelectMultiple(options = tracers,
                            value = [tracers[0]],
                            disabled=False)

    #Function that calls functions to update the plot/s and/or map,
    #based on the selected tracer, station and color:
    def update_plot_func(Tracer, Station, Citation):
        

        #Get a list of Level-2 data object URLs, tracers, station-IDs and station sampling heights
        #that refer to the selected station and tracer/s:
        selection_dobj_url_list = [[df_lookup.dobj.loc[(df_lookup.stationId==Station[0]) &
                                                       (df_lookup.height==Station[1]) & 
                                                       (df_lookup.variable==tracer)].values,
                                    tracer.replace(' mixing ratio (dry mole fraction)', '').lower(),
                                    Station[0],
                                    Station[1]]
                                   for tracer in Tracer
                                   if len(df_lookup.dobj.loc[(df_lookup.stationId==Station[0]) &
                                                             (df_lookup.height==Station[1]) &
                                                             (df_lookup.variable==tracer)].values)>0]

        
        #If Level-2 data are available for the selected tracer and station:
        if(len(selection_dobj_url_list)>0):
            
            #Get a list of lists, where every sublist contains
            #a data-object-id, a tracer-string (e.g. 'ch4'), a station-ID string (e.g. 'GAT')
            #and the station sampling height string (e.g. '30.0'):
            selection_list = [[selection_dobj_url_list[item][0][0].replace('https://meta.icos-cp.eu/objects/', ''),
                               selection_dobj_url_list[item][1],
                               selection_dobj_url_list[item][2],
                               selection_dobj_url_list[item][3]]
                              for item in range(len(selection_dobj_url_list))]

            #Call function to return plot for the selected station:
            update_exploring_multiple_tracers_binary(selection_list)
            
            #If the "citation" checkbox is checked:
            if(Citation):
                
                #Get a list with citation info for every ICOS Level-2 data object:
                cit_ls_L2 = [get_icos_citation(dobj[0]).cit.iloc[0] for dobj in selection_dobj_url_list]
                    
                #Print citation title:
                print('\n\n\033[1m' + 'Data Citation:' +  '\033[0m')
                    
                #Loop through all citations:
                for cit in cit_ls_L2:
                    
                    #Print data object citation:
                    printmd("<sub>"+cit+"</sub>")
        
        #If no L2-data is available for the selected tracer and station:
        else:
            print("\033[0;31;1m "+'No Level-2 data available for the selected tracer and/or station/s.'+"\033[0;31;0m\n\n")



    #Create function that contains a box of widgets:
    interact_c = interact_manual(update_plot_func,
                                 Tracer=tracer,
                                 Station=station,
                                 Citation=Checkbox(value=True, description='Citation', disabled=False))

    #Set the font of the widgets included in interact_manual:
    interact_c.widget.children[0].layout.width = '420px'
    interact_c.widget.children[0].layout.height = '60px'
    interact_c.widget.children[1].layout.width = '420px'
    interact_c.widget.children[2].layout.width = '420px'
    interact_c.widget.children[3].description = 'Update Plot'
    interact_c.widget.children[3].button_style = 'danger'
    interact_c.widget.children[3].style.button_color = '#3973ac'
    #interact_c.widget.children[3].layout.width = '300px'
    interact_c.widget.children[3].layout.margin = '10px 10px 20px 180px' # top/right/bottom/left
    


<a id='exploring_single_station_multiple_tracers_plot_atc_l2'></a>
#### 2.2.2. Exploring (Single Station - Multiple Tracers) -  ICOS Level 2 Atmospheric Data ---> Plot

In [ ]:
#Call widget-function:
create_widgets_exploring_multiple_tracers()

<div style="text-align: right"> 
    <a href="#exploring_single_station_multiple_tracers_atc_l2">[Back to exploring - single station multiple tracers]</a>
    &ensp;&ensp;
    <a href="#introduction">Back to top</a>
</div>
<br>
<br>

<a id='exploring_single_tracer_multiple_stations_atc_l2'></a>

### 2.3. Exploring (Single Tracer - Multiple Stations) -  ICOS Level 2 Atmospheric Data

This part is dedicated on exploring tracer-observations for one or more stations at a time. The user can select to view results for different tracers and different stations. The selection is handled by a set of dropdown-list widgets. It is possible to select and view results for one tracer for more than one stations. Multiple values can be selected with shift and/or ctrl (or command) pressed and mouse clicks. The result of the selection becomes visible once the "Update plot"-button is clicked.

An interactive toolbar is available, on the right side of theplot, with tools such as pan, ZoomBox (zoom-in by rectangle), WheelZoom (zoom-in by scrolling), undo (undo the last change in the plot's state), redo (redo the last change in the plot's state), reset (reset plot to initial state), save (save plot as png) and hover (display tooltips on hover-events). The tools in the toolbar can be activated or deactivated by on-click-events. An active tool is highligted with a blue line on the left side of its corresponding icon. 


<br>
<div style="text-align: right"> 
    <a href="#exploring_atc_l2">[Back to exploring]</a>
</div>

#### 2.3.1. Exploring (Single Tracer - Multiple Stations) -  ICOS Level 2 Atmospheric Data ---> Python Code

<div style="text-align: right"> 
    <a href="#exploring_single_tracer_multiple_stations_plot_atc_l2">[Go to plot]</a>
</div>

In [ ]:
def plot_icos_single_tracer_multiple_stations_binary(df_list):
    
    """
    Project:         'ICOS Carbon Portal'
    Created:          Tue May 07 10:30:00 2018
    Last Changed:     Tue May 07 10:30:00 2019
    Version:          1.0.0
    Author(s):        Karolina
    
    Description:      Function that takes as input a list of sublists with dataframes, where every sublist
                      contains a metadata and a data dataframe for a tracer measured at a specific station. 
                      The function creates an interactive Bokeh plot with the contents of the dataframes and
                      returns a Bokeh Figure (plot).
                      
                      
    Input parameters: List of sublists with ICOS Level-2 Atmospheric data and metadata dataframes
                     (var_name: 'df_list', var_type: List)

    Output:           Bokeh Plot
    
    """
    
    #Import modules to create figure:
    import pandas as pd
    from bokeh.plotting import figure
    from bokeh.models import ColumnDataSource, HoverTool, Label, Legend
    from datetime import datetime

    #Dictionary for subscript transformations of numbers:
    SUB = str.maketrans("0123456789", "₀₁₂₃₄₅₆₇₈₉")
    SUP = str.maketrans("0123456789", "⁰¹²³⁴⁵⁶⁷⁸⁹")

    

    #Create a figure object:
    p = figure(plot_width=900,
               plot_height=500,
               x_axis_label='Time (UTC)', 
               y_axis_label=df_list[0][1]['tracer_info'].replace(' mixing ratio (dry mole fraction)', '').translate(SUB)+
               ' (' + df_list[0][1]['tracer_unit'].translate(SUP) + ')',
               x_axis_type='datetime',
               title = df_list[0][1]['tracer_info'].translate(SUB),
               tools='pan,box_zoom,wheel_zoom,reset,save')

    #Get colormap:
    colormap = get_colormap(len(df_list))
    
    #Create an empty list that will store the legend info:
    legend_it = []

    #Create a glyph with a different colour for every tracer:
    for item, color in zip(df_list, colormap):

        #Extract time and tracer values for every tracer category:
        x = pd.to_datetime(item[2]['TIMESTAMP'], unit='ms') 
        y = item[2][item[1]['tracer_info'].replace(' mixing ratio (dry mole fraction)','').lower()] 
        

        #Create a circle and line glyph for the values of every emission category:
        r0 = p.circle(x, y, radius=.12, color=color)
        r1 = p.line(x, y, line_width=1, color=color, name=item[0]['station_code']+' ('+
                    item[0]['station_sampling_height']+')')

        #Add the name and glyph info (i.e. colour and marker type) to the legend:
        legend_it.append((item[0]['station_code']+' ('+item[0]['station_sampling_height']+')', [r0,r1]))



    #Add tooltip on hover:
    p.add_tools(HoverTool(tooltips=[
        ('Station','$name'),
        ('Time (UTC)','@x{%Y-%m-%d %H:%M:%S}'),
        (item[1]['tracer_info'].replace(' mixing ratio (dry mole fraction)','').translate(SUB),'@y{0.f}'),
        ],
        formatters={
            '@x'      : 'datetime', # use 'datetime' formatter for 'date' field
            },
        # display a tooltip whenever the cursor is vertically in line with a glyph
        mode='vline'
        ))    

    #Create legend:
    legend = Legend(items=legend_it, location= 'bottom_center')
    legend.orientation = 'horizontal'
    legend.click_policy='hide'
    legend.spacing = 10 #sets the distance between legend entries

    #Set title attributes:
    p.title.align = 'center'
    p.title.text_font_size = '13pt'
    p.title.offset = 15

    #Set axis label font style:
    p.xaxis.axis_label_text_font_style = 'normal'
    p.yaxis.axis_label_text_font_style = 'normal'
    p.xaxis.axis_label_standoff = 15 #Sets the distance of the label from the x-axis in screen units
    p.yaxis.axis_label_standoff = 15 #Sets the distance of the label from the y-axis in screen units

    #Set the copyright label position:
    label_opts = dict(x=0, y=10,
                      x_units='screen', y_units='screen')

    #Create a label object and format it:
    caption1 = Label(text="© ICOS ERIC", **label_opts)
    caption1.text_font_size = '8pt'
    
    #Deactivate hover-tool, which is by default active:
    p.toolbar.active_inspect = None

    #Add label to plot:
    p.add_layout(caption1, 'below')
    
    #Add legend to figure:
    p.add_layout(legend, 'below')

    #return plot:
    return p
    


In [ ]:
def update_exploring_multiple_stations_binary(station_dobj_ls):
    
    """
    Project:         'ICOS Carbon Portal'
    Created:          Tue May 07 10:30:00 2018
    Last Changed:     Tue May 07 10:30:00 2019
    Version:          1.0.0
    Author(s):        Karolina
    
    Description:      Function that takes as input a list of sublists, where every sublist contains
                      one or more lists with the ICOS Atmospheric Level-2 data-object-ID, the station
                      code, the sampling height and the selected tracer for a selected station and
                      sampling height.
                      More than one lists per selected station, sampling height and tracer can occur
                      for ICOS Level-2 data in cases where the measuring equipment has changed (i.e.
                      installation of new instrument with new instrument ID). When a new instrument
                      is used, the measurments from this instrument will be put in a different file,
                      which in turns corresponds to a new data object ID.
                      The function reads in the data from the data file corresponding to every data
                      object ID and stores it in metadata- and data-dataframes. One set of metadata-
                      and data-dataframes are produced for every file. The function checks if there
                      are more than one files related to the same station with the same sampling
                      height and tracer, and if this is the case, it concatenates the data-dataframes.
                      All sets of metadata- and data-dataframes are then stored in sublists of a list
                      and sent as input parameters to a plot function, that returns a Bokeh Figure (plot).
                      The Bokeh Figure is then returned as output.
                      
                      
    Input parameters: List of sublists with list(s) including info about an ICOS Atmospheric Level-2
                      data-object-ID, the station code, the sampling height and the selected tracer
                      (var_name: 'station_dobj_ls', var_type: List)

    Output:           Bokeh Plot
    
    """
    
    #import modules:
    from bokeh.layouts import column
    from icoscp.cpb.dobj import Dobj
    
    #Create list to store the metadata & data dataframes for all combinations of tracers and stations:
    station_df_ls = []
    
    
    #Loop through every data object ID in the list:
    for station_ls in station_dobj_ls:
        
        #Create dictionary to store tracer info:
        tracer_info_dict = {}
        
        #Create dict to store the station info:
        station_info_dict = {}
        
        #Create a file object from the current data object id:
        file = Dobj(station_ls[0][0]) 
        
        #Get pandas dataframe with all ICOS stations:
        icos_stations_df = get_coords_icos_stations_atc()
        
        
        #Get the tracer description:
        tracer_info_dict['tracer_info'] = file.info[1].valueType.loc[file.info[1].colName==station_ls[0][1]].values[0]

        #Get tracer unit:
        tracer_info_dict['tracer_unit'] = file.info[1].unit.loc[file.info[1].colName==station_ls[0][1]].values[0]
            
        #Get station info:
        station_info_dict['station_name'] = icos_stations_df.stationName.loc[icos_stations_df.stationId==station_ls[0][2]].values[0]
        station_info_dict['station_code'] = station_ls[0][2]
        station_info_dict['station_country_code'] = icos_stations_df.Country.loc[icos_stations_df.stationId==station_ls[0][2]].values[0]
        station_info_dict['station_country'] = get_country_fullname_from_iso3166_2char(station_info_dict['station_country_code'])
        station_info_dict['station_lat'] = icos_stations_df.lat.loc[icos_stations_df.stationId==station_ls[0][2]].values[0]
        station_info_dict['station_lon'] = icos_stations_df.lon.loc[icos_stations_df.stationId==station_ls[0][2]].values[0]
        station_info_dict['station_sampling_height'] =  station_ls[0][3]  
        
        #Create list to store the data dataframes of all data object IDs for the same station,
        #tracer and sampling height:
        df_ls = []

        #station_info[0] --- > dataobjid (e.g. 'MdYIndlCMyEp2BoGwUL_0Jqq')
        #station_info[1] --- > Tracer (e.g. 'co2')
        #station_info[2] --- > Station Code (e.g. 'HPB') 
        #station_info[3] --- > Sampling Height (e.g. '50.0')
        
        #Download data from all files that correspond to the same station,
        #at the same sampling height and include data for the same tracer
        #in metadata- and data- dataframes that are stored as sublists in a list:
        df_ls = [Dobj(station_info[0]).get() for station_info in station_ls]

        #Concatenate the data-dataframes that include tracer data for
        #the same station at the same sampling height, to one data-dataframe:
        data_df = pd.concat([df for df in df_ls])
        data_df.sort_index(inplace=True)
       

        #Append metadata-dataframe and concatenated data-dataframe to list:
        station_df_ls.append([station_info_dict, tracer_info_dict, data_df])

        
    #Get plot:
    p = plot_icos_single_tracer_multiple_stations_binary(station_df_ls) 
    
    #Output should be in the notebook
    output_notebook()
    
    #Show plot
    show(p, notebook_handle=True)
   

In [ ]:
def create_widgets_exploring_multiple_stations():
    """

    """

    #Create lookup dataframe:
    df_lookup = create_lookup_df_atc_L2()
    
    #Create a list including all tracers (e.g. CO2, CO, CH4)
    tracers = df_lookup.variable.unique().tolist()
    
    #reverse list order:
    tracers.reverse()
    
    stations = create_station_labels(df_lookup)

    #Create widgets:
    tracer = Dropdown(options = tracers)
    station = SelectMultiple(options = stations, disabled=False)

    
    
    #Function that calls functions to update the plot/s and/or map,
    #based on the selected tracer, station and color:
    def update_plot_func(Tracer, Station, Citation):

        #Get tracer (e.g. 'co2'):
        tracer_low_case = Tracer.replace(' mixing ratio (dry mole fraction)', '').lower()
        
        #Get a list of sublists, where every sublist contains the following:
        #1. ICOS Level-2 data object URL
        #2. Tracer/Gas (e.g. 'co2')
        #3. ICOS Station ID (3-character code)
        #4. ICOS Station Sampling Height
        #that refer to the selected station(s) and tracer:
        selection_dobj_url_list = [[df_lookup.dobj.loc[(df_lookup.stationId==station[0]) &
                                                       (df_lookup.height==station[1]) &
                                                       (df_lookup.variable==Tracer)].values,
                                    tracer_low_case,
                                    station[0],
                                    station[1]]
                                   for station in Station
                                   if len(df_lookup.dobj.loc[(df_lookup.stationId==station[0]) &
                                                              (df_lookup.height==station[1]) &
                                                              (df_lookup.variable==Tracer)].values)>0]
        
        
        #If Level-2 data are available for the selected tracer and station(s):
        if(len(selection_dobj_url_list)>0):

            #Get a list of lists, where every sublist contains a data-object-ID, the selected tracer,
            #the station code and the station sampling height.
            #E.g. ['U4VYazHdmZwzr7DxUowMtUu-', 'co2', 'GAT', '30.0']:
            selection_list = [[selection_dobj_url_list[i][0][j].replace('https://meta.icos-cp.eu/objects/', ''),
                               selection_dobj_url_list[i][1],
                               selection_dobj_url_list[i][2],
                               selection_dobj_url_list[i][3]]
                              for i in range(len(selection_dobj_url_list))
                              for j in range(len(selection_dobj_url_list[i][0]))]
            
            ####
            #ICOS Atmospheric Level-2 Data for a given station, a given tracer and  
            #a given sampling height can be stored in two different files in cases
            #where the measuring instrument has changed.
            #The following code controls for such occurances and merges the data
            #if necessary.
            ####
            
            #Get a list of tuples (e.g. ('co2', 'HPB', '50.0')) with unique occurances of
            #"station code" - "station sampling height" - "tracer" triplets:
            station_unique_ls = list(set([(item[1], item[2], item[3]) for item in selection_list]))
            
            #Group selection_list items refering to tracer-data from the same station and
            #sampling height to lists:  
            station_dobj_ls = [[item for item in selection_list
                                if((item[1]==station_id[0]) & (item[2]==station_id[1]) & (item[3]==station_id[2]))]
                               for station_id in station_unique_ls]
            
            #Get plot displaying tracer-values for the selected station/s:
            update_exploring_multiple_stations_binary(station_dobj_ls)
            
            #If the "citation" checkbox is checked:
            if(Citation):
                
                #Get a list with citation info for every ICOS Level-2 data object:
                cit_ls_L2 = [get_icos_citation(dobj[0]).cit.iloc[0] for dobj in selection_dobj_url_list]
                    
                #Print citation title:
                print('\n\n\033[1m' + 'Data Citation:' +  '\033[0m')
                    
                #Loop through all citations:
                for cit in cit_ls_L2:
                    
                    #Print data object citation:
                    printmd("<sub>"+cit+"</sub>")
        
        #If no L2-data is available for the selected tracer and station:
        else:
            print("\033[0;31;1m "+'No Level-2 data available for the selected tracer and/or station/s.'+"\033[0;31;0m\n\n")
                


    
    
    #Create function that contains a box of widgets:
    interact_c = interact_manual(update_plot_func,
                                 Tracer=tracer,
                                 Station=station,
                                 Citation=Checkbox(value=True, description='Citation', disabled=False))

    #Set the font of the widgets included in interact_manual:
    interact_c.widget.children[0].layout.width = '420px'
    interact_c.widget.children[0].layout.margin = '40px 2px 2px 2px'
    interact_c.widget.children[1].layout.width = '420px'
    interact_c.widget.children[1].layout.height = '120px'
    interact_c.widget.children[2].layout.width = '420px'
    interact_c.widget.children[3].description = 'Update Plot'
    interact_c.widget.children[3].button_style = 'danger'
    interact_c.widget.children[3].style.button_color = '#3973ac'
    interact_c.widget.children[3].layout.margin = '10px 10px 40px 180px' # top/right/bottom/left

<a id='exploring_single_tracer_multiple_stations_plot_atc_l2'></a>
#### 2.3.2. Exploring (Single Tracer - Multiple Stations) -  ICOS Level 2 Atmospheric Data ---> Plot

In [ ]:
#Call widget-function:
create_widgets_exploring_multiple_stations()

#Bokeh does not yet support multi-column/multi-row legends
#Therefore, for now, it is advised that the selection in the multi-select widget is limited to max 6 stations.

<div style="text-align: right"> 
    <a href="#exploring_single_tracer_multiple_stations_atc_l2">[Back to exploring - single tracer multiple stations]</a>
    &ensp;&ensp;
    <a href="#introduction">Back to top</a>
</div>
<br>
<br>
<br>

<a id='focusing_atc_l2'></a>

## 3. Focusing on Data -  ICOS Level 2 Atmospheric Data
In this part it is possible to focus on data from one station in higher detail, check for irregularities or specific parts of interest. Two plots with data from the same station are displayed. The user can select an area of interest in the first graph using the "box-zoom", "wheel-zoom" or "pan" tool. This will trigger two events;  the content of the first plot will change to only show the selected area and a transparent rectangle will appear in the second plot, hihglighting the selected area of the first plot. It is possible to turn layers on or off in both plots using the interactive legend. The plots can show results for different stations. The choice of station is handled by the single-selection tool (dropdown list). Both plots can be saved as separate png images, by clicking on the "save" tool in their respective interactive toolbars. Additionally, the first plot includes tools for reseting the plot to its initial state and showing tooltips on hover-events.

<br>
<div style="text-align: right"> 
    <a href="#introduction">Back to top</a>
</div>

### 3.1. Focusing - ICOS Level 2 Atmospheric Data ---> Python Code
This section contains the Python code used to present the focusing-plots. Note that the following code includes calls to functions from the code-cells above. 

<br>
<div style="text-align: right"> 
    <a href="#focusing_plot_atc_l2">[Go to plot]</a>
</div>

In [ ]:
def plot_icos_focus_binary(df_data, station_info_dict, tracer_info_dict, tracer, color):
    
    """
    Project:         'ICOS Carbon Portal'
    Created:          Tue May 07 10:30:00 2018
    Last Changed:     Tue May 07 10:30:00 2019
    Version:          1.0.0
    Author(s):        Karolina
    
    Description:      Function that takes as input a metadata dataframe, a data dataframe, a string stating
                      the tracer and a string with the color for the plot of datafor a specific ICOS station. 
                      The function creates two interactive Bokeh plots with the contents of the dataframes and
                      returns two Bokeh Figures (plots).
                      
                      
    Input parameters: 1. ICOS Level-2 tracer Atmospheric Data Dataframe
                         (var_name: 'df_data', var_type: Pandas DataFrame)
                      2. Dictionary with ICOS station information 
                         (var_name: 'station_info_dict', var_type: Dictionary)
                      3. Dictionary with tracer information 
                         (var_name: 'tracer_info_dict', var_type: Dictionary)
                      4. Tracer/gas - e.g. 'co2'
                         (var_name: "tracer", var_type: String)
                      5. Plot Color
                         (var_name: "color", var_type: String)

    Output:           Bokeh Plot
    
    """

    #Import modules to create figure:
    #from bokeh.palettes import Category10
    import pandas as pd
    from bokeh.layouts import row, column
    from bokeh.models import ColumnDataSource, HoverTool, Label, Legend, CustomJS, Rect
    from bokeh.plotting import figure, show
    from datetime import datetime
    from bokeh.io import push_notebook, output_notebook


    #Dictionary for subscript transformations of numbers:
    SUB = str.maketrans("0123456789", "₀₁₂₃₄₅₆₇₈₉")
    SUP = str.maketrans("0123456789", "⁰¹²³⁴⁵⁶⁷⁸⁹")

    #Define Datasets:
    x = pd.to_datetime(df_data['TIMESTAMP'], unit='ms')
    y = df_data[tracer_info_dict['tracer_info'].replace(' mixing ratio (dry mole fraction)','').lower()].values 

    #Create a ColumnDataSource object:
    source = ColumnDataSource({'x': [], 'y': [], 'width': [], 'height': []})

    #Javascript code defining the new x-range, y-range of the plot, based on the area selected by the user:
    jscode="""
        var data = source.data;
        var start = cb_obj.start;
        var end = cb_obj.end;
        data['%s'] = [start + (end - start) / 2];
        data['%s'] = [end - start];
        source.change.emit();
    """

    #Create a figure object:
    p1 = figure(plot_width=900,
                plot_height=400,
                x_axis_label='Time (UTC)',
                y_axis_label=tracer_info_dict['tracer_info'].replace(' mixing ratio (dry mole fraction)','').translate(SUB) + ' (' +
                tracer_info_dict['tracer_unit'].translate(SUP) + ')',
                x_axis_type='datetime',
                title = tracer_info_dict['tracer_info'].translate(SUB)+'   '+
                station_info_dict['station_name']+', '+
                station_info_dict['station_country'][0]+', '+
                station_info_dict['station_sampling_height']+' m. a. g. .l.',
                tools='pan,box_zoom,wheel_zoom,undo,redo,reset,save')

    #Create glyphs:
    p1.circle(x,y, radius=.02, color=color)# ,legend=df_metadata.loc['STATION CODE'].values[0])
    p1.line(x,y, line_width=1, color=color)# ,legend=df_metadata.loc['STATION CODE'].values[0])

    #Add tooltip on hover:
    p1.add_tools(HoverTool(tooltips=[
        ('Station Code',station_info_dict['station_code']),
        ('Latitude',station_info_dict['station_lat']),
        ('Longitude',station_info_dict['station_lon']),
        ('Time (UTC)','@x{%Y-%m-%d %H:%M:%S}'),
        (tracer_info_dict['tracer_info'].replace(' mixing ratio (dry mole fraction)','').translate(SUB),'@y{0.f}'),
        ],
        formatters={
            '@x'      : 'datetime', # use 'datetime' formatter for 'date' field
            },
        # display a tooltip whenever the cursor is vertically in line with a glyph
        mode='vline'
        ))    

    #Set title attributes:
    p1.title.align = 'center'
    p1.title.text_font_size = '13pt'
    p1.title.offset = 15

    #Set label font style:
    p1.xaxis.axis_label_text_font_style = 'normal'
    p1.yaxis.axis_label_text_font_style = 'normal'
    p1.xaxis.axis_label_standoff = 15 #Sets the distance of the label from the x-axis in screen units
    p1.yaxis.axis_label_standoff = 15 #Sets the distance of the label from the y-axis in screen units

    #Set the copyright label position:
    label_opts = dict(x=0, y=10,
                      x_units='screen', y_units='screen')

    #Create a label object and format it:
    caption1 = Label(text="© ICOS ERIC", **label_opts)
    caption1.text_font_size = '8pt'
    
    #Change the plot's x-range, y-range based on the selected area (javascript callback):   
    xcb = CustomJS(args=dict(source=source), code=jscode % ('x', 'width'))
    ycb = CustomJS(args=dict(source=source), code=jscode % ('y', 'height'))
    p1.x_range.js_on_change('start', xcb)
    p1.x_range.js_on_change('end', xcb)
    p1.y_range.js_on_change('start', ycb)
    p1.y_range.js_on_change('end', ycb)
    
    #Deactivate hover-tool, which is by default active:
    p1.toolbar.active_inspect = None

    #Add label to plot:
    p1.add_layout(caption1, 'below')

    
    ############################################### CODE FOR 2nd PLOT ###################################################


    #Create a figure object:
    p2 = figure(plot_width=900,
               plot_height=400,
               x_axis_label='Time (UTC)', 
               y_axis_label=tracer_info_dict['tracer_info'].replace(' mixing ratio (dry mole fraction)','').translate(SUB) + 
               ' (' +tracer_info_dict['tracer_unit'].translate(SUP) + ')',
               x_axis_type='datetime',
               title = tracer_info_dict['tracer_info'].translate(SUB)+'   '+
               station_info_dict['station_name']+', '+
               station_info_dict['station_country']+', '+
               station_info_dict['station_sampling_height']+' m. a. g. l.',
               tools='save')

    #Create glyphs:
    p2.circle(x, y, radius=.02, color=color)# ,legend=df_metadata.loc['STATION CODE'].values[0])
    p2.line(x, y, line_width=1, color=color)# ,legend=df_metadata.loc['STATION CODE'].values[0])

    #Add tooltip on hover:
    p2.add_tools(HoverTool(tooltips=[
        ('Station Code',station_info_dict['station_code']),
        ('Latitude',station_info_dict['station_lat']),
        ('Longitude',station_info_dict['station_lon']),
        ('Time (UTC)','@x{%Y-%m-%d %H:%M:%S}'),
        (tracer_info_dict['tracer_info'].replace(' mixing ratio (dry mole fraction)','').translate(SUB),'@y{0.f}')
        ],
        formatters={
            '@x'      : 'datetime', # use 'datetime' formatter for 'date' field
            },
        # display a tooltip whenever the cursor is vertically in line with a glyph
        mode='vline'
        ))    



    #Set title attributes:
    p2.title.align = 'center'
    p2.title.text_font_size = '13pt'
    p2.title.offset = 15

    #Set label font style:
    p2.xaxis.axis_label_text_font_style = 'normal'
    p2.yaxis.axis_label_text_font_style = 'normal'
    p2.xaxis.axis_label_standoff = 15 #Sets the distance of the label from the x-axis in screen units
    p2.yaxis.axis_label_standoff = 15 #Sets the distance of the label from the y-axis in screen units


    #Set the copyright label position:
    label_opts = dict(x=0, y=10,
                      x_units='screen', y_units='screen')

    #Create a label object and format it:
    caption1 = Label(text="© ICOS ERIC", **label_opts)
    caption1.text_font_size = '8pt'

    #Add label to plot:
    p2.add_layout(caption1, 'below')
    
    #Add a rectangle with 
    rect = Rect(x='x', y='y', width='width', height='height', fill_alpha=0.1, line_color='black', fill_color='black')
    p2.add_glyph(source, rect)
    
    #Deactivate hover-tool, which is by default active:
    p2.toolbar.active_inspect = None

    #Return plots:
    return p1, p2

In [ ]:
def update_focus_plot_binary(data_obj_id_ls, station, tracer, color):
    
    """
    Project:         'ICOS Carbon Portal'
    Created:          Tue May 07 10:30:00 2018
    Last Changed:     Tue May 07 10:30:00 2019
    Version:          1.0.0
    Author(s):        Karolina
    
    Description:      Function that takes as input a list of ICOS data object IDs, the station name,
                      the tracer type and the plot color. The function reads the corresponding
                      ICOS Level-2 Atmospheric Tracer data files for every data object ID into sepparate
                      pandas dataframes. Every data file produces two sepparate pandas dataframes;
                      metadata dataframe and data dataframe. These dataframes are then set as input
                      parameters to a plot function, that returns a Bokeh Figure (plot).
                      The Bokeh Figure is then returned as output.
                      
                      
    Input parameters: 1. List with sublists of data object IDs
                         (var_name: 'data_obj_id_ls', var_type: List)
                      2. ICOS Station 3-charcter Code
                         (var_name: 'station', var_type: String)
                      3. Tracer/gas, e.g. 'co2'
                         (var_name: 'tracer', var_type: String)
                      4. Plot Color
                         (var_name: 'color', var_type: String)
                      

    Output:           Bokeh Plot
    
    """
      
    #import modules:
    from icoscp.cpb.dobj import Dobj
    from bokeh.layouts import column
    from bokeh.io import push_notebook, output_notebook
    from bokeh.plotting import show
    
    #Create dictionary to store tracer info:
    tracer_info_dict = {}
    
    #Create dict to store the station info:
    station_info_dict = {}
    
    #Create a file object from the 1st object in the data object id list:
    file = Dobj(data_obj_id_ls[0])  
    
    #Get the tracer description:
    tracer_info_dict['tracer_info'] = file.info[1].valueType.loc[file.info[1].colName==tracer].values[0]
    
    #Get tracer unit:
    tracer_info_dict['tracer_unit'] = file.info[1].unit.loc[file.info[1].colName==tracer].values[0]
    
    
    #Get pandas dataframe with all ICOS stations:
    icos_stations_df = get_coords_icos_stations_atc()
    
    #Get station info:
    station_info_dict['station_name'] = icos_stations_df.stationName.loc[icos_stations_df.stationId==station[0]].values[0]
    station_info_dict['station_code'] = station[0]
    station_info_dict['station_sampling_height'] = station[1]
    station_info_dict['station_country_code'] = icos_stations_df.Country.loc[icos_stations_df.stationId==station[0]].values[0]
    station_info_dict['station_country'] = get_country_fullname_from_iso3166_2char(station_info_dict['station_country_code'])
    station_info_dict['station_lat'] = icos_stations_df.lat.loc[icos_stations_df.stationId==station[0]].values[0]
    station_info_dict['station_lon'] = icos_stations_df.lon.loc[icos_stations_df.stationId==station[0]].values[0]
    
    
    #Create list to store the data dataframes of all data object IDs:
    data_df_ls = []
    
    #Loop through every data object ID in the list:
    for dobjid in data_obj_id_ls: 
        
        #Get data dataframe:
        obs_data_df = Dobj(dobjid).get()
        
        #Add data dataframe corresponding to the current data object ID to the list:
        data_df_ls.append(obs_data_df)
        
    #Concatenate data dataframes to one dataframe:
    data_df = pd.concat(data_df_ls)
    
    #Sort the dataframe index in ascending order:
    data_df.sort_index(inplace=True)
    
    
    ### PLOT ###
    #Plot station:
    p1, p2 = plot_icos_focus_binary(data_df, station_info_dict, tracer_info_dict, tracer, color)
    
    #Set the output to be organized columnwise (i.e. output plots one under the other):
    layout = column(p1,p2)
    
    #Set the notebook as the prefered output channel:
    output_notebook()
    
    #Show plot:
    show(layout)

    #Update plot:
    push_notebook()

In [ ]:
def create_widgets_focusing():
    
    """
    Project:         'ICOS Carbon Portal'
    Created:          Tue May 07 10:30:00 2018
    Last Changed:     Tue May 07 10:30:00 2019
    Version:          1.0.0
    Author(s):        Karolina
    
    Description:      Function that creates a set of widgets; a station dropdown list,
                      a tracer dropdown list, a color-picker and a button. The function
                      populates the dropdown lists with values and outputs the result.
                                      
    Input parameters: No Input Parameter(s)

    Output:           Python Widgets
    
    """
    
    #Dictionary for subscript transformations of numbers:
    SUB = str.maketrans("0123456789", "₀₁₂₃₄₅₆₇₈₉")

    #Create lookup dataframe:
    df_lookup = create_lookup_df_atc_L2()
    
    #Create a list including all tracers (e.g. CO2, CO, CH4)
    tracers = df_lookup.variable.unique().tolist()
    
    #reverse list order:
    tracers.reverse()

    #Create widgets:
    tracer = Dropdown(options = tracers)
    station =  Dropdown(options = create_station_labels(df_lookup))

    #Function that calls functions to update the plot/s and/or map,
    #based on the selected tracer, station and color:
    def update_plot_func(Tracer, Station, color, Citation):

        #Get tracer short:
        tracer = Tracer.replace(' mixing ratio (dry mole fraction)', '').lower()

        #Get a list of data obect URLs that refer to the selected station and tracer:
        data_obj_url_ls = df_lookup.dobj.loc[(df_lookup.stationId==Station[0]) &
                                             (df_lookup.height==Station[1]) &
                                             (df_lookup.variable==Tracer)].values

        #If L2-data is available for the selected tracer and station:
        if(data_obj_url_ls.size>0):
            
            #Get a list of data object IDs (L2-data):
            data_obj_id_ls = [data_obj_url_ls[i].replace('https://meta.icos-cp.eu/objects/', '') 
                                 for i in range(data_obj_url_ls.size)]
            
            #Call function to return plot and map for the selected station:
            update_focus_plot_binary(data_obj_id_ls, Station, tracer, color)
            
            #If the "citation" checkbox is checked:
            if(Citation):
                
                #Get a list with citation info for every ICOS Level-2 data object:
                cit_ls_L2 = [get_icos_citation(dobj).cit.iloc[0] for dobj in data_obj_url_ls]
                    
                #Print citation title:
                print('\n\n\033[1m' + 'Data Citation:' +  '\033[0m')
                    
                #Loop through all citations:
                for cit in cit_ls_L2:
                    
                    #Print data object citation:
                    printmd("<sub>"+cit+"</sub>")
        
        #If no L2-data are available for the selected tracer and station:
        else:
            print("\033[0;31;1m "+'No '+tracer.upper().translate(SUB)+' Level-2 data available for the selected station'+"\033[0;31;0m\n\n")
                

           
    #Create function that contains a box of widgets:
    interact_c = interact_manual(update_plot_func,
                                 Tracer=tracer,
                                 Station=station,
                                 color=ColorPicker(concise=False,
                                                   description='Pick a color',
                                                   value='#3973ac',
                                                   disabled=False),
                                 Citation=Checkbox(value=True, description='Citation', disabled=False))
    
    #Set the font of the widgets included in interact_manual:
    interact_c.widget.children[0].layout.width = '420px'
    interact_c.widget.children[0].layout.margin = '40px 2px 2px 2px'
    interact_c.widget.children[1].layout.width = '420px'
    interact_c.widget.children[2].layout.width = '420px'
    interact_c.widget.children[3].layout.width = '420px'
    interact_c.widget.children[4].description = 'Update Plot'
    interact_c.widget.children[4].button_style = 'danger'
    interact_c.widget.children[4].style.button_color = '#3973ac'
    interact_c.widget.children[4].layout.margin = '10px 10px 40px 180px' # top/right/bottom/left

<a id='focusing_plot_atc_l2'></a>
### 3.2. Focusing -  ICOS Level 2 Atmospheric Data ---> Plot

In [ ]:
#Call function to display widgets for the corresponding plot type:
create_widgets_focusing()

<br>
<div style="text-align: right"> 
    <a href="#focusing_atc_l2">[Back to focusing]</a>
    &ensp;&ensp;
    <a href="#introduction">Back to top</a>
</div>
<br>
<br>
<br>

<a id='statistics_atc_l2'></a>

## 4. Statistics - ICOS Level 2 Atmospheric Data
This part shows statistical information over data for one station at a time. The first section is dedicated to showing some [basic statistics](#basic_statistics_atc_l2) (e.g. min, max, mean and standard deviation) per station as tabular output. In the second section, it is possible to compute the [correlation](#statistics_correlation_atc_l2) between different tracers measured at the same or at different stations. The third section includes a tool that implements a ["smoothing-function"](#statistics_smoothing_atc_l2) over the data. The "smoothing" is applied over instances of the time variable. 

<div style="text-align: right"> 
    <a href="#introduction">Back to top</a>
</div>

<a id='basic_statistics_atc_l2'></a>

### 4.1. Statistics - Basic Statistics - ICOS Level 2 Atmospheric Data
This section presents tabular output of basic statistics (e.g. min, max, mean, standard deviation) per station. The selection of tracer and station is handled by two selection-widgets.

<div style="text-align: right"> 
    <a href="#statistics_atc_l2">[Back to statistics]</a>
</div>

#### 4.1.1. Basic Statistics -  ICOS Level 2 Atmospheric Data ---> Python Code
This part presents the python code used to produce the basic statistics tabular output.

<br>
<div style="text-align: right"> 
    <a href="#basic_stat_table_atc_l2">[Go to table]</a>
</div>


In [ ]:
def check_input_icos_L2_basic_stat(Station, start_date, end_date):
    
    """
    Project:         'ICOS Carbon Portal'
    Created:          Fri May 17 14:30:00 2019
    Last Changed:     Fri May 17 14:30:00 2019
    Version:          1.0.0
    Author(s):        Karolina
    
    Description:      Function that checks the user input to widgets and returns a boolean for
                      invalid or empty input.
                      
    Input parameters: 1. List of sublists, where every sublist contains the Station ID & Sampling Height
                         (var_name: 'Station', var_type: List)
                      2. Start date of time period
                         (var_name: 'start_date', var_type: DateTime Object)
                      3. End date of time period
                         (var_name: 'end_date', var_type: DateTime Object)

    Output:           Pandas DataFrame
    
    """
    
    #Check if NO station has been selected:
    if(len(Station)<1):

        #Print message:
        print(("\033[0;31;1m Select station! \033[0;31;0m\n\n"))

        #Return boolean:
        return False
        
    #Check if a station has been selected:
    else:

        #Check if a start-date and/or end-date have been selected:
        if((start_date==None)|(end_date==None)):

            #Print message:
            print(("\033[0;31;1m Select a start date and/or end date! \033[0;31;0m\n\n"))

            #Return boolean:
            return False

        #If a start-date and an end-date have been selected:
        else:

            #Compute the difference between end_date and start_date:
            diff = end_date - start_date

            #Check if end-date refers to an earlier date than start-date:
            if(diff.days<0):

                #Print message:
                print('\033[0;31;1m Error...\n The selected start-date corresponds to a later date than the selected end-date.\n Enter new dates!\n\n')

                #Return boolean:
                return False
            
            #If the selected start-date & end-date are valid:
            else:
                
                #Return boolean:
                return True

  

In [ ]:
def calculate_basic_statistics_binary(station_df_ls, tracer):
    
    """
    Project:         'ICOS Carbon Portal'
    Created:          Wed May 15 14:30:00 2019
    Last Changed:     Wed May 15 14:30:00 2019
    Version:          1.0.0
    Author(s):        Karolina
    
    Description:      Function that loops through a list of sublists, where every sublist contains
                      the data-dataframe. dictionary with station info and dictionary with tracer info 
                      of a specific station, and computes the min,
                      max, mean and standard deviation of the tracer-column in the data-dataframe
                      of every station. The statistical values are stored in a separate dataframe.
                      One dataframe is produced for every station. In cases where more than one
                      stations have been selected, all separate dataframes with basic statistics
                      results per station are concatenated to one dataframe that is then returned
                      as output.
                      
    Input parameters: 1. List with sublists of data-dataframes for every selected station
                         (var_name: 'station_df_ls', var_type: List)
                      2. Tracer/gas, e.g. 'co2'
                         (var_name: 'tracer', var_type: String)

    Output:           Pandas DataFrame
    
                      Columns:
                      1. The earliest date in the dataset
                         (var_name: "start_date", var_type: NumPy DateTime64)
                      2. The latest date in the dataset
                         (var_name: "end_date", var_type: NumPy DateTime64)
                      3. The minimum tracer value in the dataset
                         (var_name: 'min', var_type: float)
                      4. The maximum tracer value in the dataset
                         (var_name: 'max', var_type: float)
                      5. The average tracer value in the dataset
                         (var_name: 'mean', var_type: float)
                      6. The standard deviation of the tracer values in the dataset
                         (var_name: 'st_dev', var_type: float)
    
    """
    

    #Dictionary for subscript transformations of numbers:
    SUB = str.maketrans("0123456789", "₀₁₂₃₄₅₆₇₈₉")

    #Create list to store the metadata and data dataframes for every station:
    df_list = []
    
    #Loop through every station's metadata- & data- dataframe: 
    for df in station_df_ls:
        
        #Get time-period and statistics:
        stat_df = pd.DataFrame({'start_date': df[0].index.values.min(),
                                'end_date': df[0].index.values.max(),
                                'min': round(pd.to_numeric(df[0][tracer],errors='coerce').min(),2),
                                'max': round(pd.to_numeric(df[0][tracer],errors='coerce').max(),2),
                                'mean': round(pd.to_numeric(df[0][tracer],errors='coerce').mean(),2),
                                'st dev': round(pd.to_numeric(df[0][tracer],errors='coerce').std(),2)}, 
                               index=[df[1]['station_name']+', '+
                                      df[1]['station_sampling_height']+
                                      ' ('+tracer.upper().translate(SUB)+')'])
        
        #Add dataframe to list:
        df_list.append(stat_df)              
            
        
    #Concatenate dataframes to one dataframe:
    basic_statistics_df = pd.concat(df_list)
    
    #Sort dataframe index:
    basic_statistics_df.sort_index(inplace=True)

    #Return dataframe:
    return basic_statistics_df


In [ ]:
def update_basic_statistics_binary(station_dobj_ls, start_date, end_date):
    
    """
    Project:         'ICOS Carbon Portal'
    Created:          Wed May 15 10:30:00 2018
    Last Changed:     Wed May 15 10:30:00 2019
    Version:          1.0.0
    Author(s):        Karolina
    
    Description:      Function that takes as input a list of sublists, where every sublist contains
                      one or more lists with the ICOS Atmospheric Level-2 data-object-ID, the station
                      code, the sampling height and the selected tracer for a selected station and
                      sampling height. The Function also takes as input the selected start-date and
                      end-date.
                      More than one lists per selected station, sampling height and tracer can occur
                      for ICOS Level-2 data in cases where the measuring equipment has changed (i.e.
                      installation of new instrument with new instrument ID). When a new instrument
                      is used, the measurments from this instrument will be put in a different file,
                      which in turns corresponds to a new data object ID.
                      The function reads in the data from the data file corresponding to every data
                      object ID and stores it in metadata- and data-dataframes. One set of metadata-
                      and data-dataframes are produced for every file. The function checks if there
                      are more than one files related to the same station with the same sampling
                      height and tracer, and if this is the case, it concatenates the data-dataframes.
                      The data dataframe is filtered to only contain data between the selected start-date
                      och end-date.
                      All sets of metadata- and data-dataframes are then stored in sublists of a list
                      and sent as input parameters to a function that will compute the basic statistics
                      for the tracer-column of every data-dataframe. The dataframe including the 
                      basic statistics results for every selected station are the returned as output. 
                      
                      
    Input parameters: 1. List of sublists with list(s) including info about an ICOS Atmospheric Level-2
                         data-object-ID, the station code, the sampling height and the selected tracer
                         (var_name: 'station_dobj_ls', var_type: List)
                      2. Start date - user's input
                         (var_name: 'start_date', var_type: DateTime Object)
                      3. End date - user's input
                         (var_name: 'end_date', var_type: DateTime Object)
                      

    Output:           Pandas DataFrame
    
    """
    
    #Import modules:
    import pandas as pd
    from icoscp.cpb.dobj import Dobj
    
 
    #Create dictionary to store tracer info:
    tracer_info_dict = {}
    
    #Create a file object from the 1st object in the data object id list:
    file = Dobj(station_dobj_ls[0][0][0])  
    
    #Get the tracer description:
    tracer_info_dict['tracer_info'] = file.info[1].valueType.loc[file.info[1].colName==station_dobj_ls[0][0][1]].values[0]
    
    #Get tracer unit:
    tracer_info_dict['tracer_unit'] = file.info[1].unit.loc[file.info[1].colName==station_dobj_ls[0][0][1]].values[0]
    
    #Get pandas dataframe with all ICOS stations:
    icos_stations_df = get_coords_icos_stations_atc()
    
    
    #Create list to store the metadata & data dataframes for all combinations of tracers and stations:
    station_df_ls = []

    
    
    #Loop through every data object ID in the list:
    for station_ls in station_dobj_ls:
        
        #Create a dictionary to store all station info:
        station_info_dict = {}
    
        #Get station info:
        station_info_dict['station_name'] = icos_stations_df.stationName.loc[icos_stations_df.stationId==station_ls[0][2]].values[0]
        station_info_dict['station_code'] = station_ls[0][2]
        station_info_dict['station_sampling_height'] = station_ls[0][3]
        station_info_dict['station_country_code'] = icos_stations_df.Country.loc[icos_stations_df.stationId==station_ls[0][2]].values[0]
        station_info_dict['station_country'] = get_country_fullname_from_iso3166_2char(station_info_dict['station_country_code'])
        station_info_dict['station_lat'] = icos_stations_df.lat.loc[icos_stations_df.stationId==station_ls[0][2]].values[0]
        station_info_dict['station_lon'] = icos_stations_df.lon.loc[icos_stations_df.stationId==station_ls[0][2]].values[0]

    

        #Create list to store the data dataframes of all data object IDs
        #for all stations:
        df_ls = []

        #station_info[0] --- > dataobjid (e.g. 'MdYIndlCMyEp2BoGwUL_0Jqq')
        #station_info[1] --- > Tracer (e.g. 'co2')
        #station_info[2] --- > Station Code (e.g. 'HPB')
        #station_info[3] --- > Sampling Height (e.g. '50.0')
        

        #Download data from all files that correspond to the same station,
        #at the same sampling height and include data for the same tracer
        #in a pandas dataframe that are then stored as sublists in a list:
        df_ls = [Dobj(station_info[0]).get()
                 for station_info in station_ls]

        #Concatenate the data-dataframes that include tracer data for
        #the same station at the same sampling height, to one data-dataframe:
        data_df = pd.concat([df for df in df_ls])
        
        #Add column with datetime object:
        data_df['DateTime'] = pd.to_datetime(data_df['TIMESTAMP'], unit='ms') 

        #Create a copy of the dataframe and set "DateTime" as index:
        data_df_ind = data_df.copy().set_index('DateTime')

        #Sort the dataframe index in ascending order:
        data_df_ind.sort_index(inplace=True)
        
        
        #Filter dataframe to only contain data between the selected dates:
        data_df_filt = data_df_ind.loc[start_date:end_date]

        #Append metadata-dataframe and concatenated data-dataframe to list:
        station_df_ls.append([data_df_filt, station_info_dict, tracer_info_dict])

    #Call function to compute basic statistics for all selected stations
    #and return the result:
    return calculate_basic_statistics_binary(station_df_ls, station_dobj_ls[0][0][1])



In [ ]:
def create_widgets_basic_stat():
    
    """
    Project:         'ICOS Carbon Portal'
    Created:          Tue May 07 10:30:00 2018
    Last Changed:     Tue May 07 10:30:00 2019
    Version:          1.0.0
    Author(s):        Karolina
    
    Description:      Function that creates a set of widgets; a station multiselection dropdown list, a
                      tracer dropdown list and a button. The function populates the dropdown lists with
                      values and outputs the result.
                      
                      
    Input parameters: No Input Parameter(s)

    Output:           Python Widgets
    
    """

    #Create lookup dataframe:
    df_lookup = create_lookup_df_atc_L2()
    
    #Convert data type of date-columns from string to datetime:
    df_lookup.timeStart = [datetime.strptime(df_lookup.timeStart.iloc[i],'%Y-%m-%dT%H:%M:%SZ')
                    for i in range(len(df_lookup))]
    df_lookup.timeEnd = [datetime.strptime(df_lookup.timeEnd.iloc[i],'%Y-%m-%dT%H:%M:%SZ')
                  for i in range(len(df_lookup))]
    
    #Create a list including all tracers (e.g. CO2, CO, CH4)
    tracers = df_lookup.variable.unique().tolist()
    
    #reverse list order:
    tracers.reverse()

    #Create widgets:
    tracer = Dropdown(options = tracers)
    station = SelectMultiple(options = create_station_labels(df_lookup), disabled=False)

    #Function that calls functions to update the plot/s and/or map,
    #based on the selected tracer and station/s:
    def update_plot_func(Tracer, Station, start_date, end_date):
        
        #Check user input:
        if(check_input_icos_L2_basic_stat(Station, start_date, end_date)):
        
            #Get tracer (e.g. 'co2'):
            tracer_low_case = Tracer.replace(' mixing ratio (dry mole fraction)', '').lower()

            #Get a list of sublists, where every sublist contains the following:
            #1. ICOS Level-2 data object URL
            #2. ICOS Station ID (3-character code)
            #3. ICOS Station Sampling Height
            #4. Tracer/Gas (e.g. 'co2')
            #that refer to the selected station(s) and tracer:
            selection_dobj_url_list = [[df_lookup.dobj.loc[(df_lookup.stationId==station[0]) &
                                                           (df_lookup.height==station[1]) &
                                                           (df_lookup.variable==Tracer) &
                                                           (((df_lookup.timeStart<=pd.Timestamp(start_date))&
                                                             (df_lookup.timeEnd>=pd.Timestamp(start_date)))|
                                                            ((df_lookup.timeStart<=pd.Timestamp(end_date))&
                                                             (df_lookup.timeEnd>=pd.Timestamp(end_date)))|
                                                            ((df_lookup.timeStart>=pd.Timestamp(start_date))&
                                                             (df_lookup.timeEnd<=pd.Timestamp(end_date))))].values,
                                        station[0],
                                        station[1],
                                        tracer_low_case]
                                       for station in Station]

            #Get a list of items, where every item represents the size of the url-array for every selected station:
            check_url_ls_size = [selection_dobj_url_list[m][0].size for m in range(len(selection_dobj_url_list))]

            #If Level-2 data are available for the selected tracer, station(s) and time period:
            if(sum(check_url_ls_size)>0):

                #Get a list of lists, where every sublist contains a data-object-ID, the station code, 
                #the sampling height & the selected tracer.
                #E.g. ['U4VYazHdmZwzr7DxUowMtUu-', 'co2', 'GAT', '30.0']:
                selection_list = [[selection_dobj_url_list[i][0][j].replace('https://meta.icos-cp.eu/objects/', ''),
                                   selection_dobj_url_list[i][3],
                                   selection_dobj_url_list[i][1],
                                   selection_dobj_url_list[i][2]]
                                  for i in range(len(selection_dobj_url_list))
                                  for j in range(len(selection_dobj_url_list[i][0]))]

                #Get a list of tuples (e.g. ('HPB', '50.0', 'co2')) with unique occurances of
                #"station code" - "station sampling height" - "tracer" triplets:
                station_unique_ls = list(set([(item[2], item[3], item[1]) for item in selection_list]))

                #Group selection_list items refering to tracer-data from the same station and sampling height to lists:  
                station_dobj_ls = [[item for item in selection_list
                                    if((item[1]==station_id[2]) & (item[2]==station_id[0]) & (item[3]==station_id[1]))]
                                   for station_id in station_unique_ls]
                
                #Call function to calculate and return basic statistics dataframe for the selected stations:
                return update_basic_statistics_binary(station_dobj_ls, start_date, end_date)
                

            #If no data are available for the selected tracer, station(s) and time period:
            else:
                print("\033[0;31;1m No ICOS Atmospheric Level-2 data available.\033[0;31;0m\n\n")



    #Create function that contains a box of widgets:
    interact_c = interact_manual(update_plot_func,
                                 Tracer=tracer,
                                 Station=station,
                                 start_date=DatePicker(description='Starting Date',disabled=False),
                                 end_date=DatePicker(description='Ending Date',disabled=False))

    #Set the font of the widgets included in interact_manual:
    interact_c.widget.children[0].layout.width = '430px'
    interact_c.widget.children[0].layout.margin = '40px 2px 2px 2px'
    interact_c.widget.children[1].layout.width = '430px'
    interact_c.widget.children[1].layout.height = '120px'
    interact_c.widget.children[2].layout.width = '430px'
    interact_c.widget.children[3].layout.width = '430px'
    interact_c.widget.children[4].description = 'Update Table'
    interact_c.widget.children[4].button_style = 'danger'
    interact_c.widget.children[4].style.button_color = '#3973ac'
    interact_c.widget.children[4].layout.margin = '10px 10px 40px 180px' # top/right/bottom/left

<a id='basic_stat_table_atc_l2'></a>
#### 4.1.2. Basic Statistics -  ICOS Level 2 Atmospheric Data ---> Output Table

In [ ]:
#Call function to display widgets for the corresponding output type:
create_widgets_basic_stat()

<div style="text-align: right"> 
    <a href="#basic_statistics_atc_l2">[Back to statistics-basic statistics]</a>
    &ensp;&ensp;
    <a href="#introduction">Back to top</a>
</div>
<br>
<br>
<br>

<a id='statistics_correlation_atc_l2'></a>

### 4.2. Statistics - Correlation - ICOS Level 2 Atmospheric Data
This section is dedicated on computing the correlation between different tracers:
1. for a given station and a given sampling height (a.g.l.)
2. at different sampling heights (a.g.l.) of the same station
3. at different stations at different sampling heights (a.g.l.)

<div style="text-align: right"> 
    <a href="#statistics_atc_l2">[Back to statistics]</a>
</div>

#### 4.2.1. Correlation - ICOS Level 2 Atmospheric Data --- > Python Code
This part presents the python code used to produce the correlation tabular output.

<br>
<div style="text-align: right"> 
    <a href="#statistics_corr_table_atc_l2">[Go to table]</a>
</div>

In [ ]:
def update_corr_stat_multi_binary(station_dobj_ls, start_date, end_date):
    
    """
    Project:         'ICOS Carbon Portal'
    Created:          Tue May 07 10:30:00 2018
    Last Changed:     Tue May 07 10:30:00 2019
    Version:          1.0.0
    Author(s):        Karolina
    
    Description:      Function that takes as input a list of sublists, where every sublist contains an
                      ICOS Data Object ID and a tracer string. The function reads the corresponding
                      ICOS Level-2 Atmospheric Tracer data files for every data object ID into sepparate
                      pandas dataframes. Every data file produces two sepparate pandas dataframes;
                      metadata dataframe and data dataframe. The data dataframe is filtered to only include
                      values for the time period that was selected by the user.
                      Then the correlation is computed between every station and the results are returned
                      in a pandas dataframe.
                                        
    Input parameters: 1. List with sublists of data object IDs and tracers
                         (var_name: 'station_dobj_ls', var_type: List)
                      2. Start date - user's input
                         (var_name: 'start_date', var_type: DateTime Object)
                      3. End date - user's input
                         (var_name: 'end_date', var_type: DateTime Object)
                      
    Output:           Pandas DataFrame
    
    """
    
    #Import modules:
    import pandas as pd
    from icoscp.cpb.dobj import Dobj
    
    

    #Add dictionary to transform digits to subscript:
    SUB = str.maketrans("0123456789", "₀₁₂₃₄₅₆₇₈₉")
    
    #Get pandas dataframe with all ICOS stations:
    icos_stations_df = get_coords_icos_stations_atc()

    #Create list to store the metadata & data dataframes for all combinations of tracers and stations:
    station_df_ls = []

    
    #Loop through every data object ID in the list:
    for station_ls in station_dobj_ls:
        
        #Create a file object from the 1st object in the data object id list:
        file = Dobj(station_ls[0][0])  
        
        #Create dictionary to store tracer info:
        tracer_info_dict = {}
        
        #Create a dictionary to store all station info:
        station_info_dict = {}
        
        #Create list to store the data dataframes of all data object IDs for :
        df_ls = []
        
        #Get the tracer description:
        tracer_info_dict['tracer_info'] = file.info[1].valueType.loc[file.info[1].colName==station_ls[0][1]].values[0]

        #Get tracer unit:
        tracer_info_dict['tracer_unit'] = file.info[1].unit.loc[file.info[1].colName==station_ls[0][1]].values[0]

        #Get station info:
        station_info_dict['station_name'] = icos_stations_df.stationName.loc[icos_stations_df.stationId==station_ls[0][2]].values[0]
        station_info_dict['station_code'] = station_ls[0][2]
        station_info_dict['station_sampling_height'] = station_ls[0][3]
        station_info_dict['station_country_code'] = icos_stations_df.Country.loc[icos_stations_df.stationId==station_ls[0][2]].values[0]
        station_info_dict['station_country'] = get_country_fullname_from_iso3166_2char(station_info_dict['station_country_code'])
        station_info_dict['station_lat'] = icos_stations_df.lat.loc[icos_stations_df.stationId==station_ls[0][2]].values[0]
        station_info_dict['station_lon'] = icos_stations_df.lon.loc[icos_stations_df.stationId==station_ls[0][2]].values[0]
        

        #station_info[0] --- > dataobjid (e.g. 'MdYIndlCMyEp2BoGwUL_0Jqq')
        #station_info[1] --- > Station Code (e.g. 'HPB')
        #station_info[2] --- > Sampling Height (e.g. '50.0')
        #station_info[3] --- > Tracer (e.g. 'co2')

        #Download data from all files that correspond to the same station,
        #at the same sampling height and include data for the same tracer
        #in metadata- and data- dataframes that are stored as sublists in a list:
        df_ls = [Dobj(station_info[0]).get() for station_info in station_ls]
        
        #Check size of df_ls:
        if(len(df_ls)>1):
            #Concatenate the data-dataframes that include tracer data for
            #the same station at the same sampling height, to one data-dataframe:
            data_df = pd.concat([df for df in df_ls])
            
        else:
            data_df = df_ls[0]
            
            
        #Add column with datetime object:
        data_df['DateTime'] = pd.to_datetime(data_df['TIMESTAMP'], unit='ms') 

        #Create a copy of the dataframe and set "DateTime" as index:
        data_df_ind = data_df.copy().set_index('DateTime')

        #Sort the dataframe index in ascending order:
        data_df_ind.sort_index(inplace=True)
            
        #Filter dataframe to only contain data between the selected dates:
        data_df_filt = data_df_ind.loc[start_date:end_date]
        
        #Append metadata-dataframe and concatenated data-dataframe to list:
        station_df_ls.append([data_df_filt, station_info_dict, tracer_info_dict])
    
    
    #Extract the tracer-column from every station's data dataframe to a new pandas dataframe:
    tracer_df_ls =[pd.DataFrame({station_df_ls[i][1]['station_code']+'_'+
                                 station_df_ls[i][1]['station_sampling_height']+' ('+
                                 station_df_ls[i][2]['tracer_info'].replace(' mixing ratio (dry mole fraction)', '').translate(SUB)+
                                 ')': station_df_ls[i][0][station_df_ls[i][2]['tracer_info'].replace(' mixing ratio (dry mole fraction)', '').lower()]})
                   for i in range(len(station_df_ls))]
    
    #Concatenate dataframes to one final dataframe:
    tracers_df = pd.concat(tracer_df_ls, axis=1)
    
    #Get correlation between data:
    corr_df = tracers_df.corr(method='pearson')
    
    #Return dataframe:
    return corr_df
    


In [ ]:
def check_input_icos_L2_correlation(Tracer, Station, start_date, end_date):
    
    """
    Project:         'ICOS Carbon Portal'
    Created:          Fri May 17 14:30:00 2019
    Last Changed:     Fri May 17 14:30:00 2019
    Version:          1.0.0
    Author(s):        Karolina
    
    Description:      Function that checks the user input to widgets and returns a boolean for
                      invalid or empty input.
                      
    Input parameters: 1. List with Tracer(s)/gas(es) (Long Text)
                         (var_name: 'Tracer', var_type: List)
                      2. List of sublists, where every sublist contains the Station ID & Sampling Height
                         (var_name: 'Station', var_type: List)
                      3. Start date of time period
                         (var_name: 'start_date', var_type: DateTime Object)
                      4. End date of time period
                         (var_name: 'end_date', var_type: DateTime Object)

    Output:           Pandas DataFrame
    
    """
    
    #Check if NO tracer has been selected:
    if(len(Tracer)<1):

        #Print message:
        print(("\033[0;31;1m Select tracer! \033[0;31;0m\n\n"))

        #Return boolean:
        return False
        
    #Check if a tracer has been selected:
    else:
    
        #Check if NO station has been selected:
        if(len(Station)<1):

            #Print message:
            print(("\033[0;31;1m Select station! \033[0;31;0m\n\n"))

            #Return boolean:
            return False

        #Check if a station has been selected:
        else:

            #Check if a start-date and/or end-date have been selected:
            if((start_date==None)|(end_date==None)):

                #Print message:
                print(("\033[0;31;1m Select a start date and/or end date! \033[0;31;0m\n\n"))

                #Return boolean:
                return False

            #If a start-date and an end-date have been selected:
            else:

                #Compute the difference between end_date and start_date:
                diff = end_date - start_date

                #Check if end-date refers to an earlier date than start-date:
                if(diff.days<0):

                    #Print message:
                    print('\033[0;31;1m Error...\n The selected start-date corresponds to a later date than the selected end-date.\n Enter new dates!\n\n')

                    #Return boolean:
                    return False

                #If the selected start-date & end-date are valid:
                else:

                    #Return boolean:
                    return True

  

In [ ]:
def create_widgets_correlation_multiple_tracers_multiple_stations():
    
    """
    Project:         'ICOS Carbon Portal'
    Created:          Tue May 07 10:30:00 2018
    Last Changed:     Tue May 07 10:30:00 2019
    Version:          1.0.0
    Author(s):        Karolina
    
    Description:      Function that creates a set of widgets; a station multiselect dropdown list, a
                      tracer multiselect dropdown list and a button. The function populates the dropdown
                      lists with values and outputs the result.
                      
                      
    Input parameters: No Input Parameter(s)

    Output:           Python Widgets
    
    """

    #Create lookup dataframe:
    df_lookup = create_lookup_df_atc_L2()
    
    #Convert data type of date-columns from string to datetime:
    df_lookup.timeStart = [datetime.strptime(df_lookup.timeStart.iloc[i],'%Y-%m-%dT%H:%M:%SZ')
                    for i in range(len(df_lookup))]
    df_lookup.timeEnd = [datetime.strptime(df_lookup.timeEnd.iloc[i],'%Y-%m-%dT%H:%M:%SZ')
                  for i in range(len(df_lookup))]
    
    #Create a list including all tracers (e.g. CO2, CO, CH4)
    tracers = df_lookup.variable.unique().tolist()

    #reverse list order:
    tracers.reverse()

    #Create widgets:
    tracers = SelectMultiple(options = tracers, disabled=False)
    stations = SelectMultiple(options = create_station_labels(df_lookup), disabled=False)


    #Function that calls functions to update the plot/s and/or map,
    #based on the selected tracer, station and color:
    def update_plot_func(Tracers, Stations, start_date, end_date):
        
        #Control input parameters: 
        if(check_input_icos_L2_correlation(Tracers, Stations, start_date, end_date)):
        
            #Get a list of sublists, where every sublist contains the following:
            #1. ICOS Level-2 data object URL
            #2. ICOS Station ID (3-character code)
            #3. ICOS Station Sampling Height
            #4. Tracer/Gas (e.g. 'co2')
            #that refer to the selected station(s) and tracer(s):
            selection_dobj_url_list = [[df_lookup.dobj.loc[(df_lookup.stationId==station[0]) &
                                                           (df_lookup.height==station[1]) &
                                                           (df_lookup.variable==tracer) &
                                                           (((df_lookup.timeStart<=pd.Timestamp(start_date))&
                                                             (df_lookup.timeEnd>=pd.Timestamp(start_date)))|
                                                            ((df_lookup.timeStart<=pd.Timestamp(end_date))&
                                                             (df_lookup.timeEnd>=pd.Timestamp(end_date)))|
                                                            ((df_lookup.timeStart>=pd.Timestamp(start_date))&
                                                             (df_lookup.timeEnd<=pd.Timestamp(end_date))))].values,
                                        tracer.replace(' mixing ratio (dry mole fraction)', '').lower(),
                                        station[0],
                                        station[1]]
                                       for tracer in Tracers for station in Stations]
            
            #Get a list of items, where every item represents the size of the url-array for every selected station:
            check_url_ls_size = [selection_dobj_url_list[m][0].size for m in range(len(selection_dobj_url_list))]

            #If Level-2 data are available for the selected tracer(s), station(s) and time period:
            if(sum(check_url_ls_size)>0):

                #Get a list of lists, where every sublist contains a data-object-ID, the station code, 
                #the sampling height & the selected tracer.
                #E.g. ['U4VYazHdmZwzr7DxUowMtUu-', 'GAT', '30.0', 'co2']:
                selection_list = [[selection_dobj_url_list[i][0][j].replace('https://meta.icos-cp.eu/objects/', ''),
                                   selection_dobj_url_list[i][1],
                                   selection_dobj_url_list[i][2],
                                   selection_dobj_url_list[i][3]]
                                  for i in range(len(selection_dobj_url_list))
                                  for j in range(len(selection_dobj_url_list[i][0]))]

                #Get a list of tuples (e.g. ('HPB', '50.0', 'co2')) with unique occurances of
                #"station code" - "station sampling height" - "tracer" triplets:
                station_unique_ls = list(set([(item[1], item[2], item[3]) for item in selection_list]))

                #Group selection_list items refering to tracer-data from the same station and sampling height to lists:  
                station_dobj_ls = [[item for item in selection_list
                                    if((item[1]==station_id[0]) & (item[2]==station_id[1]) & (item[3]==station_id[2]))]
                                   for station_id in station_unique_ls]


                #Call function to calculate and return a correlation statistics dataframe for the selected station:
                return update_corr_stat_multi_binary(station_dobj_ls, start_date, end_date)

            
            #If no data are available for the selected tracer, station(s) and time period:
            else:
                print("\033[0;31;1m No ICOS Atmospheric Level-2 data available.\033[0;31;0m\n\n")



    #Create function that contains a box of widgets:
    interact_c = interact_manual(update_plot_func,
                                 Tracers = tracers,
                                 Stations = stations,
                                 start_date=DatePicker(description='Starting Date',disabled=False),
                                 end_date=DatePicker(description='Ending Date',disabled=False))

    #Set the font of the widgets included in interact_manual:
    interact_c.widget.children[0].layout.width = '430px'
    interact_c.widget.children[0].layout.height = '60px'
    interact_c.widget.children[1].layout.width = '430px'
    interact_c.widget.children[2].layout.width = '430px'
    interact_c.widget.children[3].layout.width = '430px'
    interact_c.widget.children[4].description = 'Update Table'
    interact_c.widget.children[4].button_style = 'danger'
    interact_c.widget.children[4].style.button_color = '#3973ac'
    #interact_c.widget.children[4].layout.width = '300px'
    interact_c.widget.children[4].layout.margin = '10px 10px 20px 180px' # top/right/bottom/left
    

<a id='statistics_corr_table_atc_l2'></a>
#### 4.2.2. Correlation - ICOS Level 2 Atmospheric Data --- > Output Table

In [ ]:
#Call function to create widgets:
create_widgets_correlation_multiple_tracers_multiple_stations()

<br>
<div style="text-align: right"> 
    <a href="#statistics_correlation_atc_l2">[Back to statistics-correlation]</a>
    &ensp;&ensp;
    <a href="#introduction">Back to top</a>
</div>
<br>
<br>

<a id='statistics_smoothing_atc_l2'></a>

### 4.3. Statistics - Smoothing -  ICOS Level 2 Atmospheric Data
This section includes a tool that allows the user to plot averaged observation values per tracer and station. The observation values can be averaged over a window of a selected number of days. A slider widget controls the selection of total number of days. It is possible to select between 0 and 90 days. Two single select widgets are used to control the selection of station and tracer category correspondigly.

<div style="text-align: right"> 
    <a href="#statistics_atc_l2">[Back to statistics]</a>
</div>

#### 4.3.1. Smoothing - ICOS Level 2 Atmospheric Data ---> Python Code
This part presents the python code used to produce the smoothing result.

<div style="text-align: right"> 
    <a href="#smoothing_plot_atc_l2">[Go to plot]</a>
</div>

In [ ]:
def plot_icos_single_station_smoothing_binary(df_data, station_info_dict, tracer_info_dict, tracer, color):
    
    """
    Project:         'ICOS Carbon Portal'
    Created:          Tue May 07 10:30:00 2018
    Last Changed:     Tue May 07 10:30:00 2019
    Version:          1.0.0
    Author(s):        Karolina
    
    Description:      Function that takes as input a metadata dataframe, a data dataframe, a string stating
                      the tracer and a string with the color for the plot of datafor a specific ICOS station. 
                      The function creates an interactive Bokeh plot with the contents of the dataframes and
                      returns a Bokeh Figure (plot).
                      
                      
    Input parameters: 1. ICOS Level-2 tracer Atmospheric Data Dataframe
                         (var_name: 'df_data', var_type: Pandas DataFrame)
                      2. Dictionary with ICOS Station info
                         (var_name: 'station_info_dict', var_type: Dictionary)
                      3. Dictionary with tracer info
                         (var_name: 'tracer_info_dict', var_type: Dictionary)
                      4. Tracer/gas - e.g. 'co2'
                         (var_name: "tracer", var_type: String)
                      5. Plot Color
                         (var_name: "color", var_type: String)

    Output:           Bokeh Plot
    
    """
    
    #Import modules to create figure:
    import pandas as pd
    from bokeh.plotting import figure
    from bokeh.models import ColumnDataSource, HoverTool, Label, Legend
    from datetime import datetime

    #Dictionary for subscript transformations of numbers:
    SUB = str.maketrans("0123456789", "₀₁₂₃₄₅₆₇₈₉")
    SUP = str.maketrans("0123456789", "⁰¹²³⁴⁵⁶⁷⁸⁹")

    #Define Datasets:
    x = df_data.index.values
    y1 = df_data['rolling_mean']#.astype({tracer: np.float32})[tracer] 
    y2 = df_data[tracer_info_dict['tracer_info'].replace(' mixing ratio (dry mole fraction)','').lower()]

    #Create a figure object:
    p = figure(plot_width=900,
               plot_height=400,
               x_axis_label='Time (UTC)', 
               y_axis_label=tracer_info_dict['tracer_info'].replace(' mixing ratio (dry mole fraction)','').translate(SUB) + ' (' +
               tracer_info_dict['tracer_unit'].translate(SUP) + ')',
               x_axis_type='datetime',
               title = tracer_info_dict['tracer_info'].translate(SUB)+'   '+
               station_info_dict['station_name']+', '+
               station_info_dict['station_country']+', '+
               station_info_dict['station_sampling_height']+' m. a. g. l.',
               tools='pan,box_zoom,wheel_zoom,,undo,reset,save')

    #Create an empty list that will store the legend info:
    legend_it = []
    
    #Create glyphs:
    g0 = p.circle(x, y1, radius=.02, color=color)# ,legend=df_metadata.loc['STATION CODE'].values[0])
    g1 = p.line(x, y1, line_width=1.25, color=color, name='Smoothed Line')
    
    g2 = p.circle(x, y2, radius=.02, color=color)# ,legend=df_metadata.loc['STATION CODE'].values[0])
    g3 = p.line(x, y2, line_width=1, line_dash='dotted', line_alpha=0.5, color=color, name='Original Observations')
    
    #Add the name and glyph info (i.e. colour and marker type) to the legend:
    legend_it.append(('Smoothed '+station_info_dict['station_code']+' ('+
                      station_info_dict['station_sampling_height']+')', [g1]))
    
    #Add the name and glyph info (i.e. colour and marker type) to the legend:
    legend_it.append(('Original ' + station_info_dict['station_code']+' ('+
                      station_info_dict['station_sampling_height']+')', [g3]))

    
    #Add tooltip on hover:
    p.add_tools(HoverTool(tooltips=[
        ('Type','$name'),
        ('Time (UTC)','@x{%Y-%m-%d %H:%M:%S}'),
        (tracer_info_dict['tracer_info'].replace(' mixing ratio (dry mole fraction)','').translate(SUB),'@y{0.f}'),
        ],
        formatters={
            '@x'      : 'datetime', # use 'datetime' formatter for 'date' field
            },
        # display a tooltip whenever the cursor is vertically in line with a glyph
        mode='vline'
        ))    

    #Create legend:
    legend = Legend(items=legend_it, location= 'bottom_center')
    legend.orientation = 'horizontal'
    legend.click_policy='hide'
    legend.spacing = 10 #sets the distance between legend entries

    #Set title attributes:
    p.title.align = 'center'
    p.title.text_font_size = '13pt'
    p.title.offset = 15

    #Set label font style:
    p.xaxis.axis_label_text_font_style = 'normal'
    p.yaxis.axis_label_text_font_style = 'normal'
    p.xaxis.axis_label_standoff = 15 #Sets the distance of the label from the x-axis in screen units
    p.yaxis.axis_label_standoff = 15 #Sets the distance of the label from the y-axis in screen units

    #Set the copyright label position:
    label_opts = dict(x=0, y=10,
                      x_units='screen', y_units='screen')

    #Create a label object and format it:
    caption1 = Label(text="© ICOS ERIC", **label_opts)
    caption1.text_font_size = '8pt'
    
    #Deactivate hover-tool, which is by default active:
    p.toolbar.active_inspect = None

    #Add label to plot:
    p.add_layout(caption1, 'below')
    
    #Add legend to figure:
    p.add_layout(legend, 'below')

    #return plot:
    return p

In [ ]:
#Function that updates the plot every time the user interacts with a widget:
def update_smoothing_plot_binary(data_obj_id_ls, station, tracer, days, color):
    
    """
    Project:         'ICOS Carbon Portal'
    Created:          Tue May 07 10:30:00 2018
    Last Changed:     Tue May 07 10:30:00 2019
    Version:          1.0.0
    Author(s):        Karolina
    
    
    Description:      Function that takes as input an ICOS data object ID, the station name,
                      the tracer type, the total number of days over which to average the
                      ICOS tracer data and the plot color. The function reads the corresponding
                      ICOS Level-2 Atmospheric Tracer data files for every data object ID into sepparate
                      pandas dataframes. Every data file produces two sepparate pandas dataframes;
                      metadata dataframe and data dataframe. The values of the tracer in the data
                      dataframe are then averaged by a window that corresponds to the related input
                      parameter. The updated data dataframe and the metadata dataframe are then set as input
                      parameters to a plot function, that returns a Bokeh Figure (plot).
                      The Bokeh Figure is then returned as output.
                      
                      
    Input parameters: 1. List with sublists of data object IDs
                         (var_name: 'data_obj_id_ls', var_type: List)
                      2. ICOS Station 3-charcter Code
                         (var_name: 'station', var_type: String)
                      3. Tracer/gas, e.g. 'co2'
                         (var_name: 'tracer', var_type: String)
                      4. Number of days to average by
                         (var_name: 'days', var_type: Integer)
                      5. Plot Color
                         (var_name: 'color', var_type: String)
       
       
    Output:           Bokeh Plot
    
    """
    
    #import modules:
    import pandas as pd
    from bokeh.layouts import column
    from icoscp.cpb.dobj import Dobj
    
    #Create dictionary to store tracer info:
    tracer_info_dict = {}
    
    #Create a dictionary to store the station info:
    station_info_dict = {}
    
    #Create a file object from the 1st object in the data object id list:
    file = Dobj(data_obj_id_ls[0])  
    
    #Get the tracer description:
    tracer_info_dict['tracer_info'] = file.info[1].valueType.loc[file.info[1].colName==tracer].values[0]
    
    #Get tracer unit:
    tracer_info_dict['tracer_unit'] = file.info[1].unit.loc[file.info[1].colName==tracer].values[0]
    
    
    #Get pandas dataframe with all ICOS stations:
    icos_stations_df = get_coords_icos_stations_atc()
    
    #Get station info:
    station_info_dict['station_name'] = icos_stations_df.stationName.loc[icos_stations_df.stationId==station[0]].values[0]
    station_info_dict['station_code'] = station[0]
    station_info_dict['station_sampling_height'] = station[1]
    station_info_dict['station_country_code'] = icos_stations_df.Country.loc[icos_stations_df.stationId==station[0]].values[0]
    station_info_dict['station_country'] = get_country_fullname_from_iso3166_2char(station_info_dict['station_country_code'])
    station_info_dict['station_lat'] = icos_stations_df.lat.loc[icos_stations_df.stationId==station[0]].values[0]
    station_info_dict['station_lon'] = icos_stations_df.lon.loc[icos_stations_df.stationId==station[0]].values[0]
    
    
    
    
    #Create list to store the data dataframes of all data object IDs:
    data_df_ls = []
    
    #Loop through every data object ID in the list:
    for dobjid in data_obj_id_ls: 
        
        #Get all the columns for the selected dataobject id:
        obs_data_df = Dobj(dobjid).get()
        
        #Add data dataframe of the current data object ID to the list:
        data_df_ls.append(obs_data_df)
        
    #Concatenate data dataframes to one dataframe:
    data_df = pd.concat(data_df_ls)
    
    #Add column with datetime object:
    data_df['DateTime'] = pd.to_datetime(data_df['TIMESTAMP'], unit='ms') 
    
    #Create a copy of the dataframe and set "DateTime" as index:
    data_df_ind = data_df.copy().set_index('DateTime')
    
    #Sort the dataframe index in ascending order:
    data_df_ind.sort_index(inplace=True)
    
    

    #Smoothing the glyph-line, by using a window of a selected number of days that averages the values:
    if days == 0:
        data_df_ind['rolling_mean'] = data_df_ind[tracer]
    
    #If the number of days is higher than zero:
    else:
        
        #If number of days is an even number
        if(days%2==0):
            data_df_ind['rolling_mean'] = data_df_ind[tracer].rolling('{0}D'.format(days), closed='left').mean().shift(int(-days/2)*24, freq='h')
        
        #If number of days is an odd number:
        else:
            data_df_ind['rolling_mean'] = data_df_ind[tracer].rolling('{0}D'.format(days), closed='left').mean().shift(round(-days/2)*24, freq='h')
            
    
    #Plot station:
    p = plot_icos_single_station_smoothing_binary(data_df_ind, station_info_dict, tracer_info_dict, tracer, color)
    
    #Output should be in the notebook
    output_notebook()
    
    #Show plot
    show(p, notebook_handle=True)  


In [ ]:
def create_smoothing_widgets():
    
    """
    Project:         'ICOS Carbon Portal'
    Created:          Tue May 07 10:30:00 2018
    Last Changed:     Tue May 07 10:30:00 2019
    Version:          1.0.0
    Author(s):        Karolina
    
    Description:      Function that creates a set of widgets; a station dropdown list, a tracer dropdown
                      list, a 'total num of days' slider, a color-picker and a button. The function populates
                      the dropdown lists and slider with values and outputs the result.
                      
                      
    Input parameters: No Input Parameter(s)

    Output:           Python Widgets
    
    """
    
    #Import modules:
    from bokeh.plotting import figure
    
    #Dictionary for subscript transformations of numbers:
    SUB = str.maketrans("0123456789", "₀₁₂₃₄₅₆₇₈₉")

    #Create lookup dataframe:
    df_lookup = create_lookup_df_atc_L2()
    
    #Create a list including all tracers (e.g. CO2, CO, CH4)
    tracers = df_lookup.variable.unique().tolist()
    
    #reverse list order:
    tracers.reverse()

    #Create widgets:
    tracer = Dropdown(options = tracers)
    station =  Dropdown(options = create_station_labels(df_lookup))

    #Function that calls functions to update the plot/s and/or map,
    #based on the selected tracer, station and color:
    def update_plot_func(Tracer, Station, Days, Color, Citation):

        #Get tracer (e.g. 'co2'):
        tracer_low_case = Tracer.replace(' mixing ratio (dry mole fraction)', '').lower()
        
        #Get a list of data obect URLs that refer to the selected station and tracer:
        data_obj_url_ls = df_lookup.dobj.loc[(df_lookup.stationId==Station[0]) &
                                             (df_lookup.height==Station[1]) &
                                             (df_lookup.variable==Tracer)].values
        
        #If L2-data is available for the selected tracer and station:
        if(data_obj_url_ls.size>0):
            
            #Get a list of data object IDs (L2-data):
            data_obj_id_ls = [data_obj_url_ls[i].replace('https://meta.icos-cp.eu/objects/', '') 
                                 for i in range(data_obj_url_ls.size)]
            
            #Call function to return plot and map for the selected station:
            update_smoothing_plot_binary(data_obj_id_ls, Station, tracer_low_case, Days, Color)
            
            
            #If the "citation" checkbox is checked:
            if(Citation):
                
                #Get a list with citation info for every ICOS Level-2 data object:
                cit_ls_L2 = [get_icos_citation(dobj).cit.iloc[0] for dobj in data_obj_url_ls]
                    
                #Print citation title:
                print('\n\n\033[1m' + 'Data Citation:' +  '\033[0m')
                    
                #Loop through all citations:
                for cit in cit_ls_L2:
                    
                    #Print data object citation:
                    printmd("<sub>"+cit+"</sub>")
                    
        
        #If no L2-data are available for the selected tracer and station:
        else:
            print('\033[0;31;1m '+ 'No '+tracer_low_case.upper().translate(SUB)+' Level-2 data available for the selected station' +'\033[0;31;0m\n\n')
        
    #Create function that contains a box of widgets:
    interact_c = interact_manual(update_plot_func,
                                 Tracer=tracer,
                                 Station=station,
                                 Days = (0,90),
                                 Color=ColorPicker(concise=False,
                                                   description='Pick a color',
                                                   value='#3973ac',
                                                   disabled=False),
                                 Citation=Checkbox(value=True, description='Citation', disabled=False))

    #Set the font of the widgets included in interact_manual:
    interact_c.widget.children[0].layout.width = '420px'
    interact_c.widget.children[0].layout.margin = '40px 2px 2px 2px'
    interact_c.widget.children[1].layout.width = '420px'
    interact_c.widget.children[2].layout.width = '420px'
    interact_c.widget.children[3].layout.width = '420px'
    interact_c.widget.children[4].layout.width = '420px'
    interact_c.widget.children[5].description = 'Update Plot'
    interact_c.widget.children[5].button_style = 'danger'
    interact_c.widget.children[5].style.button_color = '#3973ac'
    interact_c.widget.children[5].layout.margin = '10px 10px 40px 180px' # top/right/bottom/left

<a id='smoothing_plot_atc_l2'></a>
#### 4.3.2. Smoothing - ICOS Level 2 Atmospheric Data ---> Plot

In [ ]:
#Call function to display widgets for the corresponding output type:
create_smoothing_widgets()

<div style="text-align: right"> 
    <a href="#statistics_smoothing_atc_l2">[Back to statistics-smoothing]</a>
    &ensp;&ensp;
    <a href="#introduction">Back to top</a>
</div>
<br>
<br>
<br>

<a id='comparing_atc_l2'></a>

## 5. Comparing plots from multiple stations - ICOS Level 2 Atmospheric Data
In this part it is possible to compare plots from multiple stations by plotting them one under another. The selection of stations is handled by the multiselect-widget. If you wish to select more than one station, keep the CONTROL-key (alt. COMMAND-key) pressed and continue selecting stations by mouse-click. Once all desired stations are selected click on the "Update Plot/s" button to view the result.

It is also possible to select consecutive stations by holding down the SHIFT-key and clicking with the mouse on the first and last station.

<br>
<br>
<div style="text-align: right"> 
    <a href="#introduction">Back to top</a>
</div>

### 5.1. Comparing - ICOS Level 2 Atmospheric Data ---> Python Code
This part presents the python code used to produce the "comparing" plots.

<div style="text-align: right"> 
    <a href="#comparing_plot_atc_l2">[Go to plot]</a>
</div>

In [ ]:
def update_comparing_binary(station_dobj_ls, num_of_stations):
    
    """
    Project:         'ICOS Carbon Portal'
    Created:          Tue May 07 10:30:00 2018
    Last Changed:     Tue May 07 10:30:00 2019
    Version:          1.0.0
    Author(s):        Karolina
    
    Description:      Function that takes as input a list of sublists, where every sublist contains an
                      ICOS Data Object ID and a tracer string. The function aslo takes as input an integer
                      representing the total number of stations selected. This parameter is then used to get
                      a sublist of the same number of different colors from a colormap. The function reads
                      the corresponding ICOS Level-2 Atmospheric Tracer data files for every data object ID
                      into sepparate pandas dataframes. Every data file produces two sepparate pandas dataframes;
                      metadata dataframe and data dataframe. Then, every set of metadata- and data-dataframes
                      are sent as input parameters to a plot function, that will produce a sepparate plot
                      for every station.
                      
                      
    Input parameters: 1. List with sublists of data object IDs and tracers
                         (var_name: 'station_dobj_ls', var_type: List)
                         
                      2. Total number of stations selected
                         (var_name: 'num_of_stations', var_type: Integer)
                      

    Output:           Interactive Bokeh Plot(s)
    
    """
    
    #import modules:
    from bokeh.layouts import column
    from bokeh.io import push_notebook, output_notebook
    from bokeh.plotting import show
    from icoscp.cpb.dobj import Dobj
    
    #Get colormap:
    colormap = get_colormap(num_of_stations)
    
    #Create dictionary to store tracer info:
    tracer_info_dict = {}
    
    #Create dict to store the station info:
    station_info_dict = {}
    
    #Create a file object from the 1st object in the data object id list:
    file = Dobj(station_dobj_ls[0][0][0])  
    
    #Get the tracer description:
    tracer_info_dict['tracer_info'] = file.info[1].valueType.loc[file.info[1].colName==station_dobj_ls[0][0][1]].values[0]
    
    #Get tracer unit:
    tracer_info_dict['tracer_unit'] = file.info[1].unit.loc[file.info[1].colName==station_dobj_ls[0][0][1]].values[0]
    
    
    #Get pandas dataframe with all ICOS stations:
    icos_stations_df = get_coords_icos_stations_atc()
    
    
    
    #Define and initialize list to store station plots:
    plot_list = []
    
    #Add counter:
    counter = 0

    #Loop through every data object ID in the list:
    for station_ls in station_dobj_ls:
        
        #Get station info:
        station_info_dict['station_name'] = icos_stations_df.stationName.loc[icos_stations_df.stationId==station_ls[0][2]].values[0]
        station_info_dict['station_code'] = station_ls[0][2]
        station_info_dict['station_sampling_height'] = station_ls[0][3]
        station_info_dict['station_country_code'] = icos_stations_df.Country.loc[icos_stations_df.stationId==station_ls[0][2]].values[0]
        station_info_dict['station_country'] = get_country_fullname_from_iso3166_2char(station_info_dict['station_country_code'])
        station_info_dict['station_lat'] = icos_stations_df.lat.loc[icos_stations_df.stationId==station_ls[0][2]].values[0]
        station_info_dict['station_lon'] = icos_stations_df.lon.loc[icos_stations_df.stationId==station_ls[0][2]].values[0]


        #Create list to store the data dataframes of all data object IDs for :
        df_ls = []

        #station_info[0] --- > dataobjid (e.g. 'MdYIndlCMyEp2BoGwUL_0Jqq')
        #station_info[1] --- > Tracer (e.g. 'co2')
        #station_info[2] --- > Station Code (e.g. 'HPB')
        #station_info[3] --- > Sampling Height (e.g. '50.0')
        
        
        #Download data from all files that correspond to the same station,
        #at the same sampling height and include data for the same tracer
        #in metadata- and data- dataframes that are stored as sublists in a list:
        df_ls = [Dobj(station_info[0]).get()
                 for station_info in station_ls]

        #Concatenate the data-dataframes that include tracer data for
        #the same station at the same sampling height, to one data-dataframe:
        data_df = pd.concat([df for df in df_ls])
        #data_df.sort_index(inplace=True)
        
        #Add plot-object to list:
        plot_list.append(plot_icos_single_station_binary(data_df,
                                                         station_info_dict,
                                                         tracer_info_dict,
                                                         color=colormap[counter]))
        
        #Increase counter:
        counter = counter + 1
    
    #Organize the plots in a "column" layout:
    layout = column(plot_list)
    
    #Output will be displayed in notebook:
    output_notebook()
    
    #Show plot/s:
    show(layout, notebook_handle=True)


In [ ]:
def create_widgets_comparing():
    
    """
    Project:         'ICOS Carbon Portal'
    Created:          Tue May 07 10:30:00 2018
    Last Changed:     Tue May 07 10:30:00 2019
    Version:          1.0.0
    Author(s):        Karolina
    
    Description:      Function that creates a set of widgets; a multiselect station dropdown list, a
                      tracer dropdown list and a button. The function populates the dropdown lists
                      with values and outputs the result.
                      
                      
    Input parameters: No Input Parameter(s)

    Output:           Python Widgets
    
    """

    #Create lookup dataframe:
    df_lookup = create_lookup_df_atc_L2()
    
    #Create a list including all tracers (e.g. CO2, CO, CH4)
    tracers = df_lookup.variable.unique().tolist()
    
    #reverse list order:
    tracers.reverse()

    #Create widgets:
    tracer = Dropdown(options = tracers)
    station = SelectMultiple(options = create_station_labels(df_lookup), disabled=False)

    #Function that calls functions to update the plot/s and/or map,
    #based on the selected tracer, station and color:
    def update_plot_func(Tracer, Station, Citation):
        
        #Get tracer (e.g. 'co2'):
        tracer_low_case = Tracer.replace(' mixing ratio (dry mole fraction)', '').lower()
        
        #Get a list of sublists, where every sublist contains the following:
        #1. ICOS Level-2 data object URL
        #2. Tracer/Gas (e.g. 'co2')
        #3. ICOS Station ID (3-character code)
        #4. ICOS Station Sampling Height
        #that refer to the selected station(s) and tracer:
        selection_dobj_url_list = [[df_lookup.dobj.loc[(df_lookup.stationId==station[0]) &
                                                       (df_lookup.height==station[1]) &
                                                       (df_lookup.variable==Tracer)].values,
                                    tracer_low_case,
                                    station[0],
                                    station[1]]
                                   for station in Station
                                   if df_lookup.dobj.loc[(df_lookup.stationId==station[0]) &
                                                         (df_lookup.height==station[1]) &
                                                         (df_lookup.variable==Tracer)].values.size>0]
        
        
        #If Level-2 data are available for the selected tracer and station(s):
        if(len(selection_dobj_url_list)>0):

            #Get a list of lists, where every sublist contains a data-object-ID, the station code, 
            #the sampling height & the selected tracer.
            #E.g. ['U4VYazHdmZwzr7DxUowMtUu-', 'co2', 'GAT', '30.0']:
            selection_list = [[selection_dobj_url_list[i][0][j].replace('https://meta.icos-cp.eu/objects/', ''),
                               selection_dobj_url_list[i][1],
                               selection_dobj_url_list[i][2],
                               selection_dobj_url_list[i][3]]
                              for i in range(len(selection_dobj_url_list))
                              for j in range(len(selection_dobj_url_list[i][0]))]
            
            ####
            #ICOS Atmospheric Level-2 Data for a given station, a given tracer and  
            #a given sampling height can be stored in two different files in cases
            #where the measuring instrument has changed.
            #The following code controls for such occurances and merges the data
            #if necessary.
            ####
            
            #Get a list of tuples (e.g. ('HPB', '50.0', 'co2')) with unique occurances of
            #"station code" - "station sampling height" - "tracer" triplets:
            station_unique_ls = list(set([(item[1], item[2], item[3]) for item in selection_list]))
            
            #Group selection_list items refering to tracer-data from the same station and sampling height to lists:  
            station_dobj_ls = [[item for item in selection_list
                                if((item[1]==station_id[0]) & (item[2]==station_id[1]) & (item[3]==station_id[2]))]
                               for station_id in station_unique_ls]
            
            #Get plot displaying tracer-values for the selected station/s:
            update_comparing_binary(station_dobj_ls, len(station_unique_ls))
            
            #If the "citation" checkbox is checked:
            if(Citation):
                
                #Get a list with citation info for every ICOS Level-2 data object:
                cit_ls_L2 = [get_icos_citation(dobj[0]).cit.iloc[0] for dobj in selection_dobj_url_list]
                    
                #Print citation title:
                print('\n\n\033[1m' + 'Data Citation:' +  '\033[0m')
                    
                #Loop through all citations:
                for cit in cit_ls_L2:
                    
                    #Print data object citation:
                    printmd("<sub>"+cit+"</sub>")
        
        #If no L2-data is available for the selected tracer and station:
        else:
            print('\033[0;31;1m '+ 'No Level-2 data available for the selected tracer and/or station/s.' +'\033[0;31;0m\n\n')
            

            

    #Create function that contains a box of widgets:
    interact_c = interact_manual(update_plot_func,
                                 Tracer=tracer,
                                 Station=station,
                                 Citation=Checkbox(value=True, description='Citation', disabled=False))

    #Set the font of the widgets included in interact_manual:
    interact_c.widget.children[0].layout.width = '430px'
    interact_c.widget.children[0].layout.margin = '40px 2px 2px 2px'
    interact_c.widget.children[1].layout.width = '430px'
    interact_c.widget.children[1].layout.height = '120px'
    interact_c.widget.children[2].layout.width = '430px'
    interact_c.widget.children[3].description = 'Update Plot/s'
    interact_c.widget.children[3].button_style = 'danger'
    interact_c.widget.children[3].style.button_color = '#3973ac'
    interact_c.widget.children[3].layout.margin = '10px 10px 40px 180px' # top/right/bottom/left

<a id='comparing_plot_atc_l2'></a>
### 5.2. Comparing - ICOS Level 2 Atmospheric Data ---> Plot

In [ ]:
#Call function to display widgets for the corresponding output type:
create_widgets_comparing()

<div style="text-align: right"> 
    <a href="#comparing_atc_l2">[Back to comparing]</a>
    &ensp;&ensp;
    <a href="#introduction">Back to top</a>
</div>
<br>
<br>
<br>

<a id='icos_level1_atc'></a>

## 6. ICOS Level 1 - Near Real Time (NRT) - Atmospheric Data
This part is dedicated on ICOS Near Real Time (Level 1) Atmospheric Data. Data that belong to this category has not yet undergone all the quality checks. It is possible to plot Near Real Time Data together with Level 2 Data for a given station and compare. 

<br>
<br>
<div style="text-align: right"> 
    <a href="#introduction">Back to top</a>
</div>

### 6.1. ICOS Level 1 - Near Real Time (NRT) - Atmospheric Data ---> Python Code

<br>
<div style="text-align: right"> 
    <a href="#icos_level1_atc_plot">[Go to plot]</a>
</div>

In [ ]:
def plot_icos_single_station_L1_L2_binary(data_df_list, station_info_dict, tracer_info_dict, color):
    
    """
    Project:         'ICOS Carbon Portal'
    Created:          Tue May 07 10:30:00 2018
    Last Changed:     Tue May 07 10:30:00 2019
    Version:          1.0.0
    Author(s):        Karolina
    
    Description:      Function that takes as input a list of data dataframes, a dictionary with information for 
                      the ICOS station, a dictionary with tracer information and a string with the color for 
                      the plot line. This implementation is for one selected ICOS station and one selected tracer. 
                      The function creates an interactive Bokeh plot with the contents of the dataframes and
                      returns a Bokeh Figure (plot).
                      
                      
    Input parameters: 1. ICOS Level-2 tracer Atmospheric Data Dataframe
                         (var_name: 'data_df_list', var_type: List of dataframes)
                      2. Dictionary with info for ICOS stations
                         (var_name: 'station_info_dict', var_type: Dictionary)
                      3. Dictionary with tracer info (i.e. tracer name, tracer unit)
                         (var_name: "tracer_info_dict", var_type: Dictionary)
                      4. Selected Color for plot line
                         (var_name: "color", var_type: String)

    Output:           Bokeh Plot
    
    """
    
    #Import modules to create figure:
    from bokeh.plotting import figure
    from bokeh.models import ColumnDataSource, HoverTool, Label, Legend
    from datetime import datetime

    
    #Dictionaries for subscript/superscript transformations of numbers:
    SUB = str.maketrans("0123456789", "₀₁₂₃₄₅₆₇₈₉")
    SUP = str.maketrans("0123456789", "⁰¹²³⁴⁵⁶⁷⁸⁹")

    #Create a figure object:
    p = figure(plot_width=900,
               plot_height=500,
               x_axis_label='Time (UTC)', 
               y_axis_label=tracer_info_dict['tracer_info'].replace(' mixing ratio (dry mole fraction)', '').translate(SUB)+
               ' (' +tracer_info_dict['tracer_unit'].translate(SUP) + ')',
               x_axis_type='datetime',
               title = tracer_info_dict['tracer_info'].replace(' mixing ratio (dry mole fraction)', '').translate(SUB)+
               ' - Continuous air ( '+
               station_info_dict['station_name']+', '+#.encode('latin1').decode('utf8')+', '+
               station_info_dict['station_country']+', '+
               station_info_dict['station_sampling_height']+' m. a. g. l.)' ,
               tools='pan,box_zoom,wheel_zoom,reset,save')
    
    
    #Create an empty list that will store the legend info:
    legend_it = []

   

    #Extract time and tracer values for every data level:
    x1 = data_df_list[0].index.values
    y1 = data_df_list[0][tracer_info_dict['tracer_info'].replace(' mixing ratio (dry mole fraction)', '').lower()].values
    x2 = data_df_list[1].index.values
    y2 = data_df_list[1][tracer_info_dict['tracer_info'].replace(' mixing ratio (dry mole fraction)', '').lower()].values

    #Create a circle and line glyph for the values of every emission category:
    r0 = p.circle(x1, y1, radius=.12, color=color, alpha=0.5)
    r1 = p.line(x1, y1, line_width=2, line_dash='dotted', line_alpha=0.5, color=color, name='L1, '+station_info_dict['station_code']+' ('+station_info_dict['station_sampling_height']+')')
    r2 = p.circle(x2, y2, radius=.12, color=color)
    r3 = p.line(x2, y2, line_width=1, color=color, name='L2, '+station_info_dict['station_code']+' ('+station_info_dict['station_sampling_height']+')')
    
    #Add the name and glyph info (i.e. colour and marker type) to the legend:
    legend_it.append(('Level 1 - '+station_info_dict['station_code']+' ('+station_info_dict['station_sampling_height']+')', [r1]))
    
    #Add the name and glyph info (i.e. colour and marker type) to the legend:
    legend_it.append(('Level 2 - '+station_info_dict['station_code']+' ('+station_info_dict['station_sampling_height']+')', [r3]))



    #Add tooltip on hover:
    p.add_tools(HoverTool(tooltips=[
        ('Station','$name'),
        ('Time (UTC)','@x{%Y-%m-%d %H:%M:%S}'),
        (tracer_info_dict['tracer_info'].replace(' mixing ratio (dry mole fraction)','').translate(SUB),'@y{0.f}'),
        ],
        formatters={
            '@x'      : 'datetime', # use 'datetime' formatter for 'date' field
            },
        # display a tooltip whenever the cursor is vertically in line with a glyph
        mode='vline'
        ))    

    #Create legend:
    legend = Legend(items=legend_it, location= 'bottom_center')
    legend.orientation = 'horizontal'
    legend.click_policy='hide'
    legend.spacing = 10 #sets the distance between legend entries

    #Set title attributes:
    p.title.align = 'center'
    p.title.text_font_size = '13pt'
    p.title.offset = 15

    #Set axis label font style:
    p.xaxis.axis_label_text_font_style = 'normal'
    p.yaxis.axis_label_text_font_style = 'normal'
    p.xaxis.axis_label_standoff = 15 #Sets the distance of the label from the x-axis in screen units
    p.yaxis.axis_label_standoff = 15 #Sets the distance of the label from the y-axis in screen units

    #Set the copyright label position:
    label_opts = dict(x=0, y=10,
                      x_units='screen', y_units='screen')

    #Create a label object and format it:
    caption1 = Label(text="© ICOS ERIC", **label_opts)
    caption1.text_font_size = '8pt'
    
    #Deactivate hover-tool, which is by default active:
    p.toolbar.active_inspect = None

    #Add label to plot:
    p.add_layout(caption1, 'below')
    
    #Add legend to figure:
    p.add_layout(legend, 'below')

    #return plot:
    return p
    

In [ ]:
def update_icos_single_station_plot_L1_L2_binary(data_obj_id_L1_ls, data_obj_id_L2_ls, station_code, station_sampl_height, tracer, color):
    
    """
    Project:         'ICOS Carbon Portal'
    Created:          Tue May 07 10:30:00 2018
    Last Changed:     Tue May 07 10:30:00 2019
    Version:          1.0.0
    Author(s):        Karolina
    
    Description:      Function that takes as input a list of ICOS Level-1 data object IDs, a list
                      of ICOS Level-2 data object IDs, the tracer type and the plot color. The
                      function reads the corresponding ICOS Level-1 and Level-2 Atmospheric Tracer
                      data files for every data object ID into sepparate pandas dataframes. Every
                      data file produces two sepparate pandas dataframes; metadata dataframe and
                      data dataframe. These dataframes are then set as input parameters to a
                      plot function, that returns a Bokeh Figure (plot).
                      The Bokeh Figure is then returned as output.
                      
                      
    Input parameters: 1. List with sublists of ICOS Level-1 data object IDs
                         (var_name: 'data_obj_id_L1_ls', var_type: List)
                      2. List with sublists of ICOS Level-2 data object IDs
                         (var_name: 'data_obj_id_L2_ls', var_type: List)
                      3. Tracer/gas, e.g. 'co2'
                         (var_name: 'tracer', var_type: String)
                      4. Plot Color
                         (var_name: 'color', var_type: String)
                      

    Output:           Bokeh Plot
    
    """
    
    #Import modules:
    from icoscp.cpb.dobj import Dobj
    
    #Create dictionary to store tracer info:
    tracer_info_dict = {}
    
    #Create dict to store the station info:
    station_info_dict = {}
    
    #Get pandas dataframe with all ICOS stations:
    icos_stations_df = get_coords_icos_stations_atc()
    
    #Create a file object from the 1st object in the data object id list:
    file = Dobj(data_obj_id_L1_ls[0])  
    
    #Get the tracer description:
    tracer_info_dict['tracer_info'] = file.info[1].valueType.loc[file.info[1].colName==tracer].values[0]
    
    #Get tracer unit:
    tracer_info_dict['tracer_unit'] = file.info[1].unit.loc[file.info[1].colName==tracer].values[0]
    
    #Get station info:
    station_info_dict['station_name'] = icos_stations_df.stationName.loc[icos_stations_df.stationId==station_code].values[0]
    station_info_dict['station_code'] = station_code
    station_info_dict['station_sampling_height'] = station_sampl_height
    station_info_dict['station_country_code'] = icos_stations_df.Country.loc[icos_stations_df.stationId==station_code].values[0]
    station_info_dict['station_country'] = get_country_fullname_from_iso3166_2char(station_info_dict['station_country_code'])
    station_info_dict['station_lat'] = icos_stations_df.lat.loc[icos_stations_df.stationId==station_code].values[0]
    station_info_dict['station_lon'] = icos_stations_df.lon.loc[icos_stations_df.stationId==station_code].values[0]
    
    
    
    
    
    #Create list to store the data dataframes of all data object IDs:
    L1_data_df_ls = []
    
    #Loop through every data object ID in the list:
    for dobjid_L1 in data_obj_id_L1_ls:
        
        #Get a pandas dataframe with all the columns for the selected data-object id:
        obs_data_df_L1 = Dobj(dobjid_L1).get()
        
        #Add data dataframe of the current data object ID to the list:
        L1_data_df_ls.append(obs_data_df_L1)
        
    #Concatenate data dataframes to one dataframe:
    data_L1_df = pd.concat(L1_data_df_ls)
    
    #Add column with datetime object:
    data_L1_df['DateTime'] = pd.to_datetime(data_L1_df['TIMESTAMP'], unit='ms') 

    #Create a copy of the dataframe and set "DateTime" as index:
    data_df_ind_L1 = data_L1_df.copy().set_index('DateTime')

    #Sort the dataframe index in ascending order:
    data_df_ind_L1.sort_index(inplace=True)
        
    
    #Create list to store the data dataframes of all data object IDs:
    L2_data_df_ls = []
    
    #Loop through every data object ID in the list:
    for dobjid_L2 in data_obj_id_L2_ls:
        
        #Get a pandas dataframe with all the columns for the selected data-object id:
        obs_data_df_L2 = Dobj(dobjid_L2).get()
        
        #Add data dataframe of the current data object ID to the list:
        L2_data_df_ls.append(obs_data_df_L2)
        
    #Concatenate data dataframes to one dataframe:
    data_L2_df = pd.concat(L2_data_df_ls)

   #Add column with datetime object:
    data_L2_df['DateTime'] = pd.to_datetime(data_L2_df['TIMESTAMP'], unit='ms') 

    #Create a copy of the dataframe and set "DateTime" as index:
    data_df_ind_L2 = data_L2_df.copy().set_index('DateTime')

    #Sort the dataframe index in ascending order:
    data_df_ind_L2.sort_index(inplace=True)
       
    #Plot station:
    p = plot_icos_single_station_L1_L2_binary([data_df_ind_L1, data_df_ind_L2],
                                              station_info_dict,
                                              tracer_info_dict,
                                              color)
    
    #Show plot
    show(p)
   

In [ ]:
def update_icos_single_station_plot_LX_binary(data_obj_id_LX_ls, station_code, station_sampl_height, tracer, color, level):
    
    """
    Project:         'ICOS Carbon Portal'
    Created:          Tue May 07 10:30:00 2018
    Last Changed:     Tue May 07 10:30:00 2019
    Version:          1.0.0
    Author(s):        Karolina
    
    Description:      Function that takes as input a list of ICOS Level-X data object IDs, the
                      tracer type, the plot color and the ICOS data level. The function reads
                      the corresponding ICOS Level-1 Tracer data files for every data object ID
                      into sepparate pandas dataframes. Every data file produces two sepparate
                      pandas dataframes; metadata dataframe and data dataframe.
                      These dataframes are then set as input parameters to a plot function,
                      that returns a Bokeh Figure (plot).
                      The Bokeh Figure is then returned as output.
                      
                      
    Input parameters: 1. List with sublists of ICOS Level-X data object IDs
                         (var_name: 'data_obj_id_LX_ls', var_type: List)
                      2. Tracer/gas, e.g. 'co2'
                         (var_name: 'tracer', var_type: String)
                      3. Plot Color
                         (var_name: 'color', var_type: String)
                      4. ICOS Atmospheric Data Level
                         (var_name: 'level', var_type: Integer)
                      

    Output:           Bokeh Plot
    
    """
    
    #Import modules:
    from icoscp.cpb.dobj import Dobj
    
    #Create dictionary to store tracer info:
    tracer_info_dict = {}
    
    #Create dict to store the station info:
    station_info_dict = {}
    
    #Create list to store the data dataframes of all data object IDs:
    data_df_ls = []
    
    
    #Get pandas dataframe with all ICOS stations:
    icos_stations_df = get_coords_icos_stations_atc()
    
    #Create a file object from the 1st object in the data object id list:
    file = Dobj(data_obj_id_LX_ls[0])  
    
    #Get the tracer description:
    tracer_info_dict['tracer_info'] = file.info[1].valueType.loc[file.info[1].colName==tracer].values[0]
    
    #Get tracer unit:
    tracer_info_dict['tracer_unit'] = file.info[1].unit.loc[file.info[1].colName==tracer].values[0]
    
    #Get station info:
    station_info_dict['station_name'] = icos_stations_df.stationName.loc[icos_stations_df.stationId==station_code].values[0]
    station_info_dict['station_code'] = station_code
    station_info_dict['station_sampling_height'] = station_sampl_height
    station_info_dict['station_country_code'] = icos_stations_df.Country.loc[icos_stations_df.stationId==station_code].values[0]
    station_info_dict['station_country'] = get_country_fullname_from_iso3166_2char(station_info_dict['station_country_code'])
    station_info_dict['station_lat'] = icos_stations_df.lat.loc[icos_stations_df.stationId==station_code].values[0]
    station_info_dict['station_lon'] = icos_stations_df.lon.loc[icos_stations_df.stationId==station_code].values[0]
    
    
    

    
    
    #Loop through every data object ID in the list:
    for dobjid in data_obj_id_LX_ls: 
        
        #Get a pandas dataframe with all the columns for the selected dataobject id:
        obs_data_df = Dobj(dobjid).get()
        
        #Add data dataframe of the current data object ID to the list:
        data_df_ls.append(obs_data_df)
        
    #Concatenate data dataframes to one dataframe:
    data_LX_df = pd.concat(data_df_ls)
    
        
    #Call plotting function:
    p = plot_icos_single_station_binary(data_LX_df, station_info_dict, tracer_info_dict, level, color)
    
    #Show plot
    show(p)


In [ ]:
def create_widgets_L1():
    
    """
    Project:         'ICOS Carbon Portal'
    Created:          Tue May 07 10:30:00 2018
    Last Changed:     Tue May 07 10:30:00 2019
    Version:          1.0.0
    Author(s):        Karolina
    
    Description:      Function that creates a set of widgets; a station dropdown list, a tracer
                      dropdown list, a color-picker, a checkbox and a button. The function populates
                      the dropdown lists with values and outputs the result.
                      
                      
    Input parameters: No Input Parameter(s)

    Output:           Python Widgets
    
    """
    
    #Dictionary for subscript transformations of numbers:
    SUB = str.maketrans("0123456789", "₀₁₂₃₄₅₆₇₈₉")

    #Create lookup dataframes:
    df_lookup_L1 = create_lookup_df_atc_L1() #ICOS Level 1 Atmosphere Data
    df_lookup_L2 = create_lookup_df_atc_L2() #ICOS Level 2 Atmosphere Data
    
    #Create a list including all tracers (e.g. CO2, CO, CH4)
    tracers_L1 = df_lookup_L1.variable.unique().tolist()
    
    #reverse list order:
    tracers_L1.reverse()

    #Create widgets:
    tracer_L1 = Dropdown(options = tracers_L1)
    station_L1 =  Dropdown(options = create_station_labels(df_lookup_L1))

    #Function that calls functions to update the plot/s and/or map,
    #based on the selected tracer, station and color:
    def update_plot_func(Tracer, Station, color, Level, Citation):
        
        #Get tracer short:
        tracer = Tracer.replace(' mixing ratio (dry mole fraction)', '').lower()
        
        #If "Add Level 2 Data" is not selected and if L1-data is available for the specific station and tracer:
        if(Level==False):
            
            #Get a list of L1 data obect URLs that refer to the selected station and tracer:
            data_obj_url_L1_ls = df_lookup_L1.dobj.loc[(df_lookup_L1.stationId==Station[0]) &
                                                       (df_lookup_L1.height==Station[1]) &
                                                       (df_lookup_L1.variable==Tracer)].values
            
            #If L1-data is available for the selected tracer and station:
            if(data_obj_url_L1_ls.size>0):
                
                #Get a list of data object IDs (L1-data):
                data_obj_id_L1_ls = [data_obj_url_L1_ls[i].replace('https://meta.icos-cp.eu/objects/', '')
                                     for i in range(data_obj_url_L1_ls.size)]
                
                #Call function to return plot for the selected station (Level 1 Data):
                update_icos_single_station_plot_LX_binary(data_obj_id_L1_ls, Station[0], Station[1], tracer, color, 1)
                
                #If the "citation" checkbox is checked:
                if(Citation):

                    #Get a list with citation info for every ICOS Level-1 data object:
                    cit_ls_L2 = [get_icos_citation(dobj).cit.iloc[0] for dobj in data_obj_url_L1_ls]

                    #Print citation title:
                    print('\n\n\033[1m' + 'Data Citation:' +  '\033[0m')

                    #Loop through all citations:
                    for cit in cit_ls_L2:

                        #Print data object citation:
                        printmd("<sub>"+cit+"</sub>")
            
            
            #If no L1-data is available for the selected tracer and station:
            else:
                print('\033[0;31;1m '+ 'No '+tracer.upper().translate(SUB)+' Level-1 data available for the selected station' +'\033[0;31;0m\n\n')
                
           
        
        #If "Add Level 2 Data" is selected:
        elif(Level==True):
            
            #Get a list of L1 data object URLs that refer to the selected station and tracer:
            data_obj_url_L1_ls = df_lookup_L1.dobj.loc[(df_lookup_L1.stationId==Station[0]) &
                                                       (df_lookup_L1.height==Station[1]) &
                                                       (df_lookup_L1.variable==Tracer)].values
            
            #Get a list of L2 data object URLs that refer to the selected station and tracer:
            data_obj_url_L2_ls = df_lookup_L2.dobj.loc[(df_lookup_L2.stationId==Station[0]) &
                                                       (df_lookup_L2.height==Station[1]) &
                                                       (df_lookup_L2.variable==Tracer)].values
            
            #If Level-1 & Level-2 data are available for the selected tracer and station:
            if((data_obj_url_L1_ls.size>0) & (data_obj_url_L2_ls.size>0)):
                
                #Get a list of data object IDs (L1-data):
                data_obj_id_L1_ls = [data_obj_url_L1_ls[j].replace('https://meta.icos-cp.eu/objects/', '')
                                     for j in range(data_obj_url_L1_ls.size)]
                
                #Get a list of data object IDs (L2-data):
                data_obj_id_L2_ls = [data_obj_url_L2_ls[k].replace('https://meta.icos-cp.eu/objects/', '')
                                     for k in range(data_obj_url_L2_ls.size)]
                
                #Call function to return plot for the selected station (Level 1 & 2 Data):
                update_icos_single_station_plot_L1_L2_binary(data_obj_id_L1_ls, data_obj_id_L2_ls, Station[0], Station[1], tracer, color)
                
                #If the "citation" checkbox is checked:
                if(Citation):
                    
                    #Get a list with citation info for every ICOS Level-1 data object:
                    cit_ls_L1 = [get_icos_citation(dobj).cit.iloc[0] for dobj in data_obj_url_L1_ls]

                    #Get a list with citation info for every ICOS Level-2 data object:
                    cit_ls_L2 = [get_icos_citation(dobj).cit.iloc[0] for dobj in data_obj_url_L2_ls]

                    #Concatenate citation lists to one list: 
                    cit_ls = cit_ls_L1 + cit_ls_L2

                    #Print citation:
                    print('\n\n\033[1m' + 'Data Citation:' +  '\033[0m')

                    #Loop through all citations:
                    for cit in cit_ls:
                        
                        #Print data object citation:
                        printmd("<sub>"+cit+"</sub>")
            
            
            #If only Level-1 data are available for the selected tracer and station:
            elif((data_obj_url_L1_ls.size>0) & (data_obj_url_L2_ls.size<1)):
                
                #Print message:
                print('\033[0;31;1m '+ 'No Level-2 data available yet ...' +'\033[0;31;0m\n\n')
                
                #Get a list of data object IDs (L1-data):
                data_obj_id_L1_ls = [data_obj_url_L1_ls[j].replace('https://meta.icos-cp.eu/objects/', '')
                                     for j in range(data_obj_url_L1_ls.size)]
                
                
                #Call function to return plot for the selected station (Level 1):
                update_icos_single_station_plot_LX_binary(data_obj_id_L1_ls, Station[0], Station[1], tracer, color, 1)
                
                #If the "citation" checkbox is checked:
                if(Citation):
                    
                    #Get a list with citation info for every ICOS Level-1 data object:
                    cit_ls_L1 = [get_icos_citation(dobj).cit.iloc[0] for dobj in data_obj_url_L1_ls]


                    #Print citation:
                    print('\n\n\033[1m' + 'Data Citation:' +  '\033[0m')

                    #Loop through all citations:
                    for cit in cit_ls_L1:
                        
                        #Print data object citation:
                        printmd("<sub>"+cit+"</sub>")
            
            
            #If only Level-2 data are available for the selected tracer and station:
            elif((data_obj_url_L1_ls.size<1) & (data_obj_url_L2_ls.size>0)):
                
                #Print message:
                print('\033[0;31;1m '+ 'No Level-1 data available ...' +'\033[0;31;0m\n\n')
                
                #Get a list of data object IDs (L2-data):
                data_obj_id_L2_ls = [data_obj_url_L2_ls[j].replace('https://meta.icos-cp.eu/objects/', '')
                                     for j in range(data_obj_url_L2_ls.size)]
                
                
                #Call function to return plot for the selected station (Level-2 Data):
                update_icos_single_station_plot_LX_binary(data_obj_id_L2_ls, Station[0], Station[1], tracer, color, 2)
                
                #If the "citation" checkbox is checked:
                if(Citation):
                    
                    #Get a list with citation info for every ICOS Level-1 data object:
                    cit_ls_L2 = [get_icos_citation(dobj).cit.iloc[0] for dobj in data_obj_url_L2_ls]


                    #Print citation:
                    print('\n\n\033[1m' + 'Data Citation:' +  '\033[0m')

                    #Loop through all citations:
                    for cit in cit_ls_L2:
                        
                        #Print data object citation:
                        printmd("<sub>"+cit+"</sub>")
            
            
            #If no L1-data or L2-data are available for the selected tracer and station:
            else:
                print('\033[0;31;1m '+ 'No Level-1 or Level-2 data available for the selected tracer and station at present.\nTry a new combination!' +'\033[0;31;0m\n\n')


    #Create function that contains a box of widgets:
    interact_c = interact_manual(update_plot_func,
                                 Tracer=tracer_L1,
                                 Station=station_L1,
                                 color=ColorPicker(concise=False,description='Pick a color',value='#3973ac',
                                                   disabled=False),
                                 Level=Checkbox(value=False, description='Add Level 2 Data', disabled=False),
                                 Citation=Checkbox(value=True, description='Citation', disabled=False))

    #Set the font of the widgets included in interact_manual:
    interact_c.widget.children[0].layout.width = '430px'
    interact_c.widget.children[0].layout.margin = '40px 2px 2px 2px'
    interact_c.widget.children[1].layout.width = '430px'
    interact_c.widget.children[2].layout.width = '430px'
    interact_c.widget.children[3].layout.width = '430px'
    interact_c.widget.children[3].layout.margin = '12px 2px 2px 2px'
    interact_c.widget.children[4].layout.width = '430px'
    interact_c.widget.children[5].description = 'Update Plot'
    interact_c.widget.children[5].button_style = 'danger'
    interact_c.widget.children[5].style.button_color = '#3973ac'
    interact_c.widget.children[5].layout.margin = '10px 10px 40px 180px' # top/right/bottom/left

<a id='icos_level1_atc_plot'></a>
### 6.2. ICOS Level 1 - Near Real Time (NRT) - Atmospheric Data ---> Plot
<br>


In [ ]:
##Call function to display widgets for the corresponding output type:
create_widgets_L1()

<div style="text-align: right"> 
    <a href="#icos_level1_atc">[Back to ICOS-Level1-Atmospheric-Data]</a>
    &ensp;&ensp;
    <a href="#introduction">Back to top</a>
</div>
<br>
<br>
<br>

<a id='create_nb_account_info'></a>

## 7. Get Access to ICOS Jupyter Notebook Developing Environment
If you wish to extend the functionality of the already existent ICOS notebooks or develop your own Jupyter notebook, send an email with your request to <jupyter-info@icos-cp.eu>.

<br>
<div style="text-align: right"> 
    <a href="#introduction">Back to top</a>
</div>
<br>
<br>
<br>